In [39]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import defaultdict
import sys
sys.path.append('../data')


In [34]:
class INaturalistSSL():
    def __init__(self, root, filename, transform=None, 
                 target_transform=None, download=False, test=False):
        #super(INaturalistSSL, self).__init__(root, version, target_type=target_type, 
                                             #transform=transform, target_transform=target_transform,
                                            #download=download)
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.img_path = []
        self.labels = []
        self.test = test
        self.dist = None
        
        with open(filename) as f:
            for line in f:
                self.img_path.append(os.path.join(root, line.split()[0]))
                self.labels.append(int(line.split()[1]))
        
    def __len__(self):
        return len(self.labels)
    
    def get_data_dist(self):
        if self.dist == None:
            self.dist = defaultdict(int)
            for label in self.labels:
                self.dist[label] += 1
        return sorted(self.dist.items(), key=lambda x: x[1], reverse=True)

    def __getitem__(self, idx):
        img_name = self.img_path[idx]
        label = self.labels[idx]
        img = Image.open(img_name).convert('RGB')
        
        ret = []
        x1 = self.transform(img)
        ret.append(x1)
        if not self.test:
            x2 = self.transform(img)
            ret.append(x2)
        
        # Add two extra channles if there's only one depth
        if x1.size()[0] == 1:
            print(x1.shape)
            x1 = torch.stack([x1, x1, x1], dim=0).squeeze()
            x2 = torch.stack([x2, x2, x2], dim=0).squeeze()
        elif x1.size()[0] == 2:
            print(f'X equals 2 : {x1.shape}')
            x1 = torch.stack([x1, torch.zeros(1, x1.size()[1], x1.size()[2])], dim=0)
            x2 = torch.stack([x2, torch.zeros(1, x2.size()[1], x2.size()[2])], dim=0)
        return ret, label
 

In [31]:
baseline_accs = {499: 2, 1445: 3, 3189: 2, 6840: 1, 4755: 2, 3330: 3, 524: 3, 6281: 3, 1591: 2, 3398: 1, 5313: 2, 301: 2, 941: 2, 6958: 2, 2327: 3, 4856: 2, 4373: 2, 3169: 1, 5414: 3, 7351: 2, 7742: 3, 5600: 1, 1801: 2, 810: 3, 3501: 1, 460: 1, 1264: 2, 4037: 3, 4576: 2, 686: 3, 85: 3, 6101: 3, 2055: 3, 2769: 2, 1689: 3, 2036: 1, 1676: 1, 2685: 3, 4087: 1, 725: 2, 2652: 2, 6934: 2, 5038: 1, 2753: 1, 5568: 2, 4756: 3, 2049: 3, 6447: 3, 2750: 3, 6568: 3, 8116: 3, 798: 2, 3665: 3, 7592: 3, 6868: 1, 7878: 2, 4910: 3, 1943: 3, 3549: 1, 779: 3, 2751: 2, 6938: 3, 7441: 2, 6442: 1, 4920: 3, 4887: 2, 5211: 2, 3948: 2, 5180: 3, 1504: 2, 8022: 2, 1277: 2, 3576: 2, 2325: 3, 6443: 3, 3941: 2, 7390: 1, 607: 3, 632: 3, 6465: 2, 2101: 2, 948: 3, 2993: 2, 3406: 3, 2939: 2, 2957: 2, 814: 3, 313: 2, 384: 1, 5086: 3, 5805: 1, 1808: 2, 645: 2, 1780: 3, 938: 3, 4471: 1, 4386: 3, 4305: 2, 3174: 1, 4135: 3, 3602: 2, 4355: 3, 669: 3, 3502: 2, 4535: 2, 1281: 2, 5595: 3, 3007: 2, 6163: 3, 6125: 2, 5064: 3, 1909: 3, 3216: 1, 3900: 2, 7104: 2, 3286: 2, 1675: 2, 8032: 2, 2732: 3, 1804: 2, 773: 2, 6091: 3, 3538: 1, 1972: 2, 4672: 2, 7249: 3, 1730: 1, 3345: 2, 6337: 3, 2006: 2, 4103: 2, 1687: 2, 6271: 1, 7597: 2, 7782: 2, 2494: 2, 931: 3, 5109: 1, 3641: 1, 6221: 2, 4222: 2, 2748: 2, 4673: 3, 1307: 2, 1254: 3, 5139: 1, 3036: 3, 6621: 3, 4978: 2, 3133: 1, 897: 3, 2128: 3, 1910: 2, 3911: 3, 6937: 2, 821: 2, 6499: 1, 7996: 2, 7046: 2, 2452: 3, 1787: 3, 5967: 1, 4607: 1, 845: 2, 3413: 3, 7480: 1, 5125: 2, 707: 3, 4928: 1, 5701: 2, 89: 1, 2367: 3, 3956: 1, 6739: 2, 2565: 2, 243: 3, 5413: 3, 6589: 2, 3347: 1, 7754: 2, 710: 3, 1179: 3, 1930: 2, 3382: 3, 1928: 2, 5150: 2, 4562: 2, 885: 3, 6314: 2, 7751: 1, 5527: 2, 542: 2, 5295: 2, 2236: 3, 6414: 1, 2491: 3, 665: 2, 4173: 2, 6108: 1, 3023: 3, 145: 2, 2627: 1, 5749: 2, 4317: 3, 2496: 3, 2932: 1, 723: 3, 1784: 3, 3933: 2, 6392: 2, 249: 3, 3904: 2, 2195: 3, 2226: 2, 4089: 3, 6405: 3, 1105: 1, 1078: 2, 6323: 3, 4515: 3, 1269: 3, 5685: 1, 2633: 3, 1522: 2, 1969: 3, 5362: 3, 5501: 2, 5113: 3, 5933: 2, 4467: 2, 41: 1, 1947: 2, 2927: 2, 1709: 2, 5953: 1, 5093: 2, 6835: 2, 390: 2, 6446: 2, 6810: 2, 1956: 2, 6836: 2, 2528: 3, 3135: 2, 7825: 2, 2959: 2, 2322: 3, 820: 3, 4544: 2, 4369: 1, 5996: 3, 1959: 2, 6783: 3, 5128: 2, 6313: 1, 992: 2, 1076: 2, 4933: 1, 8028: 2, 2995: 1, 6278: 1, 3783: 2, 4367: 1, 3861: 1, 6859: 2, 5817: 2, 2465: 3, 6570: 2, 6012: 2, 2319: 3, 596: 3, 1843: 1, 7369: 1, 4900: 3, 5748: 1, 5487: 1, 2731: 3, 5569: 2, 496: 1, 764: 3, 8063: 2, 3146: 2, 7815: 3, 1426: 1, 322: 2, 3008: 1, 2272: 2, 5930: 1, 5609: 1, 6843: 1, 3594: 3, 8038: 2, 1466: 2, 4027: 2, 5169: 1, 3820: 1, 2992: 2, 1706: 2, 1535: 2, 2975: 2, 370: 1, 6913: 3, 3803: 1, 6123: 3, 6833: 2, 5353: 3, 5950: 1, 4526: 3, 4581: 3, 1881: 2, 921: 2, 1188: 2, 3703: 2, 3840: 2, 6186: 1, 6969: 1, 1290: 2, 755: 2, 4160: 1, 626: 3, 1693: 2, 6362: 2, 7230: 1, 6550: 2, 2252: 2, 2820: 2, 1260: 1, 4403: 3, 946: 2, 3545: 2, 5311: 2, 6230: 3, 4967: 3, 2994: 3, 2053: 1, 6462: 2, 6500: 1, 6029: 1, 5281: 3, 332: 2, 1802: 3, 1447: 3, 657: 3, 2389: 2, 6079: 1, 306: 2, 5750: 1, 3705: 3, 3125: 2, 6648: 2, 660: 2, 170: 2, 638: 3, 5241: 3, 3247: 2, 2714: 2, 8072: 3, 6581: 2, 6256: 1, 355: 2, 80: 1, 5869: 2, 4058: 1, 5299: 1, 512: 2, 3524: 1, 5585: 3, 3405: 3, 7967: 3, 6165: 3, 919: 2, 3138: 2, 6215: 1, 6831: 1, 3476: 3, 5608: 2, 61: 2, 5761: 2, 2131: 3, 2546: 3, 2948: 2, 8002: 2, 1461: 1, 3315: 1, 3076: 3, 5357: 1, 927: 1, 2124: 3, 1654: 2, 2326: 3, 3681: 2, 7532: 1, 661: 2, 4523: 3, 3970: 1, 848: 3, 6422: 2, 2268: 3, 4682: 2, 2846: 2, 6331: 2, 4018: 2, 4366: 2, 3624: 2, 3304: 2, 2709: 2, 6356: 3, 2097: 3, 471: 2, 3603: 1, 4789: 2, 2510: 2, 6912: 2, 7766: 3, 3225: 1, 8089: 2, 2318: 3, 4939: 2, 1578: 2, 4204: 1, 6044: 2, 4127: 2, 3163: 2, 5099: 1, 5473: 2, 2333: 2, 2171: 2, 6450: 2, 2281: 1, 4215: 2, 2280: 1, 8013: 2, 4645: 3, 5327: 3, 3115: 1, 12: 2, 2038: 1, 2793: 3, 6857: 2, 7724: 2, 2299: 2, 1208: 1, 6134: 2, 2462: 2, 6869: 1, 3289: 2, 1397: 2, 4790: 1, 3223: 3, 1858: 2, 4822: 3, 1938: 2, 4011: 3, 3981: 1, 5614: 1, 3579: 2, 6017: 1, 6699: 1, 2698: 2, 7265: 2, 599: 2, 2965: 2, 5510: 2, 588: 3, 1232: 3, 7450: 3, 5728: 1, 7274: 3, 7668: 2, 6325: 3, 1964: 2, 1658: 1, 4372: 2, 7194: 3, 5517: 3, 5062: 1, 2886: 2, 6415: 2, 1131: 3, 6978: 1, 594: 2, 2254: 3, 3401: 1, 5450: 3, 1724: 2, 7101: 2, 2742: 3, 4038: 1, 5294: 2, 5046: 2, 3785: 1, 7855: 2, 5712: 3, 3848: 2, 6991: 3, 4552: 2, 5023: 2, 1255: 3, 4110: 2, 8035: 2, 2919: 2, 1696: 2, 4619: 3, 3612: 1, 4171: 2, 7326: 3, 1140: 2, 6375: 2, 2253: 2, 1576: 2, 3591: 2, 6132: 3, 3847: 2, 5966: 1, 507: 3, 6723: 2, 4993: 2, 5364: 1, 7223: 3, 8062: 2, 4918: 2, 2608: 2, 4685: 3, 7799: 1, 6416: 3, 2662: 2, 2588: 1, 2063: 2, 3064: 2, 2954: 2, 3601: 2, 5689: 1, 6636: 1, 2483: 3, 2642: 1, 158: 3, 2596: 2, 5360: 2, 3166: 2, 681: 3, 5852: 3, 1739: 1, 5833: 3, 3478: 2, 3605: 3, 2481: 3, 1293: 3, 380: 2, 5008: 3, 5573: 2, 4543: 1, 5806: 3, 4940: 3, 4050: 2, 6172: 3, 121: 1, 5154: 2, 419: 2, 5197: 3, 5581: 3, 5644: 2, 3082: 3, 4125: 1, 6579: 3, 5148: 3, 858: 2, 5045: 3, 1641: 2, 7378: 1, 1020: 2, 352: 2, 3040: 3, 6279: 2, 4174: 1, 2798: 3, 7947: 2, 1677: 3, 2375: 3, 7411: 2, 690: 2, 833: 2, 2364: 1, 982: 2, 3170: 2, 1299: 2, 1982: 2, 3328: 1, 4617: 2, 1723: 1, 4723: 1, 5956: 2, 5229: 3, 573: 2, 6930: 2, 5142: 2, 6424: 1, 2369: 3, 6383: 3, 5746: 3, 6451: 2, 729: 3, 1842: 3, 2347: 3, 5929: 1, 617: 2, 494: 2, 5057: 3, 429: 1, 197: 2, 5459: 3, 7294: 2, 3148: 2, 6047: 3, 3331: 2, 2096: 1, 4525: 2, 7893: 2, 4298: 1, 232: 1, 2593: 2, 4957: 2, 461: 1, 1807: 2, 4744: 2, 1326: 1, 3646: 3, 1404: 3, 4253: 1, 2154: 2, 4714: 2, 204: 1, 1623: 2, 5548: 1, 6496: 2, 2416: 3, 4013: 3, 5396: 3, 976: 3, 6673: 2, 7885: 1, 902: 3, 6427: 2, 7990: 3, 6758: 1, 2242: 2, 1707: 2, 6700: 3, 3220: 2, 5209: 2, 5784: 2, 1120: 2, 5117: 1, 5627: 2, 3694: 2, 4435: 2, 1233: 2, 1627: 3, 3380: 2, 2800: 2, 8107: 2, 7103: 2, 7467: 2, 2869: 2, 7064: 3, 4521: 2, 3611: 3, 7492: 2, 4643: 3, 7670: 1, 1509: 1, 5495: 2, 7968: 2, 3338: 1, 5379: 3, 4625: 3, 5236: 2, 1479: 1, 3467: 2, 6419: 3, 2514: 3, 934: 2, 5076: 1, 552: 2, 3208: 1, 4033: 3, 7361: 2, 4175: 3, 7917: 3, 2454: 2, 7466: 2, 6237: 1, 4624: 1, 4059: 1, 4580: 2, 5612: 2, 3742: 2, 5380: 2, 2856: 2, 4346: 1, 1754: 2, 6433: 2, 1710: 3, 1574: 3, 2782: 1, 6702: 2, 2424: 3, 2620: 3, 3389: 2, 3016: 3, 2881: 2, 1985: 2, 8099: 2, 2289: 3, 5191: 3, 5653: 1, 883: 2, 7636: 2, 4771: 2, 2917: 2, 478: 1, 3865: 2, 2582: 1, 2046: 1, 4081: 1, 3951: 2, 4740: 2, 4165: 1, 6792: 1, 1586: 2, 1879: 2, 2879: 3, 6802: 3, 1759: 2, 775: 1, 294: 3, 2857: 2, 7115: 1, 5734: 2, 1602: 3, 5030: 2, 1613: 1, 2885: 2, 978: 3, 2789: 1, 323: 2, 7498: 2, 6059: 3, 1839: 3, 5363: 2, 5589: 3, 1606: 1, 2699: 3, 7925: 2, 2421: 1, 5765: 2, 988: 3, 8068: 2, 3623: 1, 4158: 3, 64: 3, 4216: 3, 2500: 3, 5443: 2, 886: 3, 6884: 2, 2345: 2, 6236: 2, 7479: 3, 1572: 2, 7067: 1, 1314: 3, 6390: 2, 7280: 2, 1944: 3, 3557: 1, 719: 1, 1824: 2, 6825: 2, 2087: 2, 6817: 1, 8001: 3, 4146: 2, 8131: 1, 6801: 2, 4754: 2, 6858: 1, 1415: 2, 1917: 3, 7266: 2, 6519: 2, 6580: 2, 5015: 3, 6947: 3, 8121: 2, 3589: 1, 4: 2, 5003: 2, 6387: 2, 1058: 2, 2291: 1, 4731: 2, 1092: 2, 6350: 2, 6106: 2, 7954: 2, 1487: 2, 5323: 3, 908: 2, 5433: 3, 1536: 2, 3751: 3, 5282: 2, 8077: 2, 3094: 2, 683: 2, 7810: 3, 6355: 3, 1129: 3, 7760: 3, 7594: 1, 6445: 2, 2065: 1, 1549: 1, 7971: 2, 5420: 1, 4748: 3, 4591: 1, 5: 2, 6103: 2, 2381: 3, 4658: 2, 6479: 1, 3744: 1, 1795: 3, 3776: 2, 5618: 2, 1028: 2, 3461: 2, 604: 2, 4780: 2, 2716: 3, 4021: 2, 349: 2, 5035: 2, 4837: 2, 4824: 1, 2805: 2, 5599: 2, 140: 1, 619: 2, 5071: 1, 6717: 2, 5537: 2, 3638: 2, 2014: 2, 4404: 3, 6319: 2, 7765: 1, 1251: 2, 4994: 2, 2482: 3, 118: 2, 4203: 2, 83: 1, 2002: 3, 1095: 2, 2480: 1, 5629: 3, 2505: 2, 4070: 1, 4644: 1, 4783: 1, 3072: 2, 5482: 3, 3167: 3, 581: 1, 4465: 3, 1668: 2, 7436: 2, 4016: 2, 6338: 2, 7691: 2, 3922: 1, 7382: 3, 5858: 1, 6743: 1, 2032: 1, 7510: 2, 493: 3, 3095: 1, 5445: 2, 7050: 1, 411: 2, 417: 1, 1345: 3, 5500: 2, 1436: 1, 3004: 3, 4295: 1, 8060: 2, 2672: 2, 4196: 1, 555: 2, 7052: 3, 2182: 2, 2538: 2, 6168: 2, 1037: 2, 2931: 1, 25: 3, 7160: 2, 745: 1, 4947: 2, 880: 2, 943: 3, 5278: 2, 2945: 2, 640: 3, 7922: 2, 7652: 1, 2177: 1, 936: 1, 2211: 1, 382: 3, 6607: 2, 4768: 2, 4976: 3, 1614: 3, 3578: 2, 3540: 1, 2595: 2, 786: 2, 439: 1, 6948: 1, 7569: 2, 667: 1, 6662: 3, 6649: 2, 728: 3, 395: 3, 1428: 2, 4934: 2, 4949: 1, 952: 3, 4154: 3, 5732: 2, 3755: 3, 2865: 2, 1384: 2, 917: 2, 6145: 2, 4007: 2, 4718: 3, 6730: 1, 4370: 3, 6599: 3, 5006: 2, 6460: 2, 6089: 1, 2654: 1, 1380: 3, 8128: 2, 7798: 1, 6841: 2, 10: 1, 4703: 2, 4720: 2, 7977: 3, 1976: 1, 709: 3, 6879: 2, 1365: 2, 2961: 3, 1531: 3, 6565: 1, 2427: 2, 2828: 1, 4819: 3, 7235: 2, 1050: 2, 609: 3, 7058: 2, 2404: 3, 1902: 1, 2132: 1, 3317: 2, 2109: 3, 2746: 1, 8024: 2, 3122: 3, 5312: 3, 1923: 2, 5727: 2, 3770: 2, 837: 1, 5992: 2, 7295: 3, 750: 2, 4867: 3, 4836: 2, 7243: 2, 5397: 1, 5339: 2, 5890: 2, 1960: 3, 1237: 2, 171: 1, 5634: 2, 7541: 1, 5976: 1, 5507: 1, 4664: 2, 7293: 2, 5083: 1, 4315: 2, 4578: 3, 92: 2, 6746: 1, 7575: 1, 5020: 3, 3284: 2, 3543: 1, 2338: 2, 2418: 2, 7538: 3, 365: 2, 6530: 2, 4466: 1, 3297: 3, 2964: 3, 4784: 2, 6800: 2, 7731: 2, 5664: 2, 4999: 3, 2552: 1, 4010: 1, 711: 3, 2126: 2, 1595: 2, 4475: 3, 1955: 2, 5697: 2, 7620: 2, 275: 1, 6995: 2, 1768: 3, 577: 3, 3015: 1, 2467: 2, 1385: 1, 4811: 1, 5848: 1, 7247: 1, 2933: 2, 6078: 3, 5747: 1, 3924: 2, 6077: 2, 2987: 1, 5395: 2, 1468: 2, 415: 1, 1182: 2, 1604: 2, 4410: 1, 5092: 3, 7201: 3, 3973: 3, 1751: 2, 199: 2, 3510: 1, 1207: 2, 403: 1, 5135: 1, 7601: 1, 5544: 1, 2646: 1, 305: 2, 1476: 2, 7829: 2, 910: 1, 5526: 1, 857: 2, 133: 3, 1799: 2, 2158: 1, 7776: 2, 2403: 1, 2337: 2, 7248: 2, 6065: 2, 4883: 3, 8125: 1, 3635: 2, 2515: 1, 7421: 1, 3721: 3, 2586: 1, 6659: 2, 887: 3, 4606: 3, 7081: 1, 5928: 2, 5132: 3, 849: 2, 6367: 3, 1722: 2, 4249: 1, 5074: 2, 4417: 3, 391: 3, 5972: 2, 5963: 1, 753: 2, 5319: 2, 510: 3, 1115: 2, 5084: 2, 1592: 3, 5922: 1, 2516: 1, 5650: 2, 4898: 3, 6864: 2, 7586: 2, 3256: 1, 1691: 1, 4668: 3, 7657: 3, 6403: 2, 6386: 2, 2303: 2, 4988: 2, 3447: 2, 6820: 3, 911: 2, 3614: 1, 2197: 2, 2056: 1, 4281: 3, 3745: 2, 2406: 3, 4006: 2, 189: 1, 1500: 2, 1612: 2, 2164: 2, 1776: 2, 5376: 2, 6304: 1, 5804: 2, 253: 2, 2086: 2, 2902: 1, 1563: 1, 1057: 2, 5914: 2, 3468: 2, 4286: 1, 6741: 3, 6245: 2, 3117: 2, 359: 1, 6412: 1, 2559: 3, 5309: 3, 4309: 2, 4293: 3, 5964: 3, 1987: 1, 7447: 1, 1419: 2, 246: 2, 2844: 1, 5217: 3, 8134: 3, 5663: 3, 7708: 2, 3774: 1, 6645: 2, 3156: 2, 1734: 1, 8124: 2, 3671: 1, 2804: 1, 7783: 1, 8130: 1, 5980: 1, 2311: 1, 2775: 1, 2571: 2, 3779: 1, 3176: 2, 4666: 2, 6501: 2, 4959: 1, 3772: 2, 299: 1, 4020: 1, 3811: 1, 5797: 2, 2307: 2, 2273: 2, 1053: 2, 5684: 2, 5332: 2, 42: 1, 1123: 3, 6733: 1, 903: 2, 2190: 2, 6585: 1, 368: 2, 7980: 3, 6624: 1, 5518: 3, 4214: 2, 6291: 3, 2934: 2, 2811: 2, 3944: 2, 6981: 1, 2724: 2, 4083: 3, 5389: 1, 760: 2, 6532: 1, 4180: 3, 7683: 2, 4572: 3, 7279: 1, 2741: 1, 2188: 1, 5572: 1, 7743: 1, 4104: 2, 6016: 1, 2021: 1, 7771: 3, 5279: 2, 8085: 3, 6176: 2, 7380: 1, 2023: 1, 3796: 2, 5906: 1, 7315: 2, 4812: 2, 295: 3, 3: 2, 841: 3, 1647: 2, 1564: 3, 7307: 1, 4319: 1, 4770: 1, 7209: 3, 69: 3, 242: 1, 6826: 2, 2918: 2, 194: 3, 3184: 2, 3370: 2, 2233: 2, 8066: 1, 8091: 3, 3388: 1, 4210: 2, 7000: 2, 981: 2, 933: 2, 3573: 2, 3723: 2, 4324: 2, 1629: 1, 5144: 3, 3621: 2, 2450: 3, 7989: 3, 6651: 2, 6812: 1, 2560: 1, 913: 3, 7719: 3, 7506: 1, 4858: 2, 5170: 2, 4815: 2, 7904: 3, 2743: 2, 6354: 2, 2207: 2, 8061: 3, 8037: 2, 3384: 1, 3729: 2, 2022: 3, 7001: 2, 4713: 2, 6529: 2, 548: 3, 3925: 1, 1763: 3, 1600: 2, 1534: 3, 1906: 2, 2640: 1, 1246: 2, 5305: 2, 353: 3, 2332: 3, 1435: 3, 59: 2, 3863: 1, 5704: 1, 1186: 2, 2518: 2, 1750: 2, 3955: 3, 6285: 2, 63: 2, 2244: 2, 5577: 1, 1757: 3, 1356: 2, 7984: 2, 200: 3, 2000: 3, 515: 1, 7099: 1, 3071: 1, 1597: 2, 1279: 3, 7959: 2, 1292: 3, 1634: 3, 2738: 2, 6088: 1, 479: 2, 991: 3, 2125: 2, 5582: 2, 5710: 1, 1626: 2, 6080: 1, 3349: 3, 6173: 3, 2966: 2, 1672: 3, 3882: 1, 1319: 3, 2179: 3, 6931: 2, 5044: 2, 1958: 2, 6152: 2, 1704: 3, 5214: 2, 4143: 2, 2723: 1, 2562: 2, 1918: 2, 6919: 2, 2431: 2, 3121: 2, 5776: 2, 5018: 2, 4047: 1, 3818: 2, 7231: 2, 4763: 2, 7358: 2, 5200: 2, 7516: 1, 7475: 2, 7208: 3, 1155: 1, 8029: 3, 5284: 2, 5320: 1, 205: 2, 1630: 1, 551: 2, 7169: 1, 5485: 3, 1271: 2, 1849: 2, 4497: 3, 480: 2, 2706: 2, 4909: 1, 7661: 1, 4659: 2, 484: 3, 60: 1, 1049: 1, 2161: 3, 3643: 3, 6629: 2, 5557: 3, 5690: 2, 2328: 2, 5116: 1, 7513: 3, 5640: 2, 4752: 2, 5104: 3, 6601: 1, 4611: 3, 6524: 1, 7791: 2, 1024: 3, 1803: 1, 3626: 3, 6051: 2, 4996: 3, 6936: 2, 958: 1, 559: 2, 1471: 2, 5237: 3, 6226: 1, 2849: 2, 7229: 2, 5391: 3, 7108: 3, 5770: 2, 2561: 3, 1913: 2, 971: 3, 3291: 3, 4142: 3, 1702: 1, 5541: 2, 4626: 1, 5936: 2, 4716: 3, 5662: 3, 6339: 2, 4577: 2, 7368: 1, 4634: 2, 5574: 1, 4147: 3, 5975: 1, 6430: 1, 5404: 2, 5308: 3, 8048: 2, 7125: 3, 5787: 1, 4430: 1, 1648: 2, 794: 1, 2517: 3, 7653: 2, 4494: 1, 6598: 1, 4056: 2, 6975: 1, 4704: 1, 664: 3, 653: 2, 52: 3, 3808: 1, 7240: 1, 2850: 3, 6459: 2, 3660: 3, 5555: 1, 7774: 1, 7082: 1, 4057: 3, 2873: 2, 546: 2, 862: 2, 4459: 3, 7165: 1, 4347: 1, 7868: 1, 652: 2, 5607: 1, 3046: 2, 7298: 2, 752: 3, 4508: 3, 922: 1, 737: 3, 3994: 1, 2283: 2, 6578: 1, 4656: 2, 7816: 2, 457: 3, 231: 3, 6234: 2, 1077: 1, 2363: 3, 5803: 3, 8014: 1, 4024: 2, 5131: 3, 6076: 3, 7414: 2, 7123: 2, 300: 2, 5715: 3, 4991: 2, 7897: 1, 5670: 2, 4302: 3, 1508: 2, 2971: 2, 5820: 1, 5467: 3, 2366: 3, 1786: 3, 6085: 1, 6653: 3, 1493: 1, 5604: 3, 3894: 2, 5026: 3, 1351: 2, 2660: 1, 4671: 2, 2943: 1, 4742: 3, 3416: 2, 6127: 1, 4484: 2, 6543: 2, 4827: 1, 2758: 2, 5025: 3, 6917: 1, 131: 2, 220: 2, 6196: 2, 4267: 2, 6092: 1, 8132: 3, 659: 3, 7845: 1, 4447: 2, 2350: 2, 4695: 2, 7114: 2, 3077: 2, 3586: 1, 7645: 1, 4932: 2, 2310: 3, 5692: 3, 2397: 1, 846: 1, 233: 2, 1262: 3, 2357: 2, 1459: 2, 2502: 1, 383: 1, 3161: 2, 289: 3, 492: 2, 6555: 2, 4667: 2, 631: 3, 2301: 1, 4651: 2, 5306: 1, 4962: 1, 3927: 1, 6406: 3, 4861: 2, 5210: 3, 5022: 2, 5127: 1, 8000: 2, 6290: 2, 2475: 2, 2391: 2, 7239: 3, 1490: 3, 7034: 2, 1079: 2, 793: 1, 4880: 2, 7915: 2, 718: 2, 6935: 3, 276: 2, 3443: 1, 3432: 2, 6704: 2, 4826: 2, 1794: 1, 1821: 3, 4041: 2, 4546: 2, 3885: 1, 4904: 2, 4810: 1, 1256: 3, 5165: 1, 2360: 1, 3041: 3, 90: 1, 2866: 1, 879: 2, 3717: 2, 3710: 2, 4594: 2, 7705: 1, 7349: 2, 1354: 3, 801: 2, 4022: 3, 6171: 1, 241: 1, 2501: 3, 2815: 1, 1825: 1, 7485: 3, 4094: 2, 470: 2, 5894: 2, 7024: 1, 3615: 2, 1840: 3, 5054: 1, 5356: 2, 863: 2, 2034: 2, 4279: 1, 6866: 1, 1363: 2, 3073: 1, 894: 3, 5189: 1, 1528: 1, 6185: 3, 4870: 1, 4921: 3, 3664: 1, 56: 2, 264: 2, 5275: 3, 2342: 2, 7407: 3, 873: 2, 6057: 1, 6167: 2, 2163: 2, 6735: 1, 1887: 3, 2859: 2, 6637: 1, 178: 1, 262: 2, 5385: 3, 2529: 3, 2704: 3, 2330: 3, 160: 2, 2892: 2, 7318: 1, 4975: 2, 8047: 3, 3662: 2, 6377: 3, 5711: 2, 6690: 1, 4989: 2, 2543: 1, 6020: 1, 5027: 2, 1258: 3, 7711: 1, 7119: 1, 6309: 1, 433: 3, 409: 1, 5538: 3, 5222: 2, 5430: 2, 3043: 2, 945: 3, 3982: 1, 7587: 1, 6205: 2, 2011: 2, 5783: 2, 5412: 1, 6921: 3, 8045: 2, 3686: 2, 4424: 3, 129: 2, 6514: 2, 4263: 3, 3231: 2, 5968: 2, 1594: 3, 7987: 1, 1660: 3, 2: 2, 6890: 1, 1335: 1, 2836: 1, 6953: 2, 4794: 1, 5103: 3, 4113: 3, 5540: 2, 288: 2, 4727: 1, 5441: 2, 228: 2, 1434: 3, 1267: 1, 2378: 2, 5638: 3, 2958: 1, 4121: 2, 722: 1, 75: 3, 7095: 2, 8101: 3, 3471: 2, 6471: 1, 227: 1, 2739: 1, 5739: 1, 3014: 2, 7199: 1, 3089: 1, 5268: 2, 2656: 3, 3350: 2, 3625: 2, 1369: 2, 4730: 1, 3892: 1, 4036: 2, 6681: 3, 1980: 3, 1238: 3, 738: 2, 3001: 1, 7644: 2, 6513: 2, 1004: 2, 1489: 3, 1069: 1, 161: 2, 7013: 3, 531: 2, 3351: 2, 465: 1, 7716: 2, 5123: 1, 8004: 2, 1852: 2, 4136: 2, 2506: 3, 3244: 1, 6218: 2, 3496: 2, 5100: 1, 5416: 2, 8040: 2, 447: 2, 4230: 2, 156: 2, 6891: 2, 1187: 2, 5402: 2, 8106: 2, 2485: 1, 1864: 2, 5826: 2, 2951: 3, 5520: 2, 291: 1, 3730: 1, 4868: 2, 491: 3, 2390: 2, 7088: 3, 7484: 3, 2428: 3, 536: 1, 2484: 1, 5654: 2, 1192: 2, 7839: 2, 7068: 3, 5910: 1, 4820: 3, 1515: 2, 7834: 1, 4700: 2, 6667: 2, 5528: 2, 8122: 2, 5977: 2, 5383: 1, 4970: 3, 7999: 2, 329: 2, 7997: 3, 1124: 1, 5421: 1, 2210: 2, 5494: 2, 6830: 2, 3688: 3, 3758: 1, 7489: 1, 3536: 2, 398: 1, 1713: 3, 2294: 3, 5937: 2, 6976: 1, 3874: 2, 5403: 2, 2446: 1, 7589: 1, 6011: 2, 1320: 3, 6945: 2, 842: 2, 693: 3, 7347: 1, 6815: 2, 4743: 2, 3322: 1, 7268: 1, 7851: 1, 7789: 3, 2720: 3, 4946: 2, 6541: 1, 522: 1, 1266: 2, 3901: 1, 1196: 2, 1514: 1, 1454: 3, 3920: 1, 4587: 3, 4721: 2, 1674: 2, 3667: 1, 5231: 3, 1939: 1, 7818: 1, 5935: 1, 1016: 2, 2700: 2, 865: 1, 6159: 1, 6816: 2, 2874: 3, 5107: 1, 1005: 3, 2765: 1, 6867: 2, 2331: 3, 2051: 1, 4764: 3, 7459: 3, 5345: 2, 1405: 1, 3029: 1, 5693: 1, 877: 2, 3760: 1, 2497: 2, 4144: 2, 1295: 3, 2058: 3, 5702: 3, 285: 1, 394: 2, 2238: 2, 2563: 3, 1491: 1, 58: 2, 3728: 1, 1618: 1, 592: 1, 2838: 1, 4873: 2, 3265: 1, 2631: 2, 147: 1, 7796: 2, 2928: 2, 483: 2, 5374: 2, 7733: 2, 3091: 1, 1605: 2, 1141: 3, 238: 3, 2983: 1, 1282: 2, 590: 1, 2632: 2, 528: 2, 6576: 3, 4908: 3, 7656: 1, 183: 1, 6609: 3, 799: 2, 1003: 2, 6805: 2, 2414: 2, 4751: 3, 2727: 2, 4529: 1, 8135: 1, 689: 1, 4972: 3, 682: 3, 6980: 1, 3151: 2, 4225: 2, 2499: 3, 373: 2, 3855: 2, 274: 3, 341: 1, 6776: 3, 5619: 3, 2105: 1, 4627: 2, 1667: 2, 7860: 3, 1898: 3, 2287: 1, 1994: 1, 2947: 2, 4101: 3, 3939: 1, 6400: 2, 7051: 1, 1450: 1, 6955: 1, 4639: 2, 859: 2, 5183: 2, 1287: 2, 2100: 2, 2306: 2, 1950: 2, 84: 2, 8005: 2, 6623: 2, 3698: 1, 4039: 3, 6357: 3, 7207: 1, 3883: 2, 1474: 3, 5157: 2, 4806: 3, 3011: 1, 1311: 1, 4097: 2, 2227: 1, 7667: 2, 3287: 2, 381: 3, 762: 3, 4938: 3, 4560: 2, 5631: 2, 6546: 2, 4241: 3, 2120: 3, 2249: 1, 1174: 2, 2093: 1, 3926: 2, 2113: 2, 3726: 2, 3620: 1, 3030: 2, 4559: 2, 5096: 1, 4750: 3, 4126: 2, 3431: 3, 2064: 2, 4853: 1, 990: 3, 45: 1, 2170: 2, 3407: 1, 7593: 2, 953: 2, 3032: 1, 1091: 1, 400: 1, 1027: 2, 1541: 1, 3749: 2, 5058: 1, 4722: 3, 3385: 2, 962: 3, 5205: 2, 273: 1, 5434: 2, 7501: 1, 5155: 2, 7483: 2, 5293: 2, 79: 2, 5419: 2, 906: 2, 1373: 3, 6967: 1, 2649: 1, 6904: 1, 7386: 2, 5453: 1, 3553: 2, 5539: 2, 4678: 1, 6533: 1, 1755: 2, 5490: 3, 1494: 2, 3158: 2, 4240: 1, 2167: 1, 7993: 2, 965: 1, 2304: 2, 6148: 2, 6665: 2, 316: 3, 2020: 2, 3798: 1, 5695: 1, 280: 2, 4604: 3, 4767: 2, 6244: 1, 2537: 2, 1010: 3, 1234: 2, 1327: 1, 5464: 3, 5230: 2, 3120: 3, 4694: 1, 2266: 1, 6789: 2, 4251: 3, 2541: 3, 3180: 2, 3310: 2, 6675: 1, 4288: 1, 5089: 2, 1853: 2, 1178: 1, 4579: 1, 2313: 1, 5911: 3, 7698: 1, 2372: 3, 5426: 1, 1616: 3, 6613: 1, 5248: 2, 1631: 1, 4992: 3, 751: 3, 1312: 3, 4042: 3, 519: 2, 7446: 3, 5195: 2, 2180: 1, 7305: 2, 6315: 2, 7138: 2, 3655: 1, 5463: 1, 2339: 3, 704: 3, 3984: 1, 7424: 3, 7619: 2, 2580: 2, 826: 2, 180: 2, 4066: 2, 4416: 3, 2587: 2, 2155: 1, 6476: 2, 3849: 1, 1395: 3, 3479: 1, 7772: 1, 169: 2, 4538: 2, 7976: 2, 1390: 1, 4728: 2, 4955: 1, 589: 2, 1912: 3, 8010: 1, 3587: 1, 1995: 1, 4227: 2, 5815: 1, 104: 1, 87: 2, 68: 3, 561: 2, 593: 2, 296: 1, 5556: 2, 5543: 1, 386: 3, 3657: 3, 1655: 1, 4385: 1, 2664: 2, 2457: 2, 6107: 2, 5949: 1, 1220: 1, 5336: 1, 5626: 2, 434: 3, 7211: 3, 989: 1, 2425: 1, 6764: 1, 3440: 1, 1181: 1, 6170: 1, 1806: 3, 7521: 1, 4653: 3, 5918: 2, 3182: 2, 5166: 1, 202: 2, 1507: 2, 4004: 2, 5287: 1, 850: 3, 3222: 3, 870: 1, 7288: 2, 4351: 1, 256: 2, 5753: 1, 6050: 1, 5310: 1, 1341: 1, 173: 3, 5243: 2, 5280: 3, 2584: 2, 1371: 3, 6197: 1, 2321: 3, 3766: 2, 6871: 1, 7935: 3, 7908: 1, 5920: 3, 6644: 2, 4161: 2, 741: 1, 2028: 2, 6752: 1, 4877: 2, 2764: 2, 8027: 3, 2712: 3, 7287: 3, 7975: 1, 5553: 2, 4914: 2, 7890: 1, 1891: 1, 5208: 1, 5809: 3, 813: 1, 7966: 3, 634: 1, 3896: 2, 290: 2, 5351: 2, 440: 3, 3731: 1, 4603: 2, 5871: 2, 4941: 3, 2420: 2, 4429: 3, 267: 3, 832: 2, 5119: 1, 2812: 1, 1692: 1, 3598: 2, 3718: 1, 1880: 2, 5546: 1, 6175: 2, 6749: 2, 4799: 1, 2223: 3, 7352: 3, 5769: 1, 344: 2, 7430: 2, 6821: 2, 3978: 1, 3079: 1, 3724: 2, 2008: 2, 3483: 2, 830: 3, 3976: 3, 3829: 1, 1700: 1, 4338: 1, 2744: 1, 3129: 1, 1063: 2, 6634: 2, 5315: 2, 4030: 2, 1217: 1, 475: 3, 6005: 1, 6333: 2, 7222: 1, 1775: 2, 7317: 2, 7882: 2, 8030: 3, 7458: 1, 2634: 2, 3648: 2, 1984: 1, 4724: 2, 6707: 2, 8059: 2, 5682: 2, 3827: 2, 2639: 2, 979: 1, 3173: 1, 3190: 3, 4791: 3, 937: 3, 1682: 3, 6683: 2, 7524: 1, 5423: 2, 48: 3, 1206: 3, 1962: 3, 1657: 3, 222: 3, 5375: 2, 3390: 2, 4093: 1, 5766: 3, 4735: 2, 7647: 3, 7881: 1, 6277: 2, 1153: 2, 1735: 1, 4796: 3, 1162: 2, 5115: 2, 5757: 2, 3024: 2, 3832: 1, 143: 1, 5530: 3, 2851: 2, 2637: 3, 5649: 2, 6957: 1, 1442: 1, 4411: 1, 1214: 2, 21: 1, 2847: 3, 7995: 2, 3929: 2, 7033: 2, 1736: 2, 4570: 3, 3235: 2, 675: 3, 1974: 1, 3571: 2, 5881: 1, 502: 2, 1670: 1, 5052: 2, 4425: 2, 8133: 2, 838: 1, 622: 2, 3051: 3, 2735: 1, 6010: 1, 2261: 2, 2960: 2, 1239: 2, 4283: 1, 1274: 2, 2578: 3, 230: 1, 2271: 3, 818: 1, 2773: 1, 777: 2, 5233: 2, 108: 2, 4064: 3, 7419: 3, 5011: 2, 6944: 2, 4760: 1, 7678: 1, 4191: 2, 6943: 1, 782: 3, 4184: 2, 7025: 2, 7354: 1, 4092: 1, 815: 3, 4128: 2, 4687: 1, 6411: 1, 6492: 1, 973: 2, 7122: 1, 4782: 2, 5768: 2, 119: 2, 5213: 3, 699: 2, 2371: 3, 5028: 1, 1760: 2, 7426: 2, 3868: 2, 3298: 3, 7151: 1, 742: 1, 1855: 2, 7811: 1, 237: 3, 251: 1, 5108: 1, 2251: 3, 13: 1, 2267: 2, 920: 3, 6522: 2, 4757: 3, 4496: 2, 8017: 1, 1643: 3, 2407: 2, 1925: 1, 2925: 2, 74: 2, 1056: 1, 1109: 2, 6371: 2, 7930: 2, 3066: 2, 4906: 3, 3421: 2, 2173: 1, 5470: 2, 5532: 2, 4998: 1, 2159: 2, 65: 2, 3789: 1, 379: 3, 2408: 2, 2784: 1, 5927: 2, 4473: 1, 5688: 2, 3554: 1, 4593: 1, 5439: 2, 458: 2, 529: 1, 1128: 2, 676: 1, 6767: 1, 625: 2, 6796: 1, 5993: 2, 2864: 2, 1875: 3, 3733: 1, 239: 1, 4610: 1, 2540: 1, 5921: 1, 1039: 3, 455: 1, 5273: 3, 616: 1, 1635: 1, 3782: 2, 217: 2, 5335: 3, 7627: 3, 4732: 2, 4608: 2, 5760: 2, 1942: 2, 2285: 2, 5839: 2, 7303: 2, 888: 3, 1309: 3, 4912: 3, 7311: 1, 6073: 1, 7643: 1, 2952: 2, 281: 2, 1933: 3, 7522: 2, 1408: 3, 4273: 2, 7940: 2, 3541: 2, 6225: 1, 802: 3, 2153: 2, 1012: 2, 485: 3, 6006: 3, 5264: 1, 1525: 3, 839: 3, 5039: 2, 4711: 1, 7511: 2, 5465: 1, 7448: 2, 6670: 1, 2602: 1, 1991: 2, 4649: 1, 3252: 2, 4828: 1, 2157: 2, 5742: 2, 1845: 2, 6034: 3, 3613: 2, 6794: 2, 2141: 1, 8141: 1, 1761: 1, 6385: 2, 6536: 2, 2536: 3, 3314: 2, 4766: 3, 1412: 1, 1135: 1, 3777: 3, 488: 1, 7648: 2, 2612: 1, 7551: 2, 2657: 1, 5377: 3, 1486: 1, 2108: 3, 975: 1, 5800: 1, 6482: 3, 452: 2, 3531: 3, 1185: 2, 8105: 1, 3872: 1, 1348: 3, 4902: 3, 3377: 1, 1838: 2, 5990: 1, 3378: 1, 2577: 3, 2174: 2, 2103: 1, 4960: 3, 7129: 1, 4343: 1, 5981: 3, 6788: 1, 7094: 3, 4534: 1, 3273: 1, 4326: 1, 5451: 2, 2257: 3, 5857: 3, 3068: 2, 5411: 1, 7745: 2, 2005: 1, 7292: 2, 3969: 2, 7944: 2, 7195: 3, 6586: 2, 5719: 1, 3709: 2, 3201: 3, 6617: 2, 1590: 3, 7402: 3, 3198: 3, 4623: 2, 2315: 2, 3768: 2, 1951: 2, 4072: 1, 2532: 2, 2362: 2, 1503: 1, 3000: 1, 3784: 3, 3627: 3, 4683: 1, 5061: 1, 4916: 1, 1066: 2, 2876: 1, 6997: 3, 4616: 2, 5834: 2, 7150: 2, 326: 2, 4123: 1, 7177: 3, 4396: 2, 4565: 3, 3618: 2, 6784: 3, 4014: 2, 5989: 2, 3013: 2, 4028: 2, 3326: 3, 5973: 1, 32: 2, 6420: 2, 795: 2, 3997: 2, 8036: 1, 2694: 2, 5112: 2, 5471: 1, 5588: 1, 4879: 3, 2081: 2, 6630: 3, 3404: 2, 1690: 1, 6069: 1, 1088: 1, 1372: 2, 4851: 2, 338: 3, 951: 2, 1047: 2, 1863: 1, 16: 2, 3472: 1, 1410: 1, 7218: 2, 1110: 2, 5014: 3, 5007: 2, 4943: 1, 2907: 1, 3529: 2, 808: 2, 6121: 3, 5499: 2, 3815: 2, 4155: 3, 4635: 2, 7553: 2, 2090: 2, 4252: 1, 6142: 2, 3991: 1, 7692: 1, 2263: 3, 286: 2, 7757: 2, 1431: 3, 7529: 2, 955: 1, 1922: 1, 5387: 1, 5388: 2, 3264: 2, 5798: 1, 4388: 2, 2316: 2, 4438: 2, 2767: 2, 2393: 2, 628: 3, 5908: 3, 224: 1, 2470: 2, 5511: 2, 5812: 1, 4583: 2, 6827: 2, 4153: 3, 5090: 1, 6786: 2, 5774: 3, 7121: 2, 2351: 2, 6178: 1, 4198: 2, 1401: 1, 6241: 3, 8094: 2, 4407: 1, 7943: 2, 3719: 2, 3983: 2, 54: 3, 2276: 3, 7787: 1, 1021: 3, 6143: 1, 1871: 2, 4590: 1, 4334: 2, 3147: 1, 914: 2, 6266: 1, 2270: 3, 3062: 1, 3162: 2, 6635: 2, 7238: 2, 7111: 2, 44: 2, 2835: 2, 1041: 2, 1032: 2, 6494: 2, 1598: 2, 2255: 1, 7474: 2, 1270: 2, 3481: 2, 188: 2, 5620: 1, 3403: 2, 7926: 1, 1247: 1, 6346: 1, 7296: 3, 7375: 2, 4769: 2, 4168: 1, 3042: 1, 3157: 2, 7118: 2, 1617: 3, 7869: 1, 4469: 2, 8079: 2, 1562: 1, 2508: 1, 851: 3, 4291: 2, 3194: 3, 5823: 1, 6971: 1, 5296: 3, 2772: 2, 6626: 3, 5875: 3, 7405: 2, 1064: 1, 5247: 1, 3938: 2, 4217: 2, 4005: 1, 1797: 3, 3369: 2, 1130: 1, 1000: 2, 7432: 1, 2138: 3, 5523: 2, 5792: 1, 3371: 2, 5256: 2, 7234: 3, 6900: 1, 2893: 1, 5651: 3, 1337: 2, 311: 3, 7143: 2, 5909: 3, 2202: 1, 4325: 2, 6873: 2, 4936: 3, 3778: 1, 6288: 1, 3402: 2, 5410: 3, 3140: 2, 3852: 3, 853: 3, 2111: 2, 1059: 3, 1383: 2, 2771: 2, 2045: 1, 3974: 3, 966: 2, 7985: 2, 6233: 3, 4117: 3, 4374: 2, 490: 2, 611: 2, 4930: 3, 3954: 2, 2077: 2, 4777: 2, 7427: 2, 62: 3, 2683: 1, 4015: 2, 1425: 3, 7938: 1, 6979: 1, 6924: 1, 8058: 3, 3975: 1, 2669: 3, 5181: 2, 3810: 1, 860: 2, 2047: 2, 5179: 3, 7758: 1, 706: 2, 3386: 1, 150: 3, 3687: 2, 7341: 2, 6832: 2, 1544: 1, 4802: 3, 716: 3, 1463: 2, 5121: 2, 4850: 3, 1715: 3, 7872: 3, 2068: 2, 3812: 1, 3992: 1, 1610: 2, 3736: 1, 2920: 2, 5134: 3, 2853: 1, 3374: 2, 6347: 1, 5917: 3, 5354: 2, 7383: 2, 766: 2, 4000: 3, 6512: 1, 3879: 3, 5864: 2, 1423: 2, 1745: 2, 293: 2, 4132: 2, 5736: 3, 3700: 2, 3397: 1, 4238: 2, 7598: 3, 2080: 2, 7948: 2, 6992: 1, 2915: 1, 4085: 1, 3887: 2, 5633: 2, 4982: 2, 4952: 1, 6475: 1, 3558: 1, 1628: 2, 7682: 2, 4061: 1, 656: 2, 2648: 1, 7703: 2, 5409: 1, 3692: 2, 8003: 2, 4792: 2, 5051: 2, 4840: 2, 3031: 1, 5700: 3, 6404: 3, 2734: 1, 3473: 1, 2622: 2, 4237: 3, 2525: 2, 7704: 2, 4178: 1, 5724: 2, 1977: 2, 4563: 2, 1031: 1, 4212: 1, 5178: 3, 3272: 1, 3677: 3, 4112: 2, 967: 2, 544: 3, 2419: 1, 7431: 1, 6181: 3, 5130: 3, 2135: 2, 7040: 2, 3260: 2, 2626: 2, 4446: 2, 3495: 2, 4332: 2, 7916: 3, 6680: 2, 4854: 3, 6745: 1, 4772: 1, 7831: 3, 1374: 1, 6852: 1, 5160: 3, 5959: 2, 1809: 2, 425: 1, 6239: 3, 7565: 2, 6229: 1, 831: 1, 8080: 1, 7392: 2, 1963: 2, 3905: 1, 7434: 3, 6886: 1, 6600: 2, 50: 1, 5188: 3, 4973: 2, 1793: 2, 893: 3, 6228: 2, 4460: 1, 6760: 2, 5371: 1, 2286: 3, 2400: 2, 3227: 1, 7928: 1, 1389: 3, 5841: 2, 6307: 1, 4642: 3, 983: 2, 4550: 2, 2513: 3, 5863: 1, 3835: 3, 3866: 2, 1169: 1, 4574: 3, 2827: 1, 2435: 2, 2104: 2, 3909: 1, 4981: 2, 3410: 1, 5957: 3, 464: 1, 563: 1, 1127: 3, 46: 3, 5825: 2, 6491: 2, 3341: 1, 3738: 2, 5234: 3, 3321: 2, 4364: 3, 7445: 3, 1705: 1, 1823: 2, 6528: 2, 2245: 1, 4729: 2, 1191: 1, 4397: 1, 4997: 1, 6710: 1, 1323: 3, 1927: 3, 4078: 3, 2430: 2, 7983: 2, 5475: 2, 51: 3, 1868: 1, 6133: 1, 1231: 3, 4589: 2, 2411: 3, 4054: 3, 8039: 2, 5101: 2, 5878: 1, 1349: 1, 4829: 1, 99: 3, 6998: 2, 3230: 1, 2978: 2, 6240: 2, 5661: 2, 2736: 1, 324: 1, 6007: 3, 2609: 3, 5955: 3, 5830: 2, 4478: 1, 2834: 2, 3191: 1, 7093: 1, 8138: 1, 6369: 1, 3696: 2, 7809: 1, 6678: 1, 5290: 2, 5204: 2, 1539: 3, 1767: 2, 445: 1, 7456: 1, 2225: 3, 2250: 2, 474: 1, 5919: 2, 2693: 1, 2985: 2, 7838: 3, 2667: 1, 4266: 2, 8111: 2, 5424: 1, 5945: 2, 572: 2, 5168: 1, 969: 3, 7425: 2, 4734: 3, 6703: 2, 4294: 2, 7453: 2, 5192: 2, 7566: 2, 2466: 1, 8127: 2, 2453: 1, 5227: 2, 876: 1, 4833: 2, 4055: 1, 6939: 2, 5325: 1, 696: 3, 6782: 1, 7693: 2, 6090: 1, 785: 2, 3454: 1, 3092: 2, 2432: 3, 5744: 1, 7452: 2, 3126: 1, 792: 1, 7007: 3, 3965: 1, 5625: 2, 1529: 1, 5601: 1, 5671: 3, 1218: 1, 1661: 1, 5703: 2, 6632: 1, 2713: 2, 2747: 2, 4265: 3, 4968: 3, 30: 3, 1685: 3, 776: 1, 4470: 2, 7183: 2, 2949: 1, 2297: 2, 4839: 2, 4357: 1, 7324: 2, 7374: 2, 5328: 3, 1393: 1, 5343: 2, 2861: 1, 6217: 2, 7028: 1, 514: 2, 684: 3, 7722: 2, 5737: 2, 1228: 2, 2663: 2, 1424: 3, 5559: 3, 2479: 2, 2458: 2, 7457: 1, 642: 1, 695: 1, 2464: 1, 4586: 1, 4322: 2, 1510: 3, 1699: 3, 1721: 3, 3781: 2, 303: 3, 1907: 1, 2095: 1, 3086: 2, 8093: 1, 1783: 2, 3356: 2, 1717: 1, 7302: 1, 2213: 2, 3663: 1, 3375: 1, 2490: 2, 1339: 1, 7267: 2, 5066: 3, 1018: 2, 4630: 1, 5951: 2, 393: 1, 5489: 2, 388: 2, 2625: 2, 3074: 2, 6502: 2, 4441: 2, 6360: 1, 3950: 1, 5745: 3, 4774: 1, 1798: 1, 2348: 3, 7886: 2, 2621: 2, 2278: 1, 6941: 3, 6742: 3, 442: 1, 8046: 3, 525: 2, 4086: 3, 1302: 1, 102: 3, 3052: 2, 6705: 2, 7946: 2, 6892: 2, 6270: 1, 37: 2, 6844: 2, 3123: 2, 697: 2, 1527: 2, 523: 1, 4509: 2, 909: 3, 2449: 3, 2193: 2, 6719: 1, 2474: 2, 5999: 2, 7391: 3, 6849: 2, 5897: 2, 2554: 3, 4019: 3, 1908: 2, 7690: 2, 790: 2, 2503: 2, 5326: 1, 172: 2, 4612: 3, 1149: 1, 3453: 2, 236: 3, 360: 2, 4206: 2, 4807: 3, 2192: 1, 5551: 3, 1559: 2, 1273: 3, 3186: 1, 1209: 2, 2115: 2, 1497: 1, 5221: 1, 2999: 1, 6693: 1, 2780: 1, 5400: 1, 5842: 2, 1338: 3, 4597: 3, 2855: 2, 4201: 1, 401: 2, 2802: 3, 6138: 2, 3005: 1, 1900: 3, 4913: 2, 5110: 3, 3293: 1, 3985: 2, 4710: 2, 5381: 2, 451: 2, 6692: 1, 443: 2, 221: 3, 6828: 3, 7381: 1, 8110: 1, 6301: 1, 5547: 2, 925: 1, 4638: 2, 2189: 2, 2963: 2, 828: 3, 2269: 2, 1596: 2, 1261: 2, 5756: 1, 2760: 2, 4990: 3, 2290: 1, 539: 2, 558: 1, 5720: 1, 8009: 2, 6458: 2, 1046: 2, 3039: 2, 6911: 2, 6994: 1, 2956: 1, 5033: 1, 3740: 2, 5491: 2, 7992: 3, 1469: 2, 5133: 3, 1387: 2, 7043: 1, 3278: 1, 128: 2, 4582: 3, 1136: 3, 4739: 2, 258: 1, 1107: 1, 852: 3, 5579: 2, 2581: 1, 1421: 2, 6112: 1, 759: 2, 926: 2, 3996: 1, 8076: 2, 3737: 2, 7073: 2, 7091: 2, 7964: 1, 4115: 1, 8065: 2, 6343: 2, 40: 1, 7502: 2, 3830: 2, 772: 2, 1712: 2, 3192: 2, 7803: 2, 4313: 2, 6154: 1, 4232: 3, 7112: 2, 1164: 2, 2981: 2, 3741: 1, 7903: 1, 5140: 2, 2461: 2, 6351: 3, 2136: 2, 3444: 3, 3026: 2, 3107: 2, 2469: 1, 100: 2, 3666: 1, 6220: 1, 4031: 2, 7180: 2, 3181: 1, 396: 3, 7462: 2, 2247: 1, 6353: 2, 2829: 2, 3219: 2, 3691: 1, 4269: 1, 4956: 1, 997: 1, 34: 2, 1243: 3, 2935: 1, 3816: 1, 5669: 3, 5182: 1, 7953: 1, 3679: 2, 3436: 1, 2329: 3, 1194: 1, 3308: 1, 2591: 2, 66: 1, 7476: 1, 3902: 1, 7357: 2, 5829: 2, 310: 2, 265: 3, 3415: 2, 4629: 3, 6118: 1, 3383: 2, 3155: 2, 8041: 2, 5725: 2, 1451: 3, 7941: 1, 4100: 3, 1446: 3, 6432: 3, 3684: 1, 5277: 1, 7022: 2, 5225: 1, 8092: 1, 8023: 2, 4310: 1, 1148: 2, 1391: 3, 5149: 1, 4514: 3, 3394: 2, 2110: 1, 1828: 1, 788: 3, 4157: 1, 6312: 2, 3409: 3, 27: 2, 2217: 3, 768: 2, 94: 1, 6757: 1, 1236: 2, 541: 2, 2659: 1, 4437: 2, 5466: 2, 2875: 1, 3500: 1, 2929: 2, 5534: 2, 5338: 3, 4229: 1, 2882: 3, 4745: 3, 4500: 1, 3175: 1, 6557: 2, 2766: 2, 8070: 1, 4032: 1, 3296: 3, 537: 1, 635: 2, 4661: 2, 371: 3, 620: 1, 7214: 3, 6587: 2, 6762: 1, 6455: 2, 2402: 2, 1257: 2, 7308: 1, 2472: 1, 5151: 1, 5019: 2, 3412: 2, 2140: 3, 4773: 3, 2883: 1, 4371: 2, 7559: 2, 2684: 1, 3596: 2, 3670: 1, 6202: 2, 7461: 1, 5199: 1, 136: 1, 7346: 2, 5854: 1, 4301: 1, 7008: 1, 565: 3, 5146: 2, 4622: 2, 2564: 1, 6032: 1, 5851: 2, 6549: 1, 2841: 1, 2186: 1, 3279: 1, 7290: 1, 4219: 1, 7478: 1, 4971: 1, 4690: 2, 212: 3, 376: 2, 2228: 1, 7514: 1, 1171: 2, 5896: 2, 4575: 3, 6734: 2, 2872: 3, 1638: 2, 5001: 3, 1517: 1, 3433: 2, 5512: 2, 3505: 1, 7936: 1, 7413: 3, 7360: 3, 5010: 2, 713: 2, 3817: 1, 7106: 1, 7986: 1, 6747: 2, 2674: 2, 7481: 2, 1202: 2, 517: 1, 591: 1, 2284: 3, 6531: 2, 7764: 2, 7482: 2, 1306: 3, 309: 3, 7321: 3, 7913: 2, 1268: 2, 4669: 2, 4318: 2, 124: 1, 7927: 2, 1837: 2, 1370: 3, 1180: 1, 279: 2, 6330: 1, 416: 2, 4390: 3, 1462: 2, 4063: 2, 7490: 1, 5288: 1, 5515: 2, 7684: 2, 4843: 2, 3792: 2, 1889: 2, 1048: 2, 4531: 2, 6294: 1, 968: 2, 578: 2, 1241: 3, 5602: 1, 8031: 3, 29: 2, 6064: 2, 2447: 2, 473: 3, 6793: 2, 5915: 2, 7018: 1, 420: 1, 6135: 2, 854: 1, 2786: 1, 7712: 3, 4555: 1, 7026: 2, 6687: 2, 7155: 3, 229: 3, 4268: 2, 4419: 2, 2302: 1, 4472: 1, 1321: 1, 7258: 1, 4795: 3, 2878: 1, 2946: 1, 6326: 3, 4567: 3, 6190: 1, 1294: 3, 4259: 2, 3814: 2, 3172: 1, 1111: 2, 3799: 2, 6780: 1, 7312: 2, 7820: 1, 1441: 2, 2665: 1, 8050: 2, 3137: 3, 7203: 1, 6525: 2, 3196: 2, 4166: 1, 7056: 1, 196: 3, 7286: 3, 7970: 2, 7454: 1, 5709: 1, 3059: 1, 4852: 3, 7205: 2, 375: 2, 1433: 2, 7746: 2, 387: 1, 7689: 1, 5266: 3, 698: 1, 3482: 1, 315: 1, 6893: 1, 5786: 2, 4450: 1, 2681: 1, 1989: 2, 1498: 3, 2774: 1, 135: 1, 3722: 3, 6130: 1, 3813: 2, 2888: 2, 898: 3, 3420: 2, 1316: 2, 1911: 3, 2001: 2, 1546: 3, 3393: 1, 2405: 1, 6210: 2, 881: 2, 6614: 2, 3335: 1, 4487: 1, 1581: 1, 369: 2, 2623: 1, 5261: 1, 4749: 2, 549: 2, 6399: 2, 1866: 1, 6781: 1, 1996: 3, 8081: 1, 915: 3, 1146: 3, 5647: 3, 5244: 2, 6829: 2, 1083: 1, 6903: 2, 2169: 1, 3193: 2, 5925: 2, 8087: 3, 5606: 3, 1133: 2, 7097: 1, 7613: 2, 6726: 1, 4849: 1, 8119: 2, 5560: 3, 320: 2, 6141: 1, 633: 1, 7717: 2, 2048: 1, 6124: 1, 1455: 1, 4348: 1, 3241: 1, 2050: 2, 110: 2, 4618: 1, 1242: 2, 3836: 1, 5772: 3, 3936: 3, 3897: 2, 5873: 2, 4502: 3, 1161: 2, 7679: 1, 72: 1, 2395: 2, 8043: 1, 5002: 2, 4491: 1, 2997: 1, 2172: 2, 4903: 3, 843: 1, 3559: 1, 4808: 3, 700: 1, 4674: 2, 2292: 3, 6423: 1, 2042: 2, 4995: 2, 2531: 3, 3188: 1, 5668: 2, 2520: 1, 6923: 2, 208: 1, 3215: 3, 4108: 1, 7227: 3, 4571: 2, 6365: 1, 5545: 3, 1235: 1, 4899: 2, 929: 1, 1082: 1, 1851: 1, 6902: 1, 7755: 1, 7898: 1, 7174: 1, 614: 1, 5605: 2, 2374: 1, 7972: 1, 5347: 1, 986: 1, 2512: 1, 767: 1, 6434: 1, 1857: 1, 5162: 1, 3585: 1, 2821: 1, 4228: 2, 6616: 1, 985: 1, 6538: 2, 2697: 2, 6712: 1, 5318: 3, 2203: 2, 3753: 1, 3932: 1, 1210: 1, 5431: 1, 2088: 1, 2982: 2, 3270: 2, 6960: 2, 2082: 1, 2487: 2, 2521: 2, 7345: 2, 6540: 1, 1376: 1, 3762: 1, 4717: 1, 3593: 1, 7253: 1, 240: 2, 7184: 2, 3102: 1, 5637: 2, 4391: 2, 3363: 2, 3462: 2, 4102: 2, 3862: 1, 422: 1, 3498: 2, 6036: 1, 7256: 2, 6200: 2, 2094: 2, 134: 2, 6684: 1, 5998: 2, 3070: 2, 5386: 3, 2840: 2, 3980: 1, 7923: 1, 6282: 1, 6063: 2, 3396: 2, 356: 1, 2757: 1, 495: 2, 7237: 1, 6061: 1, 4545: 1, 526: 1, 4074: 2, 6248: 1, 3651: 1, 268: 1, 8055: 2, 3367: 2, 5941: 1, 7036: 1, 7109: 1, 1139: 1, 6915: 3, 5486: 2, 3518: 2, 3325: 1, 4383: 2, 2904: 1, 4275: 2, 6837: 1, 6813: 1, 3209: 1, 3045: 2, 3336: 2, 7277: 2, 6147: 2, 3226: 2, 5666: 2, 3251: 2, 6940: 2, 4464: 1, 2550: 1, 6823: 1, 1300: 2, 5616: 1, 4621: 1, 3959: 2, 4314: 2, 3152: 1, 7625: 3, 621: 2, 4408: 1, 6374: 1, 5779: 2, 3232: 2, 1019: 2, 6773: 2, 3096: 1, 1044: 3, 5220: 1, 2673: 1, 1361: 3, 747: 1, 4209: 2, 6067: 2, 5476: 2, 7141: 2, 2507: 1, 8054: 1, 6652: 2, 101: 2, 765: 3, 1589: 2, 6318: 2, 4652: 2, 835: 2, 4062: 3, 4244: 2, 7812: 3, 191: 2, 4245: 1, 3263: 2, 1726: 1, 235: 2, 4762: 2, 5492: 1, 7617: 1, 1773: 2, 1711: 1, 1679: 3, 3532: 2, 3895: 1, 3477: 1, 8123: 2, 1537: 1, 5974: 3, 5673: 3, 5699: 3, 703: 1, 6039: 1, 8115: 1, 4211: 2, 7330: 1, 7328: 1, 3417: 2, 1332: 3, 1159: 1, 3837: 1, 7329: 2, 7191: 2, 5923: 1, 5694: 2, 5097: 1, 3590: 1, 1396: 1, 3488: 2, 1283: 3, 6539: 1, 1245: 2, 4937: 3, 1762: 2, 6328: 2, 6575: 1, 3348: 1, 1561: 1, 5780: 3, 6161: 1, 4935: 3, 4068: 3, 4527: 2, 6898: 3, 7780: 1, 6872: 1, 7181: 2, 6001: 3, 1223: 2, 5458: 2, 1935: 1, 5603: 2, 3654: 3, 7932: 1, 1221: 1, 4140: 2, 1346: 3, 3466: 2, 3537: 1, 4049: 3, 7451: 2, 3307: 1, 4922: 1, 6363: 1, 7219: 2, 2547: 3, 5049: 2, 804: 2, 1820: 3, 1822: 1, 4223: 2, 7133: 1, 2908: 1, 7332: 1, 4660: 2, 1865: 1, 5835: 3, 7148: 1, 2522: 1, 5226: 1, 3069: 2, 7060: 1, 6811: 3, 7072: 1, 7713: 1, 2262: 2, 2729: 1, 3860: 1, 6572: 1, 5524: 1, 7695: 3, 1827: 1, 33: 1, 4944: 3, 5587: 1, 5844: 2, 6664: 2, 1134: 2, 666: 2, 6015: 3, 3858: 1, 4282: 3, 4001: 1, 856: 2, 1603: 2, 5683: 1, 1156: 1, 4860: 2, 6238: 1, 5819: 1, 1926: 1, 1394: 2, 8090: 1, 2312: 1, 2776: 3, 4350: 1, 957: 2, 6099: 2, 889: 3, 2594: 2, 1741: 2, 4148: 1, 7732: 1, 2127: 1, 4106: 1, 7905: 1, 5824: 1, 4915: 3, 1501: 2, 5056: 2, 1883: 3, 8075: 2, 3411: 2, 7873: 1, 6098: 1, 7654: 1, 5659: 2, 643: 1, 6209: 2, 162: 1, 2938: 1, 789: 2, 4261: 2, 780: 1, 513: 1, 1792: 1, 6663: 1, 5079: 2, 771: 1, 1671: 2, 392: 2, 5971: 1, 3340: 1, 4303: 3, 7880: 2, 5401: 2, 3428: 2, 4556: 1, 2796: 1, 3544: 2, 408: 2, 6542: 2, 4009: 2, 2460: 2, 2940: 1, 3943: 1, 4233: 3, 6361: 2, 629: 3, 257: 1, 2119: 1, 4942: 2, 3245: 1, 3342: 1, 6669: 1, 2288: 1, 7372: 2, 891: 3, 4602: 3, 5041: 1, 5509: 3, 4505: 1, 5735: 2, 5788: 1, 1166: 1, 7931: 1, 5733: 1, 1518: 2, 2279: 2, 674: 1, 3921: 1, 8: 2, 6697: 3, 4256: 1, 5009: 1, 6503: 1, 6317: 1, 2535: 1, 168: 1, 7790: 1, 3080: 2, 7800: 1, 4307: 2, 2752: 2, 2142: 2, 6348: 2, 5566: 2, 3179: 1, 3637: 2, 6466: 2, 7988: 1, 1637: 1, 4260: 3, 7494: 2, 5369: 2, 6022: 1, 7675: 2, 4040: 2, 1669: 2, 2887: 1, 7615: 1, 1770: 2, 70: 1, 385: 3, 1636: 3, 175: 2, 2379: 1, 426: 3, 8020: 1, 3344: 1, 6523: 1, 7939: 1, 2231: 2, 6327: 1, 7276: 1, 3452: 2, 5474: 3, 1334: 3, 4239: 2, 4035: 3, 2986: 2, 6413: 3, 812: 3, 5232: 1, 5785: 1, 144: 3, 5660: 2, 4798: 3, 4569: 2, 4335: 1, 5979: 1, 757: 2, 5050: 1, 1542: 1, 5613: 3, 6136: 2, 1904: 2, 2003: 1, 4553: 1, 331: 1, 216: 1, 7836: 2, 2035: 2, 1355: 1, 2320: 3, 7641: 1, 5872: 3, 5289: 3, 7304: 1, 6906: 2, 3771: 1, 3262: 1, 827: 1, 7861: 2, 4098: 3, 6041: 1, 3118: 3, 4680: 2, 1779: 1, 5449: 1, 7069: 1, 2013: 1, 1836: 1, 7139: 1, 209: 1, 1516: 1, 7126: 2, 5493: 1, 3460: 2, 5677: 1, 4558: 3, 1819: 2, 2759: 1, 2040: 1, 6834: 1, 2566: 1, 1456: 2, 2184: 2, 3912: 1, 4813: 3, 5452: 3, 995: 2, 3381: 1, 5177: 1, 2455: 2, 4871: 2, 556: 2, 2426: 2, 2651: 1, 6865: 2, 6870: 3, 7554: 2, 3763: 1, 7528: 1, 7607: 1, 7517: 1, 6344: 1, 4129: 3, 5652: 2, 1834: 1, 550: 2, 4341: 1, 4758: 2, 4847: 2, 866: 2, 2650: 1, 245: 1, 1703: 2, 8088: 1, 4896: 1, 3355: 1, 3572: 2, 1652: 3, 2052: 2, 6062: 1, 7962: 2, 5176: 1, 872: 2, 1811: 3, 7175: 1, 7418: 3, 1042: 1, 5098: 2, 2477: 1, 7720: 1, 961: 3, 2542: 2, 5849: 1, 3720: 1, 7952: 1, 7385: 2, 4888: 1, 1937: 2, 4375: 1, 8006: 2, 9: 2, 4733: 2, 1199: 1, 6426: 2, 5111: 1, 2069: 1, 2437: 1, 2264: 2, 366: 1, 1805: 2, 6066: 2, 7471: 1, 4440: 1, 2429: 2, 7316: 1, 3214: 1, 3353: 2, 8042: 1, 7797: 2, 8049: 2, 3295: 2, 2984: 1, 4395: 1, 3512: 2, 187: 3, 3619: 1, 8137: 2, 7157: 1, 5808: 2, 905: 2, 5676: 2, 2259: 2, 362: 1, 8117: 2, 314: 3, 6984: 1, 8071: 2, 5043: 2, 142: 3, 6909: 1, 5838: 1, 4345: 1, 1640: 2, 1328: 3, 2942: 1, 2401: 3, 7192: 3, 7410: 3, 7395: 2, 5070: 1, 157: 1, 3806: 2, 4803: 1, 5088: 1, 1154: 2, 6352: 2, 7113: 2, 7397: 2, 1147: 2, 5944: 1, 7166: 1, 5159: 1, 4451: 1, 4775: 3, 7949: 2, 618: 2, 3486: 1, 8052: 2, 6461: 2, 5913: 2, 5740: 1, 3203: 3, 6131: 3, 7734: 2, 7919: 2, 3503: 1, 8100: 2, 6929: 1, 4080: 1, 7840: 1, 1940: 2, 1347: 2, 2527: 1, 2583: 3, 2083: 2, 6785: 1, 648: 1, 907: 2, 1988: 1, 1499: 1, 7167: 1, 6966: 2, 6250: 1, 7770: 2, 1219: 1, 5158: 2, 1453: 3, 5892: 1, 340: 2, 1789: 1, 5087: 1, 2451: 2, 1067: 2, 3514: 2, 3491: 1, 3430: 1, 4330: 2, 7626: 1, 4492: 1, 7444: 1, 354: 1, 4765: 2, 1324: 2, 283: 1, 4170: 1, 869: 2, 3177: 1, 6714: 2, 4965: 2, 3534: 2, 2912: 2, 4905: 3, 7628: 1, 7100: 1, 566: 2, 7933: 1, 7153: 1, 1382: 2, 6480: 2, 4646: 1, 5496: 1, 1665: 1, 2358: 2, 7725: 1, 5593: 1, 7624: 1, 2936: 1, 3334: 1, 3254: 1, 3093: 2, 3825: 1, 692: 1, 534: 2, 6689: 2, 5889: 1, 5292: 2, 1919: 2, 7140: 1, 829: 2, 8126: 2, 7621: 2, 5610: 1, 7858: 3, 7464: 2, 1752: 1, 7549: 2, 5743: 1, 7196: 2, 2422: 1, 2941: 1, 6845: 1, 2544: 2, 6428: 1, 1411: 1, 2486: 1, 4776: 3, 7487: 3, 3357: 1, 3949: 2, 3550: 1, 377: 1, 5741: 2, 6768: 1, 8011: 3, 1540: 1, 272: 2, 4379: 3, 864: 3, 940: 2, 6448: 3, 5514: 2, 1342: 2, 2718: 2, 7420: 3, 4026: 2, 5398: 3, 2930: 1, 3819: 2, 1903: 1, 1112: 3, 5341: 1, 7835: 3, 4963: 1, 2777: 2, 4504: 2, 7365: 1, 5017: 1, 1708: 2, 1062: 3, 4498: 3, 7864: 2, 3212: 1, 2122: 2, 3711: 2, 7107: 2, 3249: 1, 701: 3, 3565: 2, 662: 2, 7738: 2, 2444: 3, 730: 2, 6876: 2, 6249: 1, 7914: 1, 6676: 2, 3204: 2, 6547: 2, 5561: 3, 1163: 2, 7813: 1, 769: 2, 4137: 1, 3258: 1, 5519: 1, 437: 2, 7210: 1, 7848: 2, 2567: 1, 2600: 3, 734: 1, 1086: 1, 7658: 2, 4048: 3, 20: 2, 3492: 1, 1357: 2, 3661: 2, 4512: 3, 4533: 2, 4234: 1, 5506: 3, 3400: 1, 2702: 1, 571: 2, 3881: 1, 2871: 2, 7637: 1, 8082: 1, 7376: 1, 6722: 1, 7428: 2, 2691: 1, 1200: 2, 1013: 3, 2603: 2, 610: 2, 7646: 1, 1651: 1, 7749: 2, 3640: 2, 7921: 1, 6166: 1, 47: 1, 4821: 2, 1422: 1, 6485: 1, 5912: 2, 3426: 1, 8129: 3, 4248: 2, 1861: 1, 1653: 1, 3108: 1, 6084: 2, 508: 1, 151: 2, 2848: 2, 5250: 1, 5686: 1, 3809: 1, 954: 2, 6024: 2, 4003: 2, 2737: 2, 6151: 3, 4387: 2, 4356: 2, 325: 3, 4706: 3, 6211: 2, 6310: 2, 4073: 1, 4951: 2, 2240: 2, 836: 2, 823: 3, 4479: 2, 449: 3, 3276: 2, 5899: 1, 1859: 1, 5126: 1, 266: 3, 1001: 3, 1176: 1, 1043: 3, 7769: 1, 2057: 1, 5621: 1, 4809: 3, 2377: 2, 6551: 1, 5329: 1, 3971: 2, 1769: 1, 7697: 1, 1297: 1, 15: 1, 580: 3, 2613: 1, 608: 2, 7937: 1, 2762: 1, 4420: 1, 5216: 3, 732: 2, 7055: 1, 3097: 1, 1029: 2, 7918: 3, 7110: 2, 7567: 1, 5016: 2, 7366: 1, 7173: 1, 6223: 1, 116: 1, 2220: 1, 6251: 1, 2229: 1, 1632: 1, 6795: 2, 3088: 1, 2867: 1, 3844: 2, 5558: 1, 5365: 3, 8073: 2, 4838: 1, 6440: 1, 6321: 2, 7021: 1, 2495: 2, 4825: 3, 5156: 1, 5196: 3, 6695: 1, 6854: 1, 153: 3, 6517: 1, 3575: 1, 3987: 2, 5077: 2, 4605: 2, 504: 2, 4979: 1, 5073: 2, 2200: 2, 6320: 1, 6033: 1, 2298: 2, 4647: 3, 5893: 1, 6562: 3, 1458: 3, 3236: 1, 1553: 1, 7865: 1, 4412: 1, 3255: 2, 7542: 1, 4221: 1, 7015: 1, 5122: 3, 123: 1, 871: 2, 3859: 1, 5456: 2, 7463: 2, 623: 3, 3592: 2, 3302: 1, 1052: 2, 2212: 2, 270: 1, 3267: 2, 5832: 2, 7837: 3, 3422: 2, 1545: 1, 6777: 1, 5483: 3, 3111: 3, 778: 1, 5722: 1, 7762: 1, 6094: 2, 4613: 1, 7189: 2, 6115: 2, 3165: 2, 7164: 2, 8139: 3, 2204: 2, 2373: 1, 3233: 2, 67: 2, 6058: 1, 1566: 2, 2308: 1, 456: 1, 4818: 1, 7591: 1, 1175: 1, 3205: 1, 7788: 1, 7981: 2, 7715: 1, 3735: 1, 5508: 3, 6409: 3, 6265: 1, 4984: 2, 2761: 1, 2498: 1, 2025: 1, 2911: 1, 2539: 1, 5265: 1, 3199: 1, 6316: 2, 38: 2, 8114: 2, 1488: 2, 4187: 2, 1941: 2, 1847: 1, 2340: 1, 7666: 1, 3748: 1, 6086: 1, 8095: 2, 4631: 2, 606: 1, 1738: 1, 612: 2, 7063: 2, 1359: 1, 5432: 3, 6968: 1, 7130: 1, 7491: 2, 7740: 2, 1094: 2, 7370: 3, 7429: 3, 1305: 3, 7929: 2, 1366: 2, 6087: 1, 960: 2, 6477: 1, 972: 1, 5153: 2, 6349: 1, 186: 1, 5983: 2, 6584: 1, 1449: 3, 7582: 2, 1764: 1, 847: 1, 4541: 1, 0: 1, 3319: 2, 28: 1, 3290: 2, 2880: 2, 1733: 1, 6082: 1, 5888: 3, 1168: 1, 7785: 2, 5095: 2, 6949: 2, 1694: 1, 2666: 2, 2415: 3, 225: 1, 7128: 1, 8083: 1, 890: 2, 3485: 3, 5301: 1, 7965: 1, 672: 2, 2557: 2, 1506: 1, 2839: 1, 430: 1, 6750: 1, 130: 2, 4079: 1, 4349: 1, 5005: 2, 1841: 3, 4449: 2, 6505: 1, 5138: 1, 3323: 1, 3449: 1, 167: 3, 6184: 1, 6604: 1, 1061: 1, 3237: 2, 5562: 2, 1719: 1, 4393: 1, 1876: 2, 6721: 1, 7373: 1, 5615: 3, 4633: 2, 7555: 3, 2504: 3, 7053: 1, 5480: 1, 2440: 2, 5885: 1, 3773: 2, 2898: 1, 8053: 1, 7596: 1, 8016: 1, 5331: 2, 6137: 1, 6874: 2, 928: 3, 109: 1, 3842: 1, 7563: 1, 7633: 2, 6164: 1, 7924: 1, 7630: 1, 673: 2, 2616: 1, 3639: 1, 7076: 2, 7603: 1, 7584: 1, 3217: 3, 6287: 2, 3124: 1, 2643: 1, 553: 3, 2071: 1, 5352: 2, 4141: 2, 7283: 2, 5793: 1, 4985: 2, 6104: 1, 1953: 2, 7272: 2, 5513: 1, 2974: 2, 2545: 2, 115: 1, 2417: 1, 7568: 2, 2382: 1, 424: 1, 5636: 1, 1263: 2, 2585: 2, 1322: 1, 3867: 2, 6556: 3, 3362: 2, 4693: 1, 3202: 3, 5172: 1, 1729: 2, 7185: 1, 6964: 3, 1352: 1, 7963: 2, 7061: 2, 4929: 1, 5828: 2, 5987: 2, 2149: 2, 5535: 2, 1173: 1, 5252: 2, 6965: 3, 3448: 1, 4131: 1, 3542: 1, 3701: 2, 4368: 1, 4636: 1, 1205: 2, 6638: 2, 4194: 3, 4120: 1, 5995: 2, 2185: 2, 343: 1, 3945: 3, 2896: 3, 2534: 1, 4895: 1, 7991: 2, 6908: 2, 3919: 1, 6382: 2, 6119: 1, 6974: 3, 6150: 1, 1990: 1, 1121: 1, 2895: 1, 5982: 1, 6054: 1, 3368: 1, 6914: 3, 1609: 2, 7961: 2, 4139: 3, 651: 2, 2336: 2, 3669: 2, 3915: 2, 7806: 2, 6040: 1, 2143: 1, 840: 3, 2448: 2, 2977: 3, 6452: 1, 5845: 1, 7087: 2, 2073: 1, 3025: 2, 234: 2, 4406: 2, 6401: 2, 3689: 1, 6594: 1, 19: 1, 3009: 2, 198: 1, 321: 1, 6740: 1, 5705: 2, 7404: 1, 3999: 2, 787: 1, 4823: 2, 2232: 1, 4277: 1, 7400: 1, 5775: 3, 95: 2, 3438: 3, 3207: 2, 1854: 1, 1084: 1, 4439: 1, 3185: 2, 4511: 1, 4747: 3, 3616: 1, 5212: 3, 4199: 1, 1313: 2, 5778: 1, 4065: 2, 2824: 2, 4688: 3, 598: 1, 1340: 1, 4195: 3, 791: 2, 418: 2, 579: 1, 4090: 2, 4296: 1, 1288: 1, 6591: 3, 688: 2, 5481: 1, 2858: 2, 5675: 2, 1778: 2, 6207: 1, 6608: 2, 421: 2, 1532: 2, 1946: 2, 3425: 1, 6612: 2, 895: 2, 4641: 3, 749: 3, 4067: 1, 1116: 1, 2201: 1, 6952: 1, 6297: 2, 4800: 3, 6162: 1, 4069: 2, 1375: 1, 2832: 1, 5344: 1, 3793: 2, 2817: 2, 6611: 1, 2604: 2, 7902: 1, 333: 1, 6846: 2, 5174: 1, 4785: 3, 6146: 3, 2980: 3, 7387: 1, 4272: 1, 1378: 2, 2323: 1, 4176: 1, 4865: 2, 7819: 3, 705: 2, 2074: 2, 3563: 1, 3134: 2, 6671: 1, 6273: 1, 1867: 1, 5105: 1, 4122: 3, 3224: 1, 1753: 1, 3990: 2, 2617: 1, 1011: 2, 3916: 1, 5068: 2, 469: 1, 8069: 1, 14: 1, 6933: 3, 6275: 2, 3934: 2, 1697: 1, 7866: 1, 5807: 3, 754: 2, 413: 1, 1367: 2, 1240: 2, 4869: 2, 8078: 1, 427: 1, 3676: 1, 3595: 1, 357: 2, 3509: 1, 4787: 1, 1303: 1, 97: 1, 6456: 1, 874: 1, 6027: 2, 2792: 1, 4462: 1, 4164: 2, 7252: 1, 7817: 1, 1896: 1, 7124: 1, 5082: 2, 1662: 2, 7377: 1, 5091: 1, 1438: 1, 7499: 1, 2243: 1, 3650: 1, 4188: 1, 2059: 1, 7147: 1, 5081: 1, 3035: 1, 3584: 2, 2473: 2, 3963: 2, 2341: 2, 7859: 3, 5462: 1, 6985: 1, 4304: 1, 7681: 2, 1975: 2, 1645: 1, 3047: 2, 7843: 2, 203: 1, 3525: 1, 3739: 3, 7089: 1, 3519: 2, 1253: 1, 3930: 1, 1480: 2, 3845: 2, 2248: 1, 4886: 1, 3606: 3, 3979: 2, 7512: 1, 3332: 1, 6668: 1, 5961: 1, 7416: 2, 5497: 2, 4192: 1, 5597: 1, 174: 2, 3053: 1, 6102: 2, 6263: 1, 2549: 3, 3633: 1, 2814: 1, 6439: 1, 3876: 1, 3631: 1, 463: 1, 7344: 2, 5438: 1, 714: 1, 2822: 2, 1460: 2, 2376: 1, 4118: 1, 287: 1, 3754: 2, 7857: 2, 3494: 1, 1008: 1, 4134: 1, 1749: 1, 2493: 2, 7606: 1, 4183: 2, 5047: 2, 4966: 2, 7618: 1, 7562: 1, 2574: 1, 3761: 3, 7736: 1, 4453: 1, 5855: 1, 7117: 2, 1157: 1, 3281: 2, 3918: 2, 444: 1, 4834: 2, 3757: 1, 8064: 1, 4841: 1, 6807: 1, 4882: 3, 7455: 1, 22: 1, 1520: 3, 4138: 1, 1747: 1, 3292: 2, 3056: 1, 4246: 1, 4573: 1, 1781: 3, 5767: 1, 1216: 1, 6: 2, 6963: 1, 1215: 1, 7650: 1, 1890: 1, 4354: 1, 6153: 1, 7449: 3, 3520: 1, 6597: 1, 1888: 1, 2258: 1, 7224: 2, 107: 1, 6754: 1, 86: 2, 2818: 2, 4360: 1, 3833: 1, 2962: 2, 113: 1, 2230: 1, 284: 1, 6920: 2, 3674: 1, 7282: 1, 4974: 2, 4761: 2, 7406: 2, 210: 2, 255: 2, 5571: 2, 1877: 1, 6157: 2, 5297: 2, 4961: 1, 2891: 1, 834: 2, 5594: 1, 6438: 1, 2803: 2, 3734: 2, 3908: 1, 5639: 1, 3747: 1, 5522: 2, 335: 2, 6474: 1, 1189: 3, 899: 2, 4881: 1, 2573: 2, 2112: 1, 6242: 1, 4814: 2, 4614: 1, 1444: 1, 884: 2, 3800: 1, 963: 1, 1625: 1, 5201: 2, 1138: 1, 560: 1, 7509: 2, 5567: 2, 5442: 1, 7854: 2, 2410: 2, 3085: 1, 6615: 2, 6564: 1, 6484: 1, 4670: 2, 1695: 2, 4384: 1, 4276: 2, 1226: 1, 2134: 1, 6454: 1, 3318: 1, 3871: 2, 2214: 2, 3899: 2, 2085: 1, 3459: 1, 8113: 1, 5106: 2, 6770: 3, 4190: 3, 2478: 2, 2463: 1, 7605: 1, 2998: 1, 7102: 1, 7297: 1, 4402: 1, 543: 1, 7540: 1, 2385: 2, 4476: 1, 5641: 1, 3159: 2, 3253: 1, 996: 3, 3395: 2, 6706: 1, 1285: 1, 7019: 2, 3708: 2, 6847: 1, 4698: 1, 6894: 1, 3864: 1, 5036: 1, 3119: 2, 5754: 2, 152: 1, 2459: 1, 5254: 1, 7660: 3, 7017: 1, 5124: 1, 3880: 1, 1788: 3, 1897: 1, 7979: 2, 1680: 1, 7557: 1, 4781: 3, 489: 2, 6246: 1, 1905: 3, 4832: 1, 7561: 1, 875: 2, 956: 2, 1714: 1, 6627: 1, 6956: 1, 4846: 1, 680: 1, 2601: 2, 7254: 1, 2511: 2, 6003: 1, 146: 2, 3888: 1, 7503: 1, 53: 1, 4684: 1, 3513: 1, 3841: 1, 2039: 2, 1296: 2, 4264: 1, 844: 1, 2589: 1, 3081: 1, 6000: 2, 7038: 1, 5810: 1, 7610: 1, 1571: 2, 3132: 1, 5029: 1, 159: 1, 3535: 2, 8021: 1, 1560: 2, 6260: 1, 6060: 1, 3446: 1, 1920: 2, 5055: 1, 3343: 3, 6759: 1, 7912: 1, 7714: 1, 4418: 1, 1399: 2, 3003: 1, 2795: 1, 2024: 1, 81: 2, 6140: 2, 3497: 1, 1791: 1, 7595: 1, 7543: 2, 2645: 1, 3113: 2, 5813: 2, 1961: 1, 5879: 2, 7862: 1, 6732: 1, 2607: 1, 6189: 1, 4456: 3, 5947: 1, 5590: 2, 4596: 1, 2199: 1, 6191: 3, 5235: 1, 2837: 3, 3622: 1, 6194: 1, 7045: 2, 5271: 1, 5533: 3, 1577: 2, 31: 1, 6095: 2, 441: 1, 1418: 1, 4805: 3, 2165: 2, 6398: 1, 7162: 2, 1929: 1, 1317: 1, 4107: 2, 2968: 1, 7759: 1, 2923: 1, 1125: 1, 2133: 3, 6590: 1, 5059: 2, 5726: 1, 7550: 1, 6267: 1, 1225: 1, 7439: 1, 3037: 1, 4945: 2, 3450: 1, 125: 1, 6537: 2, 7906: 1, 5267: 1, 3441: 1, 4312: 1, 1869: 1, 3136: 2, 2305: 2, 2972: 1, 6548: 1, 5635: 3, 896: 2, 595: 1, 3767: 1, 4599: 1, 3521: 1, 4150: 2, 3588: 1, 535: 2, 3352: 1, 6515: 1, 6772: 2, 7744: 1, 5479: 2, 6370: 1, 509: 2, 4609: 1, 6334: 1, 7289: 1, 5187: 1, 2728: 1, 600: 1, 6026: 2, 214: 1, 6625: 1, 2433: 1, 1624: 1, 347: 2, 2901: 1, 1782: 1, 82: 1, 3507: 3, 1485: 1, 4284: 1, 6973: 1, 6942: 1, 77: 1, 1329: 1, 4489: 1, 7581: 1, 5469: 2, 5075: 2, 2615: 2, 2558: 3, 5985: 1, 627: 1, 2409: 2, 2027: 1, 2708: 1, 1550: 2, 1229: 1, 76: 1, 7556: 2, 6155: 1, 6905: 1, 7136: 1, 4950: 2, 5021: 1, 8012: 1, 7202: 1, 8086: 1, 615: 2, 5228: 1, 5836: 1, 1569: 1, 6407: 3, 6463: 3, 7844: 2, 721: 2, 3164: 2, 4044: 2, 6182: 3, 4200: 1, 3680: 1, 7674: 1, 5580: 2, 807: 2, 1414: 1, 4872: 3, 5399: 1, 5429: 2, 7003: 1, 2701: 1, 2370: 1, 3427: 1, 2797: 1, 3693: 2, 1364: 1, 7612: 1, 2710: 1, 5521: 1, 3678: 2, 468: 1, 6962: 1, 5831: 2, 4177: 1, 247: 1, 6378: 2, 7604: 1, 5415: 1, 4156: 2, 277: 2, 23: 1, 2476: 2, 4340: 1, 1993: 2, 481: 2, 3743: 1, 2991: 1, 4615: 1, 1113: 1, 6303: 1, 1601: 2, 944: 1, 7134: 1, 1144: 1, 2618: 1, 2394: 1, 3457: 2, 4105: 1, 7629: 1, 1567: 2, 6417: 1, 7907: 1, 2335: 1, 2396: 1, 1746: 2, 6633: 1, 4884: 1, 7233: 1, 1774: 2, 5270: 1, 2688: 1, 3168: 2, 211: 1, 3964: 1, 7037: 1, 3275: 1, 5504: 1, 5355: 1, 6862: 1, 2354: 3, 7313: 1, 3238: 1, 2066: 2, 2009: 1, 2194: 1, 1093: 1, 2668: 2, 6286: 1, 3527: 1, 1744: 1, 8096: 2, 1870: 1, 5680: 1, 1470: 1, 3300: 2, 3562: 2, 7016: 1, 1698: 1, 1850: 1, 2031: 2, 5194: 1, 3716: 2, 6977: 1, 1512: 2, 6183: 2, 6925: 2, 949: 1, 1916: 1, 4662: 1, 3893: 2, 1151: 2, 4620: 2, 5717: 1, 4699: 2, 5448: 1, 6679: 1, 1227: 1, 1894: 1, 3940: 1, 3020: 2, 3652: 1, 7359: 2, 7891: 1, 6724: 1, 3547: 1, 1090: 1, 6306: 1, 7763: 1, 5586: 2, 206: 3, 3337: 2, 950: 1, 4290: 2, 7982: 1, 2658: 2, 3144: 1, 5646: 1, 783: 2, 207: 2, 3197: 1, 7433: 1, 2526: 1, 7707: 1, 7870: 1, 5969: 1, 7687: 1, 3574: 2, 677: 2, 1663: 1, 3580: 1, 5198: 2, 3465: 1, 93: 1, 3647: 1, 2139: 1, 5623: 2, 73: 2, 1878: 1, 1284: 1, 1732: 1, 5777: 1, 7010: 1, 4893: 1, 6646: 2, 6779: 1, 2870: 1, 252: 2, 7216: 1, 1022: 2, 6881: 2, 7875: 1, 3630: 1, 6322: 2, 2256: 1, 467: 2, 4737: 3, 6631: 2, 271: 1, 2246: 1, 1432: 2, 4247: 1, 7576: 1, 5346: 1, 1684: 1, 6715: 1, 5529: 1, 164: 2, 1289: 1, 103: 2, 1686: 1, 4235: 1, 4145: 2, 3658: 2, 2860: 1, 4907: 2, 4655: 1, 1872: 2, 6478: 1, 7546: 3, 1276: 2, 7969: 2, 5350: 1, 6025: 2, 2168: 1, 7182: 1, 1971: 1, 2696: 1, 4428: 1, 6495: 1, 712: 1, 7632: 1, 6013: 1, 3828: 2, 3977: 1, 1505: 1, 6070: 1, 6685: 2, 4091: 1, 3387: 1, 7473: 1, 5592: 1, 6376: 1, 6038: 1, 1530: 1, 2569: 1, 2619: 1, 932: 2, 5031: 1, 374: 1, 1198: 2, 3962: 2, 3802: 1, 5004: 1, 5771: 1, 2456: 2, 2705: 1, 4316: 1, 1948: 1, 518: 1, 7500: 1, 5024: 3, 1167: 1, 2806: 2, 4987: 1, 2492: 1, 694: 2, 3870: 1, 587: 1, 5283: 1, 5219: 2, 6269: 1, 3493: 2, 6188: 1, 3419: 2, 1197: 1, 43: 2, 6574: 1, 3050: 1, 4060: 1, 3787: 1, 5681: 2, 5461: 2, 4801: 3, 1: 1, 2205: 2, 2670: 1, 4632: 2, 1265: 1, 7250: 1, 412: 1, 3508: 2, 1118: 1, 1204: 1, 5865: 2, 5322: 2, 5316: 1, 6174: 2, 6262: 1, 7531: 1, 4537: 1, 7871: 1, 4863: 3, 4075: 2, 3937: 1, 6120: 1, 6527: 2, 4481: 2, 3099: 1, 8034: 1, 3634: 1, 2241: 2, 6787: 1, 1036: 1, 139: 2, 120: 1, 6744: 2, 1557: 3, 2218: 2, 4426: 1, 1402: 2, 4207: 1, 5069: 1, 2819: 2, 4099: 1, 532: 1, 2979: 2, 4530: 1, 6725: 2, 2130: 1, 4243: 1, 1331: 1, 2523: 2, 3187: 2, 3530: 1, 1230: 1, 6449: 2, 4365: 1, 4715: 1, 5565: 1, 5843: 1, 3442: 1, 3423: 1, 6808: 1, 6682: 1, 7379: 2, 2118: 1, 7830: 1, 6932: 2, 3564: 1, 2098: 3, 6696: 1, 3312: 1, 7142: 1, 1038: 2, 7213: 1, 477: 2, 6308: 1, 4114: 2, 5954: 1, 4876: 2, 5048: 1, 2060: 1, 7362: 1, 3373: 1, 1496: 2, 7889: 2, 520: 1, 4423: 2, 6916: 1, 1259: 2, 7023: 1, 7226: 1, 1103: 2, 3038: 2, 6156: 1, 5970: 1, 2352: 2, 4023: 1, 1620: 1, 3075: 2, 4598: 1, 1007: 1, 1593: 2, 3600: 1, 336: 1, 3250: 1, 6728: 1, 5206: 2, 6030: 1, 8108: 1, 7955: 2, 743: 2, 7805: 2, 3690: 2, 3824: 1, 4382: 1, 4657: 1, 3206: 2, 2129: 2, 1582: 2, 6595: 1, 3010: 1, 3130: 1, 715: 2, 687: 2, 4507: 1, 647: 1, 4592: 1, 3695: 1, 3856: 1, 112: 1, 6861: 2, 1681: 1, 6560: 1, 453: 1, 3434: 1, 868: 1, 5868: 1, 3299: 1, 8025: 1, 708: 2, 1477: 1, 24: 1, 3354: 1, 733: 2, 2842: 1, 8136: 1, 6647: 1, 1688: 3, 4584: 1, 603: 1, 977: 2, 5870: 1, 5997: 1, 8044: 1, 3200: 1, 7833: 1, 2629: 1, 2680: 2, 3087: 1, 3704: 2, 2568: 2, 6208: 2, 3480: 1, 3533: 1, 7083: 2, 4431: 1, 825: 1, 2890: 2, 4082: 1, 7973: 2, 3683: 1, 71: 2, 5706: 1, 1249: 1, 1533: 1, 8019: 1, 5628: 1, 2833: 2, 3195: 1, 5939: 1, 3850: 1, 5924: 2, 6053: 1, 165: 2, 4179: 1, 974: 2, 6888: 2, 5167: 1, 6518: 2, 2813: 1, 4901: 1, 6379: 1, 5000: 2, 4488: 2, 4778: 1, 7002: 2, 803: 2, 2442: 2, 1720: 2, 1193: 1, 7071: 2, 7530: 1, 2222: 2, 649: 1, 6110: 1, 1986: 3, 7171: 2, 4564: 2, 3288: 1, 6509: 1, 4398: 1, 1104: 1, 3607: 2, 5938: 2, 1619: 2, 2678: 2, 663: 2, 5190: 1, 2359: 1, 7702: 1, 1949: 1, 3311: 3, 1114: 1, 4052: 1, 3399: 1, 4077: 1, 7775: 1, 5454: 1, 1097: 3, 330: 2, 6901: 1, 3923: 1, 4400: 2, 1190: 1, 4542: 1, 882: 1, 2260: 2, 1642: 2, 1006: 1, 3958: 1, 3048: 1, 5078: 2, 1467: 1, 2950: 1, 4025: 2, 7635: 1, 3418: 1, 2868: 2, 1089: 1, 3560: 1, 3998: 2, 912: 1, 7460: 3, 4759: 2, 4746: 1, 1886: 2, 297: 1, 5926: 1, 5707: 1, 154: 1, 6467: 1, 6187: 2, 5570: 1, 1575: 3, 184: 2, 5302: 2, 2763: 1, 7343: 2, 5714: 1, 1353: 1, 6996: 1, 6818: 2, 304: 1, 8097: 1, 3414: 1, 1413: 1, 2973: 3, 5238: 3, 5837: 1, 7435: 3, 3131: 1, 4468: 1, 4149: 2, 6709: 2, 7027: 1, 2783: 2, 4226: 1, 644: 1, 7493: 1, 4797: 1, 6008: 2, 5862: 1, 7781: 1, 378: 1, 1014: 2, 2894: 1, 6756: 1, 7671: 1, 1644: 1, 1848: 1, 1158: 1, 4588: 1, 1430: 1, 2900: 1, 639: 2, 2781: 1, 3154: 1, 2551: 1, 8015: 1, 4891: 1, 2823: 2, 3526: 1, 4413: 2, 8067: 2, 5366: 1, 6896: 1, 1224: 2, 5564: 2, 8112: 1, 3807: 2, 5257: 2, 3268: 1, 7285: 1, 2091: 1, 5723: 2, 7950: 2, 2152: 1, 4311: 1, 438: 1, 2392: 2, 4510: 1, 4352: 2, 7236: 1, 5120: 1, 4002: 1, 1816: 1, 6046: 2, 3786: 1, 7077: 1, 1177: 1, 7761: 1, 2916: 2, 7336: 1, 4540: 1, 5042: 1, 2754: 1, 6468: 1, 7062: 1, 5392: 1, 7578: 1, 406: 1, 3907: 1, 6878: 2, 6045: 1, 1330: 2, 4045: 1, 7340: 1, 476: 1, 3834: 2, 2988: 1, 5718: 1, 7241: 1, 5330: 1, 2070: 1, 2443: 1, 2790: 1, 1585: 1, 6247: 2, 923: 1, 6203: 1, 1152: 2, 7438: 2, 1874: 1, 5457: 1, 5477: 1, 7585: 1, 4601: 1, 2178: 1, 498: 1, 3078: 2, 6002: 1, 7570: 1, 2910: 2, 1362: 1, 7896: 1, 7468: 1, 298: 1, 4786: 3, 2905: 1, 7334: 1, 1813: 1, 7572: 1, 5672: 1, 7161: 2, 824: 1, 2726: 1, 3150: 1, 5795: 1, 6075: 1, 4236: 1, 4205: 2, 7047: 1, 3282: 2, 1222: 1, 916: 2, 3707: 1, 3642: 1, 3539: 1, 2106: 2, 4885: 2, 3608: 1, 4377: 1, 5129: 1, 6855: 1, 49: 2, 1884: 2, 2361: 1, 2863: 1, 3280: 2, 2468: 1, 6736: 1, 7337: 1, 4677: 1, 5262: 1, 6596: 1, 3629: 2, 5498: 1, 1583: 1, 4337: 1, 5370: 1, 1160: 2, 3116: 1, 7842: 1, 7246: 2, 122: 1, 4008: 1, 6109: 1, 5300: 1, 3301: 1, 6192: 1, 5298: 1, 3788: 1, 3697: 1, 223: 1, 7338: 1, 4628: 1, 2368: 1, 4779: 2, 1718: 1, 7978: 1, 547: 1, 4532: 1, 2219: 1, 4415: 1, 364: 1, 4702: 1, 1101: 1, 6819: 1, 1737: 2, 5317: 1, 7057: 1, 7408: 1, 2016: 1, 334: 1, 2334: 2, 5946: 1, 2990: 1, 3846: 2, 126: 1, 1407: 1, 6302: 1, 7894: 1, 624: 2, 2899: 1, 3548: 1, 7009: 1, 6254: 1, 39: 1, 6116: 2, 6691: 1, 2072: 1, 6804: 2, 4878: 1, 6883: 2, 6410: 1, 809: 2, 7752: 1, 1400: 1, 5584: 1, 2216: 1, 5032: 1, 575: 1, 3143: 1, 5716: 1, 6358: 1, 7278: 1, 3597: 1, 3339: 1, 1526: 1, 4889: 2, 7728: 1, 6863: 1, 4071: 2, 5986: 1, 5536: 1, 3246: 1, 339: 1, 7727: 1, 1145: 2, 3644: 1, 7852: 1, 4753: 1, 96: 1, 6235: 2, 2862: 1, 3408: 1, 5931: 1, 6336: 1, 3213: 1, 4124: 1, 3464: 1, 6489: 1, 7401: 1, 3361: 1, 2196: 1, 5418: 1, 1588: 1, 2590: 1, 5874: 1, 7669: 1, 6071: 1, 797: 1, 7012: 1, 4389: 1, 1873: 1, 3002: 1, 527: 1, 5687: 2, 7663: 1, 6698: 1, 2076: 1, 3831: 1, 1350: 2, 1250: 1, 7623: 1, 3953: 2, 4536: 1, 6429: 1, 5905: 1, 7363: 1, 3474: 1, 3171: 2, 4503: 1, 2234: 1, 511: 2, 959: 2, 7275: 1, 3826: 1, 1132: 2, 3316: 1, 2344: 1, 2317: 1, 1464: 3, 5821: 1, 6535: 1, 2749: 1, 3628: 2, 5583: 1, 1756: 1, 1835: 1, 1102: 1, 7564: 2, 5435: 1, 2605: 1, 5080: 1, 702: 1, 7909: 1, 2075: 2, 3839: 1, 7273: 1, 4741: 1, 3285: 1, 8018: 1, 1165: 1, 3248: 1, 6126: 1, 7044: 1, 7807: 1, 4463: 1, 263: 1, 1615: 2, 924: 1, 1830: 2, 6763: 2, 3027: 1, 2599: 2, 2707: 2, 6311: 2, 3504: 1, 6431: 1, 5891: 1, 2084: 1, 3775: 1, 4686: 1, 2831: 1, 1478: 1, 3672: 1, 5611: 1, 497: 1, 2641: 1, 6437: 1, 1318: 1, 6009: 1, 5185: 1, 4029: 1, 3294: 1, 6583: 1, 4600: 1, 3732: 1, 318: 1, 1248: 1, 6222: 1, 4152: 1, 1931: 1, 1785: 1, 1437: 1, 6672: 1, 6213: 2, 3469: 1, 613: 1, 1538: 1, 3986: 1, 5408: 1, 4681: 1, 7638: 1, 3659: 1, 1427: 1, 1633: 1, 1448: 1, 4953: 1, 4427: 1, 7884: 1, 3365: 1, 431: 1, 5012: 1, 35: 1, 3583: 1, 2062: 1, 3966: 1, 3271: 1, 1409: 1, 4547: 1, 5856: 1, 3632: 1, 4162: 1, 6018: 1, 6284: 1, 2725: 1, 4568: 1, 6019: 1, 4172: 1, 4285: 1, 2324: 2, 5406: 1, 5223: 1, 3228: 1, 1377: 1, 6718: 1, 5902: 1, 892: 1, 3033: 1, 141: 1, 1483: 1, 3765: 1, 1475: 1, 5245: 1, 1885: 1, 6464: 1, 4051: 2, 7364: 1, 7105: 1, 817: 1, 2555: 1, 4816: 1, 1771: 1, 5164: 1, 4258: 1, 4549: 1, 654: 1, 2274: 1, 3437: 1, 1521: 1, 5405: 1, 1416: 1, 3715: 1, 7470: 1, 3854: 2, 6622: 1, 1934: 1, 1096: 1, 6490: 2, 1023: 1, 2044: 1, 7756: 1, 6666: 2, 3903: 1, 2293: 1, 7244: 1, 6280: 1, 5359: 1, 6926: 1, 2206: 1, 2854: 1, 4169: 1, 1122: 1, 8033: 1, 1386: 2, 1201: 1, 482: 1, 7020: 1, 2903: 1, 2441: 1, 5846: 1, 6111: 1, 3756: 1, 2826: 1, 4053: 1, 3429: 1, 6656: 1, 423: 1, 7519: 1, 601: 1, 215: 1, 7228: 1, 3242: 1, 2913: 1, 5390: 1, 7186: 2, 2576: 1, 7777: 1, 1844: 1, 1524: 1, 4980: 1, 308: 1, 1973: 1, 1420: 1, 7518: 1, 6946: 1, 5136: 1, 2779: 1, 6264: 1, 5417: 1, 3487: 1, 7664: 1, 4300: 1, 6372: 1, 5790: 1, 7132: 1, 942: 1, 1646: 1, 2144: 1, 3995: 1, 1310: 1, 1511: 1, 3112: 1, 4361: 1, 5940: 1, 6097: 1, 7041: 1, 389: 1, 3952: 1, 5184: 1, 3714: 1, 2434: 2, 3239: 1, 7412: 1, 4548: 1, 1673: 1, 5186: 1, 800: 1, 4308: 1, 6778: 1, 1743: 1, 2524: 1, 720: 1, 5263: 1, 5246: 1, 5161: 1, 7440: 1, 1810: 1, 3706: 1, 5274: 1, 4111: 1, 6204: 1, 2768: 1, 1921: 1, 6472: 1, 4709: 1, 1766: 1, 3211: 1, 904: 1, 1106: 1, 2914: 1, 5285: 1, 4486: 1, 4804: 1, 1513: 1, 155: 1, 5484: 1, 637: 1, 2147: 1, 1040: 1, 5782: 1, 5962: 1, 1073: 1, 2548: 1, 7688: 1, 2365: 1, 3178: 1, 7335: 1, 4650: 1, 2019: 1, 7899: 1, 1611: 1, 88: 1}

In [32]:
lotar_accs = {3916: 1, 710: 3, 2012: 2, 2275: 1, 2527: 2, 3537: 1, 6792: 2, 3070: 3, 4308: 2, 5887: 1, 3203: 3, 5117: 2, 1830: 2, 68: 2, 6632: 2, 4950: 2, 7061: 1, 1799: 1, 4880: 1, 252: 3, 7746: 2, 4806: 3, 2922: 1, 1239: 3, 2536: 3, 5743: 1, 5392: 1, 23: 1, 5726: 1, 3629: 2, 5699: 3, 3041: 2, 3838: 2, 4774: 2, 2433: 3, 2625: 2, 4350: 2, 8017: 1, 1691: 2, 2531: 2, 1246: 2, 3335: 2, 7566: 2, 5398: 1, 4113: 3, 4849: 3, 8127: 3, 6672: 2, 2814: 2, 4957: 1, 1518: 2, 2633: 2, 4018: 2, 2286: 2, 3072: 2, 2394: 2, 3202: 2, 1527: 3, 291: 1, 5877: 3, 2139: 2, 6422: 1, 1141: 3, 2278: 1, 8128: 2, 4177: 2, 2875: 2, 1435: 2, 4816: 2, 1787: 2, 4621: 2, 1707: 3, 2539: 1, 7413: 3, 7873: 1, 6801: 2, 1235: 3, 904: 2, 2173: 1, 193: 1, 1938: 2, 6626: 3, 2000: 3, 2678: 2, 7018: 2, 2245: 2, 6850: 1, 3263: 1, 795: 2, 1018: 2, 2838: 2, 3045: 3, 936: 1, 5317: 2, 894: 3, 392: 2, 2431: 3, 959: 3, 743: 2, 5237: 1, 2652: 3, 1021: 3, 5057: 3, 4182: 1, 7136: 2, 6487: 1, 1255: 3, 4215: 2, 7629: 1, 6137: 2, 4475: 2, 1066: 2, 5481: 1, 7712: 3, 1372: 2, 5092: 2, 684: 2, 5675: 3, 8041: 1, 1313: 3, 2279: 2, 7011: 1, 2054: 1, 6919: 1, 5338: 1, 7579: 1, 5829: 3, 6194: 2, 4745: 2, 2716: 2, 7853: 2, 127: 2, 4906: 3, 7192: 2, 3123: 3, 5518: 3, 1093: 2, 491: 1, 2871: 1, 3920: 1, 6669: 3, 2981: 2, 1832: 2, 6694: 2, 2479: 2, 6461: 2, 987: 2, 4503: 1, 5937: 1, 3281: 2, 4190: 3, 206: 3, 2313: 1, 4912: 3, 5300: 3, 7294: 2, 5108: 1, 6968: 1, 3433: 3, 3654: 3, 1941: 1, 7657: 3, 5089: 1, 3740: 1, 2547: 3, 2537: 2, 4465: 3, 6362: 2, 2474: 2, 7001: 2, 5111: 2, 1330: 3, 5803: 3, 7364: 2, 7248: 1, 2032: 2, 7123: 3, 376: 3, 4424: 2, 1716: 1, 3782: 3, 704: 3, 4573: 2, 3343: 3, 5324: 1, 6172: 2, 5251: 2, 4001: 1, 6045: 2, 391: 2, 581: 1, 1644: 2, 4704: 3, 6985: 2, 6666: 3, 6566: 2, 3976: 3, 7444: 2, 2402: 2, 5355: 1, 7695: 2, 1205: 3, 4911: 1, 3120: 2, 5027: 2, 5685: 1, 2818: 2, 5127: 1, 6504: 1, 2058: 2, 5078: 1, 7330: 1, 1971: 3, 1355: 2, 3784: 3, 5569: 3, 6458: 1, 988: 2, 4879: 3, 7865: 2, 1604: 2, 4083: 3, 3865: 3, 115: 2, 2064: 3, 1132: 2, 7872: 3, 6609: 2, 555: 2, 612: 3, 6013: 2, 2371: 1, 2454: 3, 113: 1, 5669: 1, 5250: 1, 1405: 3, 4928: 2, 4705: 2, 1669: 3, 4522: 1, 351: 1, 3692: 1, 658: 1, 3057: 1, 3: 2, 488: 2, 43: 2, 4157: 2, 6107: 2, 1375: 2, 4064: 2, 6733: 3, 3961: 1, 5567: 1, 2659: 3, 2501: 2, 4203: 3, 648: 3, 5162: 2, 3701: 1, 2375: 3, 824: 3, 1410: 1, 332: 3, 2244: 2, 8133: 1, 1637: 1, 215: 3, 39: 2, 4643: 3, 7351: 1, 3222: 3, 3317: 3, 518: 1, 4656: 3, 4970: 3, 5715: 3, 7256: 1, 2973: 1, 4861: 3, 4043: 2, 3812: 2, 3248: 2, 7484: 3, 210: 1, 4967: 3, 3013: 1, 815: 3, 7307: 1, 2010: 1, 5289: 2, 6262: 2, 7719: 3, 3939: 1, 1039: 3, 2079: 1, 3145: 1, 4498: 2, 1172: 2, 4364: 3, 6522: 2, 4016: 2, 4230: 3, 2119: 2, 1302: 3, 1314: 2, 6175: 1, 2441: 3, 7930: 3, 4459: 3, 869: 2, 4101: 3, 752: 3, 6681: 2, 119: 2, 5835: 3, 664: 3, 2882: 3, 8061: 2, 3194: 2, 7357: 1, 7818: 1, 7471: 2, 1168: 1, 5881: 2, 5165: 2, 6884: 2, 1117: 1, 697: 1, 7953: 1, 4147: 2, 6102: 2, 7797: 1, 2592: 1, 1454: 2, 8068: 2, 923: 2, 7321: 3, 873: 3, 3853: 1, 5687: 2, 3533: 1, 5920: 2, 5110: 1, 1095: 2, 4217: 3, 6089: 1, 1160: 2, 6541: 2, 5514: 2, 1904: 3, 3227: 2, 3466: 2, 7448: 2, 2965: 3, 777: 2, 4447: 3, 705: 3, 6529: 1, 3596: 2, 7194: 2, 5735: 1, 834: 2, 888: 3, 1808: 2, 7693: 1, 6763: 3, 5397: 3, 5964: 2, 7929: 2, 4130: 2, 5698: 2, 4566: 2, 132: 1, 7637: 3, 4049: 3, 7586: 1, 2234: 3, 1531: 3, 698: 2, 437: 3, 2168: 1, 2395: 3, 2945: 2, 4223: 2, 5755: 1, 5592: 1, 548: 3, 4185: 1, 1838: 2, 4090: 3, 5578: 2, 6118: 1, 6617: 1, 8029: 1, 561: 3, 6608: 2, 2997: 1, 2650: 2, 3076: 3, 1763: 3, 7167: 2, 7935: 2, 7002: 2, 6245: 1, 1261: 3, 4800: 2, 1060: 1, 782: 3, 4294: 1, 153: 2, 4972: 2, 3244: 3, 1483: 2, 7549: 2, 6280: 2, 7426: 2, 7322: 3, 5489: 3, 7896: 1, 2468: 1, 6787: 2, 4612: 3, 953: 3, 2392: 2, 3907: 2, 5211: 2, 7466: 1, 6131: 3, 2283: 3, 1995: 2, 3883: 2, 4263: 2, 5455: 1, 4195: 3, 7996: 3, 8024: 2, 3132: 1, 7752: 1, 6480: 3, 6456: 2, 1319: 3, 1629: 1, 576: 1, 5386: 3, 1462: 1, 7705: 1, 4994: 2, 1730: 2, 6634: 1, 3684: 2, 3197: 1, 5791: 1, 5460: 1, 4584: 1, 5207: 2, 6600: 2, 5616: 2, 1603: 1, 4580: 2, 4163: 1, 6550: 1, 7991: 2, 6810: 1, 971: 2, 6135: 2, 5854: 2, 1227: 3, 1263: 3, 5668: 2, 1505: 1, 5627: 1, 7287: 3, 2292: 3, 4728: 2, 2264: 2, 858: 1, 3400: 2, 2600: 2, 7777: 2, 1288: 2, 2496: 3, 7004: 1, 7210: 1, 1965: 2, 2122: 3, 6339: 1, 4720: 1, 6161: 1, 5050: 3, 6241: 1, 3435: 1, 4781: 3, 3177: 3, 1424: 2, 6173: 3, 3554: 1, 1243: 3, 4026: 3, 3657: 3, 3509: 1, 4257: 1, 3068: 2, 3613: 1, 2039: 1, 1180: 2, 1916: 2, 4269: 1, 5618: 2, 7674: 2, 5638: 1, 3416: 3, 2742: 2, 2522: 1, 365: 1, 1811: 2, 1208: 2, 786: 2, 1303: 2, 2896: 2, 5046: 2, 5104: 2, 7897: 3, 6861: 1, 3969: 3, 96: 1, 223: 2, 1721: 3, 1123: 1, 5976: 1, 1712: 3, 5058: 1, 5374: 1, 728: 3, 3755: 3, 4847: 2, 5531: 1, 260: 1, 1277: 3, 7668: 1, 4232: 3, 7343: 2, 654: 2, 729: 3, 7939: 2, 3195: 2, 1279: 2, 7362: 1, 6316: 2, 8141: 2, 2367: 3, 898: 3, 5090: 2, 6872: 2, 5625: 1, 802: 2, 3914: 2, 3610: 1, 543: 3, 3885: 1, 2197: 2, 5923: 3, 3547: 1, 5274: 2, 417: 1, 8112: 2, 4104: 2, 934: 3, 5210: 3, 6568: 3, 3228: 3, 2331: 3, 5742: 1, 964: 2, 2732: 2, 7724: 2, 1855: 1, 4834: 2, 7731: 3, 701: 3, 4134: 1, 6168: 2, 1930: 1, 1833: 1, 4356: 1, 7009: 3, 1290: 2, 3234: 1, 6807: 1, 5690: 2, 3726: 2, 746: 1, 2822: 1, 1843: 2, 1089: 3, 1453: 1, 6237: 1, 5134: 2, 2325: 3, 4437: 1, 5749: 2, 6590: 1, 3725: 1, 5546: 2, 2594: 3, 5693: 1, 3660: 2, 4011: 2, 3601: 2, 4027: 3, 7224: 3, 4602: 3, 7289: 1, 3756: 2, 6116: 1, 793: 2, 467: 1, 6741: 3, 6369: 2, 4310: 3, 893: 3, 6322: 1, 1316: 3, 2607: 1, 2056: 2, 4617: 2, 7281: 1, 2373: 1, 2284: 3, 3258: 1, 5396: 3, 5017: 2, 95: 1, 4568: 3, 4872: 3, 599: 3, 6073: 1, 130: 2, 5479: 2, 5896: 1, 4629: 3, 4565: 1, 1193: 1, 2344: 1, 4859: 1, 3490: 1, 1270: 3, 7316: 1, 3768: 3, 5931: 1, 7908: 1, 7571: 2, 6351: 2, 4003: 2, 8001: 2, 4488: 1, 2141: 1, 3022: 2, 4735: 2, 5079: 1, 1912: 3, 909: 3, 7849: 1, 5912: 2, 360: 2, 6413: 2, 8030: 1, 2861: 2, 3933: 1, 2315: 3, 4628: 1, 2874: 3, 1223: 3, 902: 3, 2377: 1, 560: 1, 5011: 2, 887: 3, 5293: 2, 6836: 2, 4183: 2, 3511: 1, 6041: 2, 992: 2, 6123: 2, 5280: 3, 866: 1, 5182: 1, 2587: 3, 3761: 3, 3817: 1, 7601: 3, 6342: 1, 6543: 2, 2578: 2, 6078: 2, 4408: 2, 8036: 2, 3495: 1, 7623: 1, 5404: 2, 4073: 1, 4106: 2, 5177: 2, 4456: 2, 2933: 1, 5154: 2, 6964: 3, 1329: 1, 3742: 3, 5676: 1, 1295: 3, 1937: 2, 3623: 1, 3014: 3, 1905: 2, 5275: 2, 6791: 1, 7856: 2, 1976: 2, 6066: 1, 2557: 1, 7554: 2, 3162: 1, 4649: 2, 3973: 2, 4758: 2, 4709: 1, 5651: 1, 1722: 2, 3677: 2, 803: 3, 5841: 2, 524: 3, 385: 3, 764: 3, 5852: 2, 1380: 3, 6389: 1, 2410: 1, 2019: 3, 398: 2, 5842: 2, 6911: 2, 1900: 3, 3265: 3, 4309: 2, 4060: 1, 5601: 1, 7438: 2, 6285: 2, 3089: 1, 5571: 2, 2469: 2, 5972: 2, 6016: 2, 2729: 2, 8136: 3, 6319: 3, 2028: 2, 6702: 2, 6921: 1, 7128: 2, 3835: 1, 2962: 3, 4467: 1, 4254: 1, 4811: 2, 7567: 2, 7529: 3, 6030: 2, 5857: 2, 144: 2, 1795: 3, 6390: 2, 7470: 1, 134: 2, 6328: 2, 5879: 3, 618: 1, 3936: 2, 325: 2, 6605: 2, 1526: 2, 915: 2, 4009: 3, 7971: 3, 6772: 3, 211: 1, 7481: 2, 7997: 3, 3942: 1, 770: 2, 2674: 3, 1793: 1, 6726: 2, 412: 1, 749: 2, 6014: 1, 2823: 1, 7894: 2, 4282: 1, 5184: 2, 2817: 1, 737: 2, 6852: 1, 230: 1, 312: 1, 1767: 3, 1550: 2, 4916: 1, 985: 3, 7080: 1, 1475: 1, 6941: 3, 880: 1, 1300: 2, 6916: 2, 6357: 3, 5198: 2, 7625: 1, 7780: 1, 1851: 1, 7676: 1, 7644: 2, 2324: 3, 3069: 1, 4746: 2, 8137: 2, 1358: 2, 3839: 3, 2460: 2, 6154: 1, 826: 2, 537: 2, 6668: 1, 5534: 1, 7435: 2, 6139: 1, 768: 1, 2664: 2, 3957: 2, 6974: 3, 1757: 3, 2398: 2, 5512: 3, 429: 1, 5909: 2, 1005: 2, 8028: 1, 5938: 2, 1700: 1, 2378: 2, 3874: 2, 2913: 2, 848: 3, 1231: 1, 7235: 1, 3224: 2, 4542: 1, 2376: 2, 2170: 2, 243: 3, 4642: 1, 1643: 2, 1496: 3, 6163: 2, 1986: 3, 525: 2, 3929: 1, 5336: 2, 1619: 1, 4606: 2, 4724: 2, 7830: 2, 1878: 1, 4900: 3, 3102: 3, 2502: 1, 4947: 3, 1653: 2, 7622: 1, 3369: 3, 5889: 1, 1022: 3, 4473: 2, 3215: 3, 2259: 2, 4987: 3, 1386: 1, 1487: 2, 8082: 1, 4504: 3, 2666: 1, 4655: 1, 1798: 3, 2984: 3, 6649: 1, 2976: 3, 918: 3, 1366: 2, 2236: 2, 1268: 3, 6703: 2, 4982: 3, 1747: 2, 426: 3, 7698: 2, 1384: 1, 4664: 3, 54: 3, 4937: 3, 3594: 2, 6816: 1, 4317: 3, 2181: 1, 1167: 3, 3007: 2, 1427: 3, 4468: 1, 2588: 3, 5427: 1, 2834: 3, 6783: 2, 6174: 3, 3283: 1, 4838: 2, 28: 1, 3855: 3, 4277: 2, 5395: 2, 3841: 1, 6476: 2, 2825: 1, 2213: 3, 1574: 2, 5190: 3, 6321: 1, 2980: 3, 762: 3, 2498: 3, 3893: 2, 375: 1, 2157: 2, 4570: 3, 3710: 1, 7870: 2, 6375: 2, 2109: 2, 5346: 3, 695: 1, 5846: 1, 5694: 1, 5575: 1, 4850: 3, 7155: 1, 4332: 2, 2140: 2, 4341: 1, 2478: 1, 4611: 2, 6317: 2, 979: 2, 6577: 1, 2142: 2, 5974: 3, 1423: 2, 465: 1, 3187: 3, 5045: 3, 4439: 1, 3084: 1, 5929: 2, 3040: 3, 6834: 2, 5683: 2, 2606: 1, 3922: 2, 7681: 1, 2238: 3, 1894: 3, 1638: 3, 5171: 1, 1247: 1, 1014: 3, 6958: 3, 799: 3, 2389: 3, 8080: 2, 4874: 1, 4543: 3, 6393: 2, 7199: 1, 4597: 3, 4063: 2, 1520: 3, 5911: 2, 4777: 2, 2630: 1, 6034: 1, 5068: 2, 5116: 2, 3423: 1, 6589: 2, 5824: 3, 3121: 2, 5779: 3, 4068: 2, 4281: 3, 1652: 3, 7784: 2, 1445: 3, 5838: 1, 814: 2, 7363: 2, 2310: 2, 344: 2, 1960: 2, 1312: 1, 6156: 2, 642: 2, 4139: 3, 958: 1, 6304: 3, 140: 2, 1935: 3, 6973: 2, 5704: 2, 2615: 1, 3446: 1, 334: 2, 3453: 1, 4366: 3, 3796: 1, 2584: 3, 1432: 2, 2273: 2, 2963: 3, 2204: 2, 7038: 2, 1778: 2, 6862: 1, 1501: 1, 527: 2, 5619: 2, 5195: 2, 3949: 3, 583: 1, 1206: 3, 2579: 1, 4798: 2, 1202: 3, 3043: 1, 2523: 1, 3078: 2, 3256: 1, 1097: 3, 5105: 1, 2643: 2, 6290: 2, 6966: 3, 5490: 3, 3287: 2, 2722: 2, 702: 2, 3689: 2, 474: 2, 1837: 2, 167: 2, 129: 2, 4180: 1, 1032: 3, 6455: 2, 6361: 3, 481: 2, 4818: 2, 4138: 1, 347: 1, 5910: 1, 4074: 3, 6804: 2, 1672: 3, 7988: 2, 5335: 1, 5539: 1, 6181: 3, 4596: 3, 6563: 1, 1000: 2, 6306: 2, 8009: 3, 7851: 1, 358: 1, 2878: 1, 4172: 1, 2528: 1, 8027: 3, 663: 1, 5469: 1, 1940: 1, 1732: 2, 982: 2, 1216: 1, 3888: 1, 1147: 2, 1013: 3, 4276: 2, 7230: 3, 2481: 3, 7204: 1, 6770: 1, 5320: 1, 5393: 1, 6325: 3, 2332: 2, 5666: 2, 3709: 1, 7161: 2, 3718: 3, 7429: 2, 1925: 3, 542: 2, 1854: 1, 5008: 2, 476: 1, 4499: 1, 3395: 1, 5351: 1, 3437: 1, 2561: 1, 4752: 2, 6711: 1, 5264: 2, 672: 2, 1956: 1, 2483: 1, 1509: 1, 4673: 2, 1788: 3, 2642: 2, 11: 1, 7147: 2, 1109: 1, 7462: 3, 606: 1, 3150: 1, 346: 2, 67: 2, 7140: 2, 6591: 3, 4876: 2, 270: 3, 4674: 3, 5: 1, 5541: 2, 1679: 2, 6432: 3, 1407: 2, 1148: 1, 2665: 2, 1692: 1, 7944: 2, 2740: 1, 4265: 3, 7408: 1, 7968: 1, 4236: 2, 1190: 1, 5086: 3, 4589: 1, 7200: 1, 7978: 1, 1192: 3, 2348: 3, 1373: 2, 6151: 1, 6898: 2, 7520: 1, 5918: 2, 6401: 1, 3272: 1, 5099: 2, 3544: 2, 8044: 1, 3603: 2, 1533: 2, 1127: 3, 5797: 2, 3172: 3, 122: 2, 4742: 3, 5766: 2, 4932: 2, 14: 1, 7589: 2, 7398: 1, 169: 2, 6614: 2, 8091: 3, 5547: 1, 4613: 3, 1944: 3, 7125: 3, 1706: 1, 35: 1, 7844: 2, 5545: 3, 3095: 1, 484: 3, 1680: 1, 1245: 3, 514: 2, 4836: 3, 3793: 1, 4293: 3, 2532: 3, 4497: 3, 1012: 3, 2470: 3, 3285: 2, 4895: 1, 2996: 1, 6383: 3, 7296: 3, 4023: 2, 6586: 2, 4733: 2, 6993: 1, 235: 1, 5175: 2, 40: 3, 5359: 1, 6782: 2, 5010: 3, 2432: 3, 5385: 1, 2097: 2, 108: 1, 4417: 3, 3449: 1, 3605: 2, 6411: 1, 7725: 1, 6142: 2, 4304: 3, 2290: 2, 6247: 2, 6377: 2, 7959: 1, 1210: 2, 4347: 3, 7568: 1, 5391: 3, 1274: 3, 7446: 3, 3743: 1, 5041: 2, 2886: 2, 573: 2, 27: 2, 611: 2, 3205: 2, 2299: 2, 5712: 2, 5891: 2, 3409: 1, 7091: 3, 931: 3, 5012: 2, 4946: 2, 4361: 1, 5853: 1, 7384: 2, 5087: 1, 5442: 2, 2328: 3, 2776: 2, 2517: 2, 7228: 1, 309: 3, 6518: 1, 2956: 1, 7603: 2, 655: 2, 2767: 2, 1371: 3, 558: 1, 5533: 3, 6837: 1, 188: 3, 6385: 1, 2268: 2, 8121: 3, 25: 3, 6358: 1, 47: 2, 1306: 2, 2138: 3, 6612: 1, 2385: 2, 7976: 2, 4102: 1, 3958: 2, 5826: 3, 613: 1, 2877: 1, 1677: 3, 238: 2, 1699: 2, 1500: 3, 4135: 3, 1666: 1, 6318: 2, 4749: 2, 5285: 3, 5908: 3, 7540: 1, 5034: 1, 7654: 2, 4413: 1, 5642: 2, 4795: 2, 2572: 2, 6623: 2, 2171: 3, 5261: 1, 1266: 1, 5772: 2, 4665: 1, 6811: 1, 2810: 1, 2900: 1, 5778: 1, 4780: 2, 7179: 1, 7848: 2, 192: 1, 4873: 1, 2316: 2, 2453: 2, 4755: 2, 6737: 1, 1955: 1, 7949: 2, 2509: 2, 7203: 1, 6960: 1, 8101: 3, 5055: 3, 817: 1, 4252: 1, 2815: 2, 6315: 2, 6938: 2, 2329: 3, 6601: 2, 3827: 1, 4143: 3, 7238: 1, 6637: 1, 986: 3, 4601: 3, 4748: 2, 2868: 3, 1450: 1, 2887: 1, 5148: 3, 1665: 1, 5001: 3, 4178: 2, 3801: 1, 1251: 1, 844: 1, 7842: 3, 7119: 1, 2073: 1, 1859: 1, 7805: 2, 2124: 3, 1964: 2, 449: 1, 1639: 2, 5160: 2, 2423: 1, 2354: 2, 7265: 2, 1922: 1, 3628: 3, 7118: 3, 4910: 2, 6068: 2, 4630: 2, 7707: 1, 5692: 2, 5458: 2, 7402: 2, 189: 1, 139: 2, 5703: 1, 354: 2, 2281: 2, 6128: 2, 5484: 1, 3854: 3, 4079: 1, 8099: 1, 103: 2, 2351: 2, 8047: 2, 5634: 2, 6796: 2, 7728: 3, 5986: 2, 4013: 3, 2185: 2, 1605: 2, 430: 2, 521: 2, 4992: 3, 7068: 2, 5169: 1, 5276: 2, 3485: 3, 1624: 2, 7166: 1, 7475: 1, 3422: 2, 5456: 3, 766: 3, 304: 3, 738: 2, 6599: 3, 6680: 2, 2094: 2, 5181: 3, 6867: 2, 1791: 2, 1996: 3, 686: 3, 6790: 1, 5605: 2, 620: 2, 1348: 3, 5517: 3, 327: 3, 2294: 3, 504: 2, 1599: 2, 1874: 1, 1363: 2, 6736: 2, 6513: 1, 517: 2, 4342: 2, 4141: 2, 5227: 1, 6886: 2, 4144: 3, 1451: 3, 1186: 3, 5997: 1, 5308: 2, 7742: 1, 6690: 2, 4015: 3, 3747: 2, 751: 2, 5454: 1, 4956: 2, 4786: 3, 1877: 3, 857: 3, 5129: 2, 498: 3, 5989: 1, 5402: 2, 1283: 2, 5325: 1, 6878: 2, 1846: 2, 6001: 1, 0: 2, 155: 1, 3643: 1, 4005: 2, 3430: 1, 1431: 2, 4039: 3, 536: 1, 4794: 2, 1250: 2, 4867: 3, 974: 2, 6800: 3, 3719: 2, 540: 2, 2089: 1, 3728: 1, 2804: 1, 7488: 1, 5561: 2, 6615: 2, 5093: 3, 396: 1, 5232: 2, 4209: 1, 4605: 3, 4103: 1, 5508: 3, 7858: 3, 4582: 1, 187: 1, 3191: 1, 4771: 3, 5702: 3, 1954: 1, 3385: 1, 6496: 2, 137: 1, 7051: 1, 7916: 2, 2001: 2, 2897: 2, 8069: 2, 84: 3, 6012: 3, 5998: 2, 7415: 1, 6103: 1, 6277: 2, 3115: 2, 6353: 3, 544: 3, 6933: 2, 4478: 2, 2979: 3, 5673: 3, 8076: 2, 2565: 2, 8135: 2, 6808: 1, 4796: 3, 7980: 2, 1488: 1, 1840: 3, 1710: 2, 6740: 1, 7206: 1, 6083: 1, 3276: 2, 6420: 2, 623: 2, 3334: 1, 1138: 1, 7946: 3, 937: 2, 6723: 2, 3956: 3, 3382: 1, 6743: 1, 5015: 2, 4633: 1, 4469: 1, 7904: 2, 7936: 3, 1507: 2, 85: 3, 3134: 3, 6008: 3, 1508: 2, 7314: 1, 2380: 2, 6146: 2, 2968: 2, 7740: 2, 4995: 1, 5083: 1, 2751: 2, 1749: 1, 665: 2, 4266: 1, 5356: 2, 8023: 1, 4284: 2, 2167: 1, 2680: 2, 2560: 2, 683: 2, 2623: 1, 3077: 1, 2417: 2, 4881: 2, 5839: 2, 5000: 2, 2263: 1, 2228: 1, 5727: 2, 1797: 3, 6664: 1, 1770: 2, 6470: 2, 1107: 2, 6287: 2, 713: 1, 1704: 2, 5928: 1, 4485: 1, 7349: 1, 903: 2, 1163: 2, 2276: 3, 5323: 2, 2240: 2, 2503: 2, 6007: 2, 5066: 2, 2068: 1, 2513: 3, 4307: 3, 3962: 2, 3236: 1, 1934: 2, 7445: 2, 8019: 2, 5917: 3, 563: 3, 7116: 1, 3650: 1, 4539: 1, 1070: 1, 120: 2, 2323: 2, 3325: 2, 2421: 2, 4903: 3, 2609: 3, 8098: 1, 1393: 1, 2282: 1, 2593: 2, 806: 3, 3852: 3, 5944: 3, 626: 1, 2698: 2, 1920: 2, 1836: 1, 7385: 2, 4258: 2, 6894: 2, 6565: 2, 3329: 1, 3673: 1, 5146: 1, 8078: 2, 3917: 1, 4645: 3, 5637: 1, 3250: 2, 6405: 3, 1144: 1, 539: 3, 4126: 1, 2016: 1, 5213: 2, 5443: 1, 416: 2, 5265: 1, 274: 2, 1010: 3, 5473: 2, 6352: 3, 674: 1, 457: 3, 2869: 3, 7923: 1, 150: 3, 1942: 2, 6674: 1, 5761: 3, 7692: 2, 7375: 2, 2267: 2, 5983: 2, 4809: 3, 6696: 2, 7829: 2, 377: 2, 5968: 1, 3584: 2, 4744: 2, 383: 1, 1564: 3, 221: 3, 3180: 2, 5334: 2, 7572: 2, 4348: 3, 4344: 2, 470: 1, 2566: 2, 1525: 3, 756: 1, 7095: 2, 6858: 2, 341: 1, 3319: 1, 5361: 1, 3681: 1, 7992: 3, 2414: 2, 3066: 2, 6760: 2, 329: 1, 5506: 3, 6426: 2, 20: 3, 2415: 3, 1865: 1, 7272: 3, 2500: 3, 1470: 2, 2217: 1, 2641: 1, 1839: 3, 7377: 1, 1378: 3, 6048: 1, 6706: 1, 1276: 1, 7381: 1, 5290: 2, 7803: 1, 3889: 3, 3201: 2, 64: 3, 1154: 2, 6789: 1, 4553: 2, 434: 2, 3804: 2, 6454: 1, 2925: 2, 3626: 2, 912: 3, 4379: 2, 7838: 2, 2067: 1, 1415: 2, 380: 2, 2430: 2, 1446: 2, 4298: 1, 2365: 2, 6826: 2, 3638: 2, 3588: 1, 3517: 1, 3637: 3, 4668: 3, 5255: 1, 423: 1, 1776: 1, 5751: 2, 4110: 2, 2629: 2, 6688: 1, 3213: 1, 6349: 2, 6443: 3, 7108: 3, 4913: 3, 4638: 3, 5118: 3, 6659: 1, 4952: 1, 4770: 3, 7219: 2, 1150: 1, 5133: 3, 6207: 1, 3034: 3, 4399: 3, 3254: 1, 3161: 1, 2888: 2, 1254: 3, 927: 2, 6912: 1, 3670: 1, 670: 1, 1354: 3, 2657: 2, 466: 1, 5294: 2, 6059: 3, 5801: 1, 6236: 2, 2285: 3, 2885: 2, 1321: 1, 462: 1, 1762: 3, 2510: 3, 4034: 2, 7492: 3, 651: 1, 4291: 3, 591: 1, 1753: 1, 1634: 1, 5936: 1, 4228: 3, 696: 2, 5602: 1, 7358: 2, 3519: 3, 4671: 2, 1540: 2, 6700: 2, 5609: 1, 5328: 2, 6077: 2, 4773: 1, 2499: 3, 405: 1, 5610: 3, 308: 3, 2881: 3, 2748: 1, 2225: 2, 3211: 1, 2086: 2, 199: 2, 2322: 3, 5319: 3, 2393: 3, 2250: 3, 5371: 2, 3895: 2, 8100: 1, 6675: 2, 4944: 3, 1164: 2, 6903: 1, 6821: 2, 1346: 2, 7379: 3, 5132: 2, 1024: 3, 4757: 3, 4156: 2, 7835: 3, 5660: 2, 1031: 1, 2118: 1, 5643: 2, 1532: 2, 306: 3, 2309: 1, 6192: 1, 3736: 1, 1241: 3, 2960: 2, 5156: 2, 7145: 1, 5257: 2, 7286: 2, 4648: 2, 2521: 3, 4227: 3, 3370: 1, 5558: 3, 4782: 2, 1480: 3, 7464: 1, 575: 3, 4464: 3, 5401: 2, 1547: 1, 1214: 2, 545: 2, 8014: 2, 5212: 2, 5222: 1, 2222: 2, 6636: 1, 261: 1, 7158: 1, 7211: 1, 3816: 1, 1293: 3, 2280: 2, 3273: 2, 3708: 1, 5738: 1, 4275: 2, 445: 1, 1287: 1, 135: 2, 584: 2, 7502: 1, 4785: 2, 7310: 1, 1273: 2, 7467: 1, 5661: 3, 3204: 3, 6616: 2, 1660: 1, 5061: 2, 2766: 2, 2471: 2, 7757: 3, 5768: 2, 8094: 3, 2494: 2, 926: 3, 2839: 2, 6006: 2, 2425: 3, 2656: 3, 432: 2, 7861: 2, 4290: 2, 4336: 1, 1735: 1, 5940: 1, 1588: 1, 7469: 1, 6463: 2, 3168: 1, 5767: 1, 3777: 1, 6854: 1, 3826: 2, 180: 2, 5522: 2, 3383: 2, 605: 2, 640: 2, 589: 2, 4207: 2, 1949: 3, 6101: 2, 5115: 2, 2403: 3, 7079: 2, 3137: 1, 2564: 2, 1252: 2, 2120: 1, 1059: 3, 3923: 2, 8071: 3, 494: 2, 7673: 3, 7641: 1, 209: 2, 4998: 1, 4625: 2, 1364: 1, 5973: 2, 6314: 1, 4968: 2, 4768: 2, 1331: 1, 994: 1, 8003: 2, 5077: 1, 3999: 1, 371: 3, 6890: 2, 2011: 2, 4991: 3, 7336: 1, 4904: 2, 6502: 2, 1980: 1, 841: 3, 4078: 2, 6153: 2, 4127: 2, 891: 2, 2805: 2, 1194: 2, 4688: 2, 7660: 3, 703: 2, 5073: 3, 5157: 2, 5820: 2, 2577: 3, 4559: 3, 2360: 3, 6909: 2, 1693: 2, 2179: 3, 4541: 1, 61: 2, 3627: 3, 715: 1, 6011: 1, 6086: 1, 2112: 1, 3831: 1, 6291: 2, 1618: 1, 1519: 1, 79: 2, 627: 2, 1195: 2, 5776: 3, 1779: 2, 3557: 1, 7815: 3, 3160: 2, 2491: 2, 5315: 1, 3037: 1, 3399: 2, 1991: 1, 3018: 1, 1043: 3, 1923: 3, 990: 2, 7836: 3, 7112: 3, 379: 2, 4004: 3, 4069: 1, 4017: 2, 8040: 2, 5613: 3, 8083: 2, 2683: 2, 4870: 2, 71: 2, 1320: 3, 5615: 3, 2568: 2, 8090: 2, 7455: 2, 2199: 1, 5681: 3, 1338: 3, 3284: 1, 7326: 2, 2859: 2, 451: 2, 1339: 1, 549: 1, 2277: 2, 1849: 2, 406: 1, 5043: 2, 6644: 2, 7013: 3, 311: 3, 6127: 3, 1286: 2, 7689: 1, 4292: 2, 106: 1, 2350: 2, 7418: 2, 7893: 2, 3648: 2, 4647: 3, 547: 2, 4737: 3, 4882: 3, 5452: 2, 222: 2, 3919: 1, 2487: 2, 1083: 1, 6109: 1, 5376: 1, 4808: 3, 6468: 3, 2862: 1, 948: 3, 2207: 2, 7769: 1, 4140: 2, 7748: 1, 6430: 1, 6312: 1, 6180: 1, 6000: 1, 4574: 2, 4150: 3, 1780: 3, 7111: 2, 7859: 3, 5730: 1, 2948: 1, 7985: 3, 593: 2, 305: 2, 5219: 1, 7345: 1, 7969: 1, 4886: 1, 3218: 2, 4619: 3, 1498: 1, 7071: 1, 66: 2, 6052: 1, 7831: 3, 8039: 1, 2832: 3, 7100: 3, 5662: 1, 808: 1, 4137: 1, 5430: 3, 2540: 2, 2203: 3, 7869: 1, 7813: 2, 1464: 2, 1362: 3, 7374: 2, 7339: 1, 5950: 2, 5150: 2, 632: 1, 2770: 2, 2775: 2, 4632: 3, 7559: 1, 5459: 2, 6794: 2, 3938: 1, 1688: 3, 714: 1, 3690: 2, 7323: 1, 3844: 3, 6779: 2, 4: 1, 628: 2, 6462: 2, 7595: 1, 9: 2, 2347: 3, 381: 3, 3488: 1, 471: 1, 4627: 2, 7485: 1, 4076: 2, 3766: 3, 509: 2, 1842: 3, 7503: 1, 4983: 1, 5582: 1, 2928: 1, 4170: 2, 991: 2, 3879: 1, 5544: 2, 2182: 2, 2020: 2, 5577: 2, 7915: 2, 5817: 1, 1232: 2, 2554: 3, 2082: 1, 475: 2, 7491: 2, 4142: 3, 6323: 3, 7639: 2, 5635: 2, 1486: 3, 6851: 1, 4804: 3, 1927: 3, 6888: 2, 2131: 2, 5589: 2, 479: 2, 349: 2, 1357: 2, 5388: 1, 8116: 3, 3055: 2, 3052: 2, 4028: 1, 1557: 3, 1826: 1, 1654: 3, 5588: 3, 6309: 2, 5872: 2, 7016: 1, 4248: 1, 5537: 3, 459: 1, 5200: 2, 4054: 3, 3658: 1, 2806: 3, 1668: 3, 2258: 1, 4173: 1, 2525: 2, 5445: 1, 7528: 2, 3773: 1, 3780: 2, 53: 2, 3577: 1, 2902: 1, 2727: 2, 3209: 1, 4267: 2, 4496: 1, 5865: 2, 5373: 1, 5787: 1, 6217: 3, 7332: 1, 5369: 3, 7246: 1, 5864: 2, 7369: 1, 3635: 1, 5005: 2, 4449: 2, 2977: 3, 7407: 3, 7845: 1, 4721: 3, 7267: 2, 1908: 2, 4907: 2, 194: 1, 2924: 2, 227: 1, 5024: 2, 1166: 3, 2092: 2, 276: 2, 8089: 3, 2218: 1, 624: 3, 3639: 1, 1906: 3, 6574: 1, 5527: 1, 6631: 1, 656: 3, 1829: 1, 4572: 2, 1199: 1, 2843: 2, 950: 1, 2982: 1, 4620: 3, 7918: 3, 2172: 2, 1667: 1, 1317: 2, 2057: 2, 5139: 1, 356: 2, 231: 2, 4515: 3, 661: 3, 1783: 2, 7828: 2, 1818: 1, 4686: 2, 3587: 2, 4389: 1, 1442: 1, 2269: 2, 7504: 3, 945: 2, 4112: 2, 1609: 3, 8000: 1, 4149: 1, 1816: 1, 3147: 1, 3020: 2, 6527: 3, 5188: 3, 8033: 1, 5747: 2, 2702: 2, 4509: 1, 3665: 3, 241: 3, 4082: 1, 716: 3, 8092: 1, 1625: 1, 287: 2, 2366: 2, 5798: 2, 8020: 1, 7046: 2, 2694: 3, 1482: 1, 4168: 3, 6846: 1, 2669: 2, 2844: 3, 1189: 3, 2435: 2, 4931: 2, 5266: 2, 5051: 3, 794: 1, 7661: 2, 3311: 3, 5201: 2, 4854: 2, 382: 3, 865: 2, 439: 3, 136: 2, 2472: 1, 7076: 2, 4516: 2, 4821: 2, 6925: 2, 1463: 2, 5044: 3, 5140: 2, 3953: 1, 1804: 3, 4136: 3, 4077: 2, 2321: 3, 1687: 2, 4687: 1, 4123: 1, 5907: 1, 2504: 2, 5684: 1, 1870: 2, 7881: 1, 1701: 2, 4779: 2, 8043: 3, 1541: 1, 1528: 2, 4339: 1, 7778: 1, 6777: 1, 3848: 3, 4262: 2, 4776: 2, 7417: 1, 1176: 1, 1890: 2, 7943: 2, 1118: 1, 3376: 1, 421: 3, 7984: 3, 2507: 2, 6865: 1, 2658: 1, 3574: 2, 2999: 2, 2783: 1, 254: 1, 1743: 2, 7075: 1, 6359: 2, 4391: 2, 7397: 2, 1678: 1, 7789: 3, 3459: 1, 118: 1, 2546: 3, 1848: 2, 4322: 1, 3412: 2, 6400: 2, 2845: 2, 962: 2, 4396: 1, 6209: 1, 3342: 1, 8107: 1, 1947: 3, 4169: 1, 8134: 2, 2447: 2, 7598: 1, 3245: 2, 4820: 3, 3000: 1, 4823: 2, 7932: 2, 3974: 2, 7867: 1, 2355: 2, 2125: 1, 2704: 2, 681: 2, 4042: 2, 1067: 1, 831: 2, 7395: 1, 164: 2, 52: 1, 4211: 2, 4010: 2, 5559: 3, 6450: 3, 718: 2, 6689: 3, 7297: 1, 352: 1, 4531: 2, 6920: 1, 7229: 1, 949: 2, 813: 3, 4554: 1, 5088: 1, 2291: 2, 643: 2, 8015: 1, 5883: 1, 5888: 1, 976: 1, 7754: 2, 3297: 2, 1052: 1, 7796: 1, 300: 1, 5632: 1, 1476: 3, 7404: 1, 228: 2, 5736: 2, 7183: 1, 4133: 1, 5186: 2, 4115: 1, 5383: 2, 7135: 2, 6594: 1, 4486: 2, 8132: 1, 5312: 2, 854: 3, 951: 2, 1323: 2, 653: 1, 5557: 2, 7605: 1, 6841: 1, 5382: 1, 7382: 2, 1868: 3, 6106: 2, 2847: 3, 249: 3, 633: 2, 566: 2, 3470: 1, 2535: 2, 6136: 3, 7615: 2, 6913: 3, 1741: 1, 7052: 3, 4840: 2, 7762: 1, 4951: 2, 699: 2, 2438: 1, 5183: 2, 1977: 2, 5933: 3, 7864: 2, 2558: 2, 158: 2, 5002: 1, 4702: 1, 5566: 1, 3005: 1, 1318: 3, 4145: 2, 3083: 1, 2621: 2, 4594: 2, 1356: 1, 6225: 2, 1575: 2, 7937: 1, 2626: 3, 7771: 3, 1733: 1, 4014: 1, 323: 2, 142: 2, 90: 2, 4851: 3, 4734: 3, 5513: 1, 7511: 1, 6196: 2, 629: 3, 1752: 2, 5100: 1, 1898: 2, 5641: 1, 1862: 1, 362: 1, 5292: 1, 2177: 1, 4273: 2, 5236: 3, 1391: 2, 2136: 1, 8042: 2, 7432: 2, 5180: 2, 4021: 3, 5101: 3, 5130: 2, 3512: 1, 6963: 2, 587: 1, 2298: 2, 81: 1, 4609: 3, 4966: 1, 4676: 1, 5064: 2, 932: 1, 1387: 3, 6345: 1, 7679: 1, 4305: 2, 1040: 1, 4852: 3, 6336: 2, 3947: 1, 1989: 3, 3990: 2, 5124: 3, 3828: 3, 4964: 1, 6224: 1, 5565: 1, 1601: 1, 5223: 1, 910: 3, 757: 2, 4743: 3, 2774: 1, 2646: 2, 3791: 1, 1781: 3, 7278: 1, 2035: 2, 6132: 2, 5431: 1, 3565: 1, 5580: 2, 863: 3, 5898: 1, 265: 1, 541: 3, 3447: 1, 5049: 2, 650: 3, 2648: 1, 4238: 2, 2837: 2, 4863: 2, 4107: 2, 1086: 1, 3772: 1, 1489: 3, 7396: 1, 4756: 1, 1460: 2, 3230: 1, 4174: 1, 8072: 3, 2111: 2, 965: 2, 3038: 1, 3088: 1, 2700: 1, 1428: 1, 7087: 1, 2333: 3, 5568: 1, 5179: 3, 2627: 1, 4766: 2, 1389: 3, 791: 2, 3389: 2, 5380: 2, 7277: 3, 331: 1, 6406: 2, 5418: 2, 2003: 1, 4839: 3, 5477: 1, 758: 2, 694: 1, 1774: 1, 5039: 1, 5214: 3, 8130: 2, 6980: 3, 3474: 2, 6039: 1, 6337: 1, 366: 2, 2099: 1, 4312: 2, 3898: 1, 4369: 1, 4187: 1, 3904: 2, 1207: 3, 2272: 2, 7271: 1, 294: 2, 1134: 2, 3514: 1, 478: 1, 4787: 3, 63: 2, 721: 1, 213: 1, 7684: 2, 3461: 2, 3059: 2, 2419: 3, 6949: 2, 675: 1, 5954: 1, 8086: 2, 2495: 1, 7121: 2, 2929: 1, 6538: 2, 6429: 2, 840: 2, 3406: 2, 1761: 1, 4884: 2, 5926: 3, 6524: 1, 7110: 2, 2529: 2, 301: 3, 110: 3, 6875: 1, 1737: 3, 1662: 2, 6997: 3, 942: 3, 1228: 2, 2214: 2, 1611: 1, 3585: 1, 615: 2, 5822: 2, 16: 2, 3676: 1, 1607: 1, 3060: 1, 5052: 1, 5553: 1, 8125: 2, 7317: 1, 5515: 2, 4969: 1, 3009: 2, 7863: 1, 7569: 3, 1091: 3, 2159: 3, 6033: 1, 6273: 1, 897: 3, 310: 2, 7643: 2, 769: 2, 3277: 1, 7607: 1, 7727: 2, 7172: 1, 5194: 1, 5975: 2, 5486: 1, 2255: 1, 6638: 2, 1342: 2, 3892: 1, 1910: 3, 2104: 2, 5341: 1, 8031: 2, 7162: 2, 6138: 1, 853: 2, 1280: 3, 1951: 2, 6042: 1, 693: 3, 82: 1, 946: 3, 2362: 1, 1177: 1, 4326: 3, 552: 1, 7006: 2, 7703: 3, 4048: 3, 5952: 1, 2967: 2, 7766: 2, 8124: 3, 1351: 3, 2769: 1, 3860: 3, 3153: 1, 6403: 1, 8114: 2, 7010: 1, 4476: 1, 3646: 2, 5915: 2, 4919: 2, 8129: 3, 5441: 1, 4338: 2, 1490: 2, 821: 3, 5059: 3, 7347: 2, 288: 2, 1278: 1, 2320: 1, 707: 2, 7508: 1, 2081: 3, 7565: 1, 264: 2, 5474: 3, 5080: 2, 6399: 2, 8037: 2, 4915: 3, 4132: 1, 818: 2, 7792: 3, 608: 2, 7860: 2, 4491: 1, 2226: 2, 184: 1, 2326: 3, 1819: 3, 4261: 2, 7007: 2, 7315: 1, 3586: 1, 1658: 2, 1538: 2, 7902: 2, 6348: 2, 2597: 2, 2720: 1, 7220: 1, 3219: 1, 833: 2, 5599: 2, 6190: 1, 5784: 1, 83: 1, 7553: 2, 5563: 1, 1844: 1, 4639: 2, 2135: 1, 1340: 2, 6809: 3, 610: 3, 2241: 3, 2428: 2, 679: 1, 788: 2, 2750: 2, 4165: 3, 6778: 1, 3146: 1, 480: 3, 6286: 1, 147: 2, 8105: 1, 5085: 1, 339: 2, 1585: 1, 3738: 2, 2506: 2, 7671: 1, 3937: 2, 7073: 2, 2424: 2, 262: 2, 3901: 1, 5054: 2, 2864: 3, 6022: 2, 2812: 1, 7665: 2, 6447: 3, 3602: 3, 2002: 2, 2663: 1, 4775: 2, 6685: 3, 3813: 2, 7647: 3, 4513: 1, 5496: 2, 6650: 2, 3366: 1, 3915: 1, 3269: 1, 7017: 2, 2931: 2, 133: 2, 7941: 2, 3926: 2, 2188: 2, 6753: 1, 4129: 3, 6302: 1, 7961: 1, 3765: 3, 4641: 2, 6936: 1, 1562: 2, 5560: 1, 6868: 1, 2563: 3, 4615: 3, 2828: 1, 7931: 2, 1845: 3, 2518: 1, 275: 1, 820: 3, 4547: 1, 7117: 2, 1883: 2, 7862: 2, 5939: 2, 3711: 2, 267: 1, 4192: 2, 6397: 2, 237: 3, 3372: 1, 4492: 2, 2575: 1, 2242: 1, 940: 2, 6739: 3, 8106: 1, 3669: 1, 8058: 1, 5570: 1, 5296: 3, 2061: 1, 2744: 2, 7: 3, 5309: 2, 2691: 1, 4635: 1, 204: 2, 3341: 3, 6427: 2, 6671: 1, 229: 3, 6825: 3, 2450: 2, 896: 1, 4044: 3, 4022: 3, 154: 1, 4814: 2, 7437: 1, 6784: 3, 2254: 3, 4314: 2, 353: 3, 4993: 1, 123: 2, 4722: 3, 2050: 1, 5806: 2, 422: 1, 829: 3, 6334: 3, 4706: 2, 6835: 2, 4510: 2, 3326: 2, 7045: 1, 3457: 2, 2696: 1, 6926: 2, 7791: 2, 3411: 1, 4883: 3, 4527: 1, 395: 1, 7819: 3, 3739: 2, 2449: 2, 6446: 1, 3686: 1, 7257: 1, 5870: 2, 161: 1, 1616: 2, 1582: 2, 3207: 1, 5871: 2, 2873: 2, 5633: 2, 1057: 2, 2306: 1, 5271: 1, 7556: 1, 5869: 2, 1101: 1, 7186: 2, 3842: 2, 2233: 2, 3185: 1, 1648: 1, 7755: 2, 7058: 1, 2369: 2, 7963: 2, 7212: 2, 7513: 3, 5745: 1, 4750: 2, 2072: 1, 4805: 2, 6699: 1, 56: 1, 5969: 1, 3098: 1, 7185: 1, 5238: 2, 4000: 1, 5363: 1, 842: 3, 4677: 2, 708: 2, 6896: 1, 62: 1, 3900: 3, 5354: 3, 4667: 1, 6289: 1, 810: 3, 2338: 3, 5103: 2, 6091: 2, 2452: 2, 3821: 1, 3189: 1, 1309: 2, 7975: 2, 1422: 2, 2007: 1, 6388: 1, 5844: 1, 5235: 1, 7188: 1, 4089: 3, 2541: 3, 2260: 1, 29: 2, 441: 2, 289: 1, 5775: 2, 1233: 2, 6592: 1, 5617: 2, 5375: 2, 5647: 2, 485: 2, 1994: 2, 7714: 2, 2964: 3, 7582: 1, 7150: 2, 6221: 2, 1045: 2, 5020: 1, 4062: 3, 5014: 3, 7243: 2, 2879: 1, 273: 1, 7909: 1, 8096: 2, 1111: 2, 5076: 2, 5483: 1, 1: 2, 4812: 2, 1821: 2, 5987: 1, 6647: 1, 1038: 2, 5674: 2, 6271: 1, 7811: 2, 1561: 1, 921: 3, 6046: 2, 3142: 1, 4423: 1, 1076: 2, 5873: 1, 5658: 1, 463: 1, 2830: 2, 3570: 2, 3452: 1, 7334: 1, 6859: 1, 1823: 3, 5352: 1, 5306: 2, 4019: 2, 4333: 2, 3615: 1, 5714: 1, 281: 3, 2334: 3, 5773: 1, 3330: 1, 4006: 3, 4646: 1, 4295: 2, 316: 1, 393: 1, 4164: 1, 5878: 2, 4403: 3, 7499: 1, 7736: 2, 659: 1, 7618: 2, 4837: 3, 86: 2, 6508: 1, 2027: 2, 7386: 1, 755: 2, 4949: 2, 1419: 2, 601: 1, 4162: 3, 4097: 2, 1438: 1, 2223: 2, 534: 1, 2473: 2, 386: 3, 585: 2, 2257: 3, 6830: 1, 7989: 2, 4815: 2, 370: 1, 970: 1, 6536: 1, 389: 1, 6249: 1, 5659: 3, 2251: 2, 7575: 1, 255: 2, 7279: 1, 999: 2, 7922: 3, 7198: 1, 1425: 2, 3462: 1, 4540: 2, 2070: 2, 1802: 3, 7370: 1, 7773: 1, 886: 2, 6648: 1, 2484: 2, 2688: 2, 6813: 3, 2785: 1, 5652: 2, 444: 2, 5091: 1, 5170: 3, 3959: 1, 4225: 3, 6581: 1, 1436: 2, 5876: 1, 1513: 1, 1510: 2, 2229: 3, 4084: 1, 3004: 1, 3071: 1, 4155: 2, 258: 1, 4205: 2, 939: 1, 5492: 2, 5353: 2, 3008: 1, 2505: 2, 3909: 3, 6228: 1, 41: 1, 5344: 1, 765: 3, 943: 2, 5161: 1, 1179: 3, 1136: 2, 2985: 1, 3868: 1, 7546: 2, 2434: 2, 662: 2, 1090: 1, 6222: 1, 1104: 2, 7730: 1, 4239: 2, 3572: 1, 4551: 1, 4990: 2, 1165: 1, 4718: 3, 850: 3, 7298: 1, 1408: 2, 1334: 2, 7032: 1, 2163: 1, 5163: 1, 3998: 1, 4819: 3, 6534: 1, 5206: 2, 2404: 2, 5082: 1, 6254: 1, 6064: 2, 6085: 2, 6988: 1, 2127: 1, 8126: 2, 6479: 1, 7588: 1, 5961: 1, 5167: 1, 1553: 1, 7227: 3, 1537: 1, 7489: 1, 5350: 1, 6185: 1, 5461: 1, 682: 2, 2451: 2, 5239: 2, 7817: 2, 165: 1, 4505: 2, 7143: 2, 5774: 3, 4065: 1, 4315: 2, 6167: 2, 1285: 1, 2446: 2, 1911: 2, 3290: 2, 6464: 3, 1257: 2, 3751: 1, 6695: 1, 2391: 2, 649: 1, 7453: 1, 3897: 1, 6531: 1, 4346: 2, 2318: 2, 6802: 1, 3198: 2, 5360: 2, 2387: 1, 957: 2, 5367: 1, 559: 1, 4603: 2, 50: 1, 3336: 2, 4871: 1, 4973: 3, 5074: 1, 1030: 1, 4855: 1, 6633: 1, 952: 2, 2165: 2, 6466: 1, 2202: 3, 5603: 2, 8085: 2, 5199: 2, 2731: 2, 7138: 1, 1807: 3, 1864: 3, 4094: 2, 5314: 1, 6178: 1, 5955: 2, 996: 3, 7518: 3, 7648: 1, 131: 2, 733: 2, 2799: 1, 2685: 1, 8049: 1, 3050: 2, 5433: 3, 2520: 2, 5501: 1, 1919: 3, 6492: 2, 4662: 1, 2420: 2, 7411: 3, 1970: 1, 7541: 1, 5488: 1, 8122: 2, 1820: 2, 3716: 2, 7282: 1, 4386: 2, 4427: 3, 2442: 3, 2872: 2, 5608: 2, 3750: 1, 7431: 3, 200: 3, 3216: 1, 6448: 3, 6588: 1, 7560: 1, 6596: 1, 3952: 2, 5509: 2, 2854: 2, 4698: 1, 1879: 2, 6434: 1, 2662: 2, 3815: 1, 6259: 1, 6343: 1, 4153: 2, 5874: 1, 7774: 3, 6230: 2, 5579: 3, 314: 2, 4470: 1, 7201: 3, 2571: 2, 2209: 2, 1602: 2, 1809: 2, 5097: 2, 7416: 1, 5971: 1, 6248: 1, 6140: 3, 1055: 1, 7346: 3, 4051: 1, 908: 3, 2302: 2, 784: 2, 828: 2, 338: 2, 6994: 2, 7025: 1, 8: 1, 2400: 2, 7760: 1, 4117: 3, 1646: 1, 1633: 1, 892: 2, 2220: 1, 1987: 2, 6374: 1, 7239: 1, 2857: 1, 1552: 1, 4433: 1, 4095: 1, 7799: 1, 1825: 2, 6440: 1, 1327: 1, 7303: 1, 3548: 1, 4426: 2, 5233: 1, 2604: 1, 279: 1, 2312: 2, 5754: 3, 3117: 2, 3744: 1, 2132: 2, 2008: 2, 6897: 2, 7957: 1, 4397: 2, 1152: 1, 3266: 2, 6311: 2, 6998: 2, 3036: 2, 3608: 2, 4896: 1, 6367: 3, 7134: 2, 3869: 1, 6119: 1, 295: 2, 1559: 2, 3353: 2, 5996: 2, 4898: 3, 2595: 1, 3039: 1, 413: 1, 3946: 1, 5420: 1, 1370: 2, 303: 3, 4032: 2, 2943: 1, 4300: 3, 1222: 2, 1157: 1, 2133: 2, 7005: 2, 121: 1, 7479: 3, 6121: 3, 3597: 2, 5056: 2, 4430: 2, 1889: 2, 4558: 1, 3431: 2, 2194: 2, 5429: 2, 4487: 1, 1713: 1, 2906: 1, 4160: 2, 8131: 2, 3550: 1, 2752: 1, 5042: 2, 588: 2, 5701: 3, 812: 3, 4858: 2, 6165: 3, 3223: 3, 4887: 2, 4556: 2, 6501: 3, 4171: 1, 5243: 1, 3805: 1, 3025: 1, 687: 1, 6593: 1, 1764: 1, 7359: 1, 7669: 1, 1120: 2, 3497: 1, 455: 1, 220: 2, 6931: 2, 2681: 2, 3225: 2, 2440: 2, 6725: 2, 4918: 2, 3667: 1, 5706: 2, 4659: 1, 571: 2, 7781: 2, 3262: 1, 4751: 2, 7933: 1, 619: 3, 8060: 3, 5128: 2, 7977: 3, 2672: 3, 2708: 1, 2919: 1, 2819: 1, 6070: 1, 2152: 1, 3111: 1, 266: 1, 4404: 1, 4146: 1, 2998: 1, 5244: 1, 1221: 1, 3454: 2, 6320: 1, 1913: 2, 1041: 2, 6503: 1, 7387: 2, 1102: 2, 2946: 1, 5953: 2, 1773: 2, 2319: 3, 7208: 3, 1037: 1, 2031: 1, 293: 1, 3171: 1, 3387: 2, 4512: 3, 4740: 3, 609: 2, 4578: 3, 197: 1, 3614: 1, 5667: 1, 5708: 1, 5520: 2, 2014: 1, 7591: 1, 7807: 1, 5927: 1, 7451: 1, 4538: 2, 3647: 1, 3818: 1, 839: 3, 7283: 2, 394: 2, 2921: 1, 7716: 1, 4583: 2, 236: 3, 1961: 2, 94: 2, 2549: 3, 3993: 1, 7764: 1, 4395: 1, 3378: 1, 5234: 1, 1028: 2, 5894: 2, 1269: 2, 1019: 1, 4892: 2, 369: 2, 4552: 1, 6795: 1, 4371: 3, 3158: 1, 5379: 2, 7014: 1, 4889: 1, 2756: 1, 6486: 1, 4271: 1, 2353: 2, 5836: 1, 881: 2, 6145: 2, 4681: 2, 617: 1, 2574: 1, 2493: 1, 800: 1, 7512: 1, 5999: 1, 6556: 3, 320: 1, 711: 2, 4058: 2, 556: 2, 889: 2, 8059: 1, 760: 2, 2363: 2, 1400: 1, 5562: 2, 1784: 3, 2335: 2, 1511: 1, 5897: 1, 6120: 2, 5125: 2, 7717: 1, 1881: 3, 1969: 3, 7621: 2, 6667: 1, 1397: 3, 1589: 2, 7951: 1, 5331: 1, 947: 1, 5416: 1, 4769: 1, 5631: 1, 3500: 1, 804: 3, 7967: 2, 4226: 2, 3094: 3, 1433: 2, 6134: 1, 3798: 1, 4105: 1, 1517: 1, 472: 2, 1441: 1, 2047: 1, 5372: 1, 4669: 1, 4825: 3, 6451: 2, 1495: 1, 4214: 1, 4296: 1, 2762: 1, 1988: 1, 3606: 2, 6937: 2, 453: 1, 3611: 1, 1993: 2, 6239: 1, 6124: 1, 3546: 1, 1062: 2, 5476: 2, 7954: 2, 7498: 2, 5302: 2, 4938: 2, 5780: 2, 4940: 3, 6514: 2, 461: 1, 2620: 2, 7195: 1, 1071: 1, 3671: 1, 5828: 2, 7685: 1, 6548: 1, 7036: 1, 2379: 2, 6396: 1, 3472: 1, 7055: 1, 4388: 2, 7552: 1, 4792: 3, 1594: 3, 6709: 2, 7711: 1, 4158: 1, 5155: 2, 7184: 2, 4604: 3, 7487: 1, 2741: 1, 3491: 1, 6188: 2, 3286: 1, 3996: 2, 5003: 2, 419: 2, 8113: 1, 7883: 1, 4382: 1, 864: 3, 6274: 1, 2534: 1, 4653: 3, 1458: 3, 1926: 2, 3356: 2, 7276: 2, 5403: 2, 3321: 3, 5159: 2, 2596: 1, 4373: 1, 2265: 2, 3498: 2, 1640: 2, 3096: 1, 3832: 1, 4712: 1, 2416: 3, 7979: 2, 7420: 3, 7744: 1, 6067: 1, 6986: 1, 7050: 1, 7812: 3, 6892: 1, 700: 2, 3598: 1, 7555: 1, 3965: 1, 6860: 1, 6831: 1, 5036: 1, 8032: 1, 4644: 1, 5503: 1, 7231: 3, 1901: 1, 3788: 1, 1063: 3, 5436: 1, 1310: 1, 3580: 2, 4301: 1, 6630: 2, 7663: 1, 4713: 2, 3694: 1, 435: 1, 7163: 1, 4564: 2, 7149: 1, 5022: 1, 3757: 1, 4358: 1, 5945: 2, 1708: 2, 5670: 1, 5626: 2, 1394: 2, 7515: 2, 5519: 1, 5172: 1, 5697: 2, 1151: 1, 1264: 1, 7015: 1, 4100: 1, 6267: 1, 1739: 2, 6832: 1, 1003: 1, 1512: 2, 3429: 2, 181: 1, 5464: 2, 4747: 2, 890: 3, 6015: 1, 5902: 2, 4793: 1, 5554: 1, 1182: 3, 2066: 1, 1382: 2, 6517: 1, 6755: 1, 747: 1, 3463: 1, 1861: 1, 7981: 3, 2401: 3, 4976: 2, 3082: 2, 2436: 2, 3436: 1, 2855: 1, 1627: 2, 3960: 2, 3167: 1, 6780: 1, 3494: 1, 2100: 2, 2406: 2, 244: 1, 2210: 2, 1841: 1, 178: 1, 4920: 2, 669: 3, 2077: 1, 2359: 2, 1238: 2, 175: 2, 6500: 1, 6498: 1, 3371: 2, 6095: 1, 2408: 1, 1459: 3, 433: 3, 4035: 3, 69: 2, 427: 1, 3289: 2, 1078: 1, 1577: 2, 5664: 1, 2917: 1, 3674: 2, 7311: 1, 8022: 1, 5663: 3, 5623: 2, 7690: 1, 5913: 1, 7472: 1, 5750: 1, 1242: 2, 3001: 1, 2559: 2, 1695: 1, 4610: 2, 3476: 2, 1129: 2, 4862: 1, 403: 1, 6560: 1, 6438: 1, 5297: 1, 2934: 1, 4406: 1, 981: 2, 3703: 3, 4234: 1, 2397: 3, 6380: 1, 5628: 3, 4789: 2, 4099: 2, 5739: 1, 7914: 1, 2836: 3, 1473: 1, 7434: 2, 1593: 2, 1685: 2, 3126: 1, 6330: 1, 4050: 1, 1933: 3, 5855: 1, 789: 1, 1054: 1, 5525: 1, 2372: 1, 1591: 1, 6585: 2, 2840: 1, 3918: 2, 688: 2, 1719: 2, 5415: 3, 4581: 2, 5062: 3, 7506: 2, 363: 2, 6793: 2, 3890: 2, 5106: 2, 6731: 1, 574: 1, 6704: 2, 4283: 1, 3625: 3, 4416: 2, 1122: 2, 660: 1, 3963: 1, 2005: 1, 1377: 2, 2675: 1, 2846: 2, 4975: 3, 5722: 1, 797: 1, 4378: 2, 3732: 1, 4075: 1, 3264: 2, 4311: 1, 7424: 1, 4184: 2, 5795: 1, 4241: 3, 2224: 1, 4658: 2, 3143: 2, 4484: 1, 2256: 1, 1185: 1, 5710: 2, 5281: 3, 1610: 1, 7986: 1, 5038: 1, 2991: 2, 2134: 1, 6404: 2, 1694: 2, 1051: 2, 7564: 2, 182: 2, 1814: 2, 4567: 3, 30: 2, 6331: 1, 8077: 2, 3051: 3, 4260: 2, 1077: 1, 1256: 3, 5868: 2, 4736: 1, 2024: 1, 4682: 1, 2591: 1, 997: 1, 4204: 1, 3239: 2, 1098: 1, 7459: 2, 6141: 1, 3124: 1, 2511: 2, 5283: 2, 4189: 1, 7392: 2, 5435: 1, 2137: 1, 3253: 1, 1146: 2, 3280: 1, 4537: 1, 5809: 3, 36: 1, 6053: 1, 5813: 3, 5413: 2, 1367: 1, 4846: 3, 1723: 1, 2858: 3, 5218: 1, 5771: 1, 4081: 2, 7973: 1, 1417: 1, 6990: 1, 1347: 2, 7667: 2, 914: 1, 2548: 1, 7832: 1, 7293: 1, 5981: 1, 4832: 2, 1884: 1, 522: 2, 2143: 1, 792: 2, 7250: 1, 6460: 1, 1801: 2, 3313: 1, 3190: 3, 4233: 1, 326: 2, 1259: 1, 1349: 1, 1047: 2, 3016: 3, 5231: 3, 3122: 1, 6378: 1, 3697: 1, 4810: 1, 1011: 1, 5919: 3, 5596: 1, 763: 1, 6051: 1, 8073: 2, 8109: 1, 6347: 2, 2305: 1, 4939: 1, 1262: 1, 4761: 1, 1383: 2, 7105: 1, 1108: 1, 102: 2, 6065: 2, 7854: 1, 709: 3, 3787: 1, 4116: 1, 3971: 1, 5377: 1, 4608: 1, 4545: 3, 4802: 2, 2023: 2, 7809: 1, 1570: 1, 456: 1, 3903: 2, 2370: 2, 4725: 2, 1670: 2, 1775: 1, 6977: 1, 1590: 3, 89: 1, 4098: 2, 8011: 2, 4216: 2, 4067: 2, 4801: 2, 5893: 2, 5977: 2, 4626: 1, 6038: 2, 100: 1, 3730: 1, 3139: 1, 3391: 1, 5268: 3, 6386: 2, 5065: 1, 7202: 1, 5216: 2, 7126: 1, 6914: 2, 895: 2, 6523: 1, 2381: 2, 4732: 2, 7312: 2, 4316: 3, 6482: 1, 6576: 3, 7335: 2, 6445: 2, 1805: 1, 6551: 1, 418: 1, 3136: 2, 3314: 3, 3114: 1, 5867: 1, 5196: 3, 6244: 1, 6775: 1, 1792: 2, 1258: 3, 872: 2, 1918: 2, 3054: 1, 7103: 2, 3128: 1, 925: 2, 7022: 1, 104: 2, 4086: 2, 5410: 2, 278: 1, 4370: 1, 3482: 1, 2296: 3, 917: 2, 4767: 1, 4038: 1, 7430: 1, 4598: 1, 7743: 2, 2080: 1, 4479: 2, 837: 1, 6485: 1, 5600: 2, 1369: 1, 4091: 2, 7154: 1, 1888: 2, 6360: 1, 5991: 1, 7214: 2, 8016: 2, 6957: 1, 8079: 2, 7770: 2, 5470: 2, 5144: 3, 2605: 1, 967: 1, 4942: 1, 6530: 1, 4383: 3, 4692: 1, 4521: 2, 3571: 2, 7463: 3, 4256: 1, 6158: 1, 6991: 1, 502: 3, 1689: 2, 911: 1, 7890: 1, 1636: 2, 830: 1, 933: 2, 5604: 2, 4393: 1, 268: 2, 6613: 1, 5272: 2, 1746: 2, 6024: 2, 2826: 1, 4198: 3, 2317: 2, 3390: 1, 8139: 3, 8123: 2, 4934: 2, 1079: 1, 3148: 1, 7658: 2, 2726: 1, 6387: 2, 5564: 2, 4980: 3, 6147: 1, 4118: 2, 5482: 1, 400: 2, 2144: 1, 2733: 1, 162: 2, 2530: 1, 4607: 1, 5475: 3, 4813: 2, 1156: 2, 5466: 2, 3292: 2, 3521: 1, 3331: 2, 2622: 1, 3056: 2, 3012: 1, 4151: 2, 6864: 2, 6005: 1, 510: 2, 5491: 1, 7636: 2, 4678: 2, 6749: 2, 2699: 1, 1350: 1, 8138: 2, 6712: 1, 4033: 2, 7642: 2, 7519: 1, 2515: 1, 6721: 1, 6203: 2, 348: 1, 1311: 1, 862: 2, 6747: 2, 6908: 2, 5026: 1, 2083: 2, 3397: 1, 6507: 1, 4278: 1, 6953: 1, 2576: 1, 6887: 2, 753: 3, 1755: 3, 4212: 2, 6409: 1, 2475: 2, 8057: 3, 1674: 1, 577: 1, 2712: 2, 2461: 2, 8018: 1, 3404: 2, 1499: 1, 5131: 2, 6544: 1, 1614: 3, 7263: 1, 5587: 1, 390: 2, 4200: 1, 3064: 2, 1142: 3, 2497: 1, 6419: 1, 638: 3, 313: 2, 2983: 1, 1468: 1, 6682: 1, 4730: 1, 6283: 1, 2465: 2, 5763: 1, 5682: 2, 1282: 1, 2808: 1, 3698: 1, 6707: 1, 4650: 1, 7592: 2, 251: 1, 302: 1, 6297: 2, 6869: 2, 5480: 1, 4921: 2, 2126: 2, 6264: 1, 5585: 1, 7855: 2, 2390: 1, 1420: 2, 5327: 1, 6162: 1, 7365: 1, 408: 1, 4128: 3, 1401: 1, 7165: 2, 1474: 3, 1681: 1, 1966: 1, 3507: 2, 5241: 3, 5256: 3, 2411: 3, 666: 3, 1768: 2, 7449: 1, 6939: 3, 5152: 2, 5122: 1, 1171: 2, 1447: 2, 7947: 2, 5362: 1, 6452: 1, 7023: 2, 1064: 2, 38: 1, 1220: 1, 5825: 1, 6219: 1, 401: 3, 5016: 1, 3713: 2, 4679: 2, 1324: 2, 5914: 1, 2190: 1, 4999: 3, 5732: 1, 731: 1, 5970: 1, 706: 1, 2052: 1, 7223: 1, 7406: 3, 5329: 2, 5734: 2, 7761: 2, 2407: 2, 569: 1, 6125: 2, 3748: 2, 4908: 2, 5147: 1, 7846: 1, 6750: 1, 7493: 1, 6284: 1, 2196: 1, 58: 1, 7749: 2, 4237: 3, 2088: 1, 2761: 1, 827: 2, 5528: 1, 3880: 1, 6213: 2, 5421: 2, 4194: 1, 1305: 3, 1211: 3, 2512: 1, 5254: 1, 5830: 1, 1187: 2, 1421: 1, 780: 2, 1236: 2, 2954: 1, 3847: 3, 7816: 1, 1437: 1, 6307: 1, 6050: 1, 2336: 2, 7460: 3, 2343: 1, 2346: 1, 5094: 1, 7027: 1, 2383: 1, 257: 1, 1812: 2, 7401: 2, 6104: 1, 4533: 2, 1921: 1, 7035: 1, 8066: 1, 2793: 1, 3119: 1, 3487: 2, 7682: 1, 4984: 2, 1244: 1, 6: 2, 5611: 1, 5654: 1, 6844: 1, 4569: 1, 1112: 2, 2703: 1, 2937: 1, 1596: 2, 5586: 1, 5364: 2, 8053: 1, 1580: 1, 2718: 3, 5818: 2, 8065: 2, 4125: 2, 388: 3, 4196: 3, 4945: 2, 1361: 3, 5982: 1, 1655: 1, 7205: 1, 497: 1, 4188: 2, 2807: 1, 5951: 2, 1080: 1, 3408: 2, 256: 3, 7264: 1, 2673: 1, 3175: 1, 5084: 2, 5760: 1, 956: 3, 3806: 2, 4592: 2, 7531: 1, 2342: 1, 4587: 2, 8035: 1, 935: 3, 1029: 2, 6625: 1, 3170: 1, 2961: 1, 1598: 1, 2096: 2, 6676: 3, 5295: 1, 315: 1, 3964: 1, 2959: 2, 2713: 1, 5700: 1, 4055: 2, 7389: 1, 4715: 1, 1307: 1, 87: 2, 168: 1, 3809: 1, 5197: 3, 172: 1, 4977: 3, 3864: 1, 5783: 1, 307: 1, 8006: 1, 4206: 1, 5119: 1, 1853: 1, 4930: 1, 4856: 2, 2545: 1, 796: 1, 6767: 2, 6489: 1, 22: 1, 6204: 1, 6579: 2, 920: 2, 1617: 1, 3619: 1, 1897: 1, 4996: 3, 1395: 2, 4161: 1, 7713: 1, 3902: 1, 2063: 2, 3232: 2, 3911: 2, 3361: 1, 202: 2, 183: 1, 2304: 2, 4962: 2, 4166: 1, 551: 1, 6490: 2, 644: 1, 8046: 2, 3810: 3, 5224: 1, 4571: 1, 4354: 2, 4025: 1, 2341: 2, 1240: 2, 4398: 1, 3479: 1, 163: 1, 3955: 1, 138: 2, 5305: 1, 5843: 1, 1606: 1, 2876: 2, 7452: 1, 4689: 1, 6729: 1, 6449: 1, 4905: 2, 5963: 1, 6098: 2, 1872: 2, 1049: 2, 7240: 1, 3046: 3, 7650: 1, 2602: 1, 1973: 1, 882: 3, 4902: 3, 4037: 3, 6439: 1, 6828: 2, 1341: 1, 7414: 2, 4562: 1, 5298: 1, 4784: 2, 1085: 1, 6063: 3, 4450: 1, 4285: 3, 3600: 2, 367: 1, 7995: 1, 960: 1, 3144: 1, 647: 1, 5606: 2, 7226: 2, 3251: 1, 1196: 1, 7880: 2, 3516: 1, 5523: 2, 3200: 1, 1143: 1, 4624: 2, 3636: 1, 7209: 3, 1786: 2, 1758: 1, 1226: 1, 5426: 1, 741: 1, 6326: 3, 4387: 2, 5217: 3, 7410: 1, 7285: 2, 5885: 1, 7510: 2, 6774: 2, 2634: 2, 7425: 2, 2044: 1, 7029: 1, 2994: 2, 6781: 1, 3173: 2, 5428: 1, 337: 1, 6635: 2, 3941: 1, 7604: 1, 6308: 1, 6344: 1, 4477: 1, 5030: 2, 2297: 2, 1068: 1, 6354: 1, 4268: 2, 930: 1, 7840: 1, 5540: 1, 4483: 1, 6087: 1, 3769: 2, 4355: 2, 7292: 1, 1586: 1, 7633: 1, 1023: 1, 2850: 2, 1748: 1, 7419: 3, 8067: 2, 7393: 1, 6786: 1, 3047: 1, 225: 1, 5035: 1, 7171: 1, 5462: 1, 2051: 1, 7114: 2, 124: 2, 6148: 1, 5451: 1, 3156: 1, 2616: 1, 5347: 1, 4152: 1, 6877: 1, 8118: 1, 4056: 1, 1335: 2, 3002: 2, 8097: 2, 520: 1, 594: 3, 3966: 2, 592: 3, 5802: 1, 1891: 1, 2158: 1, 4351: 2, 2792: 1, 2848: 1, 1534: 2, 6621: 2, 6542: 1, 333: 2, 4723: 1, 7175: 1, 1345: 2, 1847: 1, 4318: 1, 3724: 1, 286: 2, 1212: 2, 2737: 1, 4053: 1, 3309: 1, 1158: 1, 1035: 1, 852: 3, 4829: 1, 7921: 2, 3906: 1, 1682: 2, 21: 1, 5060: 1, 7191: 1, 2429: 2, 5716: 1, 5109: 1, 2060: 1, 3141: 1, 1027: 1, 246: 1, 6364: 1, 2790: 1, 4822: 2, 6371: 1, 5629: 2, 1126: 1, 5411: 1, 1046: 1, 2216: 1, 7597: 2, 217: 1, 2853: 1, 771: 2, 8117: 1, 1828: 1, 512: 2, 7765: 1, 6043: 1, 2939: 1, 7275: 1, 885: 2, 5720: 1, 4210: 2, 4057: 2, 5581: 3, 676: 1, 3268: 1, 3235: 1, 1130: 1, 5112: 1, 5900: 1, 961: 3, 4909: 1, 4455: 1, 3296: 3, 6327: 1, 6472: 1, 846: 1, 5672: 2, 4933: 2, 6567: 1, 3621: 2, 3771: 1, 6519: 1, 6607: 1, 726: 1, 272: 2, 6969: 1, 6717: 1, 1376: 3, 5905: 1, 1434: 1, 4663: 2, 6901: 1, 2787: 2, 6829: 1, 7699: 2, 2957: 1, 4885: 1, 4865: 2, 6256: 1, 646: 1, 5978: 1, 860: 1, 3502: 1, 7304: 1, 5689: 1, 4593: 1, 2038: 1, 2021: 1, 1365: 2, 7476: 2, 6333: 1, 1671: 2, 3112: 1, 938: 3, 7919: 1, 4041: 1, 3745: 2, 6437: 2, 6863: 3, 2246: 1, 5018: 2, 5759: 2, 4888: 1, 46: 2, 2093: 1, 5777: 1, 4790: 2, 5834: 1, 343: 2, 492: 2, 7790: 2, 3438: 1, 2230: 1, 5796: 1, 212: 1, 6815: 1, 832: 1, 6238: 1, 5284: 1, 972: 1, 8048: 1, 4544: 2, 2916: 1, 1953: 1, 3499: 1, 5365: 2, 7189: 1, 6578: 1, 995: 2, 6549: 2, 6233: 1, 2462: 1, 5357: 1, 4763: 2, 3444: 1, 1772: 1, 2938: 1, 5345: 1, 6407: 2, 2747: 1, 1806: 3, 4636: 1, 7709: 1, 5465: 2, 1535: 1, 4672: 2, 2337: 2, 1857: 1, 7104: 1, 6370: 1, 3573: 1, 2467: 1, 5303: 1, 1698: 1, 4700: 3, 6381: 2, 3934: 1, 4731: 2, 4481: 1, 443: 1, 7966: 2, 4120: 1, 2030: 1, 33: 1, 767: 1, 4511: 2, 2647: 2, 174: 3, 6060: 1, 7021: 2, 328: 1, 7255: 1, 7259: 1, 849: 2, 3182: 1, 5439: 1, 3130: 2, 3705: 2, 6376: 1, 4246: 2, 690: 1, 5799: 1, 868: 1, 3073: 1, 5535: 1, 3954: 1, 5208: 1, 3622: 2, 4807: 2, 5551: 2, 3592: 2, 6436: 1, 3032: 1, 2463: 1, 1128: 1, 1765: 1, 8021: 1, 1579: 2, 6126: 2, 2444: 2, 1145: 2, 2637: 3, 4965: 1, 1153: 1, 4036: 2, 4875: 1, 7060: 2, 1887: 3, 6904: 1, 6201: 1, 1615: 1, 6226: 1, 1007: 1, 8075: 1, 3374: 2, 989: 2, 2195: 2, 3424: 1, 1472: 1, 966: 2, 340: 1, 7810: 1, 1539: 3, 2161: 2, 798: 2, 3688: 1, 3734: 1, 4514: 1, 4560: 1, 1485: 1, 5273: 2, 6907: 1, 4550: 1, 1131: 2, 6611: 2, 2443: 1, 4699: 1, 7244: 1, 7962: 2, 458: 2, 1332: 2, 3164: 1, 284: 1, 5423: 1, 3840: 1, 6992: 1, 2101: 1, 3983: 1, 4324: 1, 1560: 3, 7151: 1, 4121: 1, 6705: 1, 2952: 3, 7938: 2, 233: 1, 3945: 1, 7857: 1, 3872: 2, 5548: 2, 2071: 1, 3355: 1, 2833: 2, 6465: 1, 6054: 1, 4634: 1, 3722: 1, 535: 3, 913: 1, 7101: 1, 1015: 1, 7538: 3, 5966: 1, 851: 3, 4623: 1, 5287: 1, 6915: 1, 51: 1, 2459: 2, 214: 1, 3318: 1, 4914: 2, 2330: 2, 3298: 2, 5149: 2, 546: 3, 6076: 2, 8063: 1, 6324: 1, 5649: 2, 6902: 1, 3987: 3, 4563: 1, 2153: 1, 5530: 2, 823: 3, 4425: 2, 3797: 1, 4122: 1, 4651: 1, 7686: 1, 5113: 2, 201: 1, 8108: 1, 978: 2, 7733: 1, 3199: 1, 5457: 1, 1140: 1, 2426: 3, 3531: 2, 4923: 2, 3687: 1, 6202: 1, 4460: 1, 1053: 1, 1292: 1, 2570: 1, 2723: 1, 143: 1, 3693: 1, 4828: 2, 4561: 1, 6294: 1, 4259: 1, 7785: 2, 6620: 1, 2949: 2, 4961: 1, 7907: 1, 6651: 2, 3212: 1, 6310: 1, 5485: 3, 2910: 1, 4007: 1, 5707: 1, 6424: 1, 8004: 1, 2786: 1, 5037: 2, 1412: 1, 2476: 3, 4827: 1, 5337: 1, 7871: 1, 6891: 2, 4716: 2, 191: 2, 1866: 1, 4410: 1, 1058: 1, 825: 2, 1402: 1, 2076: 1, 1360: 1, 7019: 2, 3528: 1, 4864: 1, 6300: 1, 2049: 2, 595: 2, 5025: 3, 2485: 1, 2586: 1, 5120: 2, 5389: 2, 7627: 1, 4703: 2, 5924: 1, 4652: 2, 4325: 3, 3503: 1, 3260: 2, 2795: 1, 5006: 1, 2771: 1, 5029: 2, 3384: 1, 4691: 1, 179: 2, 6554: 1, 2920: 1, 4096: 1, 3345: 2, 5493: 1, 7500: 1, 1044: 2, 7412: 2, 4963: 1, 596: 2, 6255: 1, 4869: 1, 7352: 1, 1696: 1, 671: 1, 779: 1, 3604: 1, 3863: 1, 3165: 1, 3103: 1, 5510: 1, 513: 1, 3982: 1, 5752: 2, 2867: 2, 5807: 2, 2611: 1, 5859: 1, 2087: 1, 5614: 1, 4714: 1, 6410: 1, 7521: 1, 6748: 1, 7037: 1, 3208: 1, 2448: 3, 4181: 1, 2193: 1, 5467: 2, 2492: 1, 3288: 1, 4754: 2, 4108: 1, 6096: 1, 2526: 1, 1159: 1, 579: 1, 6094: 1, 5021: 1, 5107: 1, 7160: 2, 7945: 1, 6040: 1, 1315: 1, 2445: 1, 8088: 2, 2261: 2, 5737: 1, 2103: 1, 1506: 1, 2970: 2, 6253: 1, 602: 1, 6366: 1, 5499: 1, 6570: 1, 359: 1, 7882: 1, 411: 1, 4453: 1, 321: 1, 4981: 1, 7999: 2, 6183: 1, 2706: 1, 1711: 1, 2300: 2, 7691: 2, 7490: 1, 2477: 1, 2690: 1, 7234: 1, 4657: 1, 2617: 1, 7442: 2, 900: 1, 7613: 1, 3792: 1, 1291: 1, 5063: 1, 5574: 1, 787: 1, 5765: 2, 7122: 2, 6571: 1, 6467: 1, 3247: 1, 4857: 1, 5310: 1, 8012: 2, 6475: 1, 5031: 2, 3368: 1, 3656: 1, 6029: 1, 7898: 2, 7924: 1, 440: 2, 2457: 1, 4446: 1, 6494: 1, 928: 3, 1353: 1, 3927: 1, 4462: 1, 7878: 1, 742: 1, 5004: 1, 3536: 1, 8120: 1, 3085: 1, 1020: 1, 5247: 1, 3802: 1, 109: 1, 3534: 2, 6764: 1, 6618: 1, 4360: 2, 3196: 1, 4741: 1, 6246: 1, 4924: 1, 4286: 1, 7428: 1, 4695: 2, 208: 1, 2907: 1, 6799: 1, 4368: 1, 2567: 1, 3388: 2, 146: 1, 7620: 2, 636: 2, 919: 2, 750: 1, 5962: 1, 7886: 1, 7496: 1, 5550: 1, 6698: 1, 4555: 1, 2852: 1, 2327: 2, 5071: 1, 2221: 1, 5406: 1, 4670: 1, 2486: 2, 1026: 1, 1523: 1, 7542: 1, 3695: 1, 3877: 1, 3363: 1, 3618: 2, 2384: 2, 6143: 1, 3859: 2, 1622: 1, 2978: 1, 3344: 1, 3913: 1, 7619: 1, 473: 2, 2174: 1, 4372: 2, 4458: 1, 7372: 1, 6072: 1, 2034: 1, 3475: 1, 6562: 2, 6176: 1, 6288: 1, 1529: 1, 7067: 1, 2147: 1, 4575: 3, 3135: 1, 4878: 2, 7436: 1, 2918: 1, 3652: 1, 7000: 1, 3978: 1, 2797: 1, 4925: 1, 6295: 1, 3822: 1, 7847: 1, 70: 1, 3762: 1, 92: 2, 5390: 1, 2974: 1, 6346: 1, 1984: 2, 2932: 1, 1717: 1, 6880: 1, 1544: 2, 876: 1, 2253: 1, 7782: 1, 99: 2, 6687: 1, 7280: 2, 3349: 2, 6234: 1, 3590: 2, 5209: 1, 322: 1, 5340: 1, 2707: 2, 7837: 2, 645: 2, 7998: 1, 1198: 1, 32: 2, 5925: 2, 3634: 1, 6521: 1, 1017: 1, 3231: 1, 6423: 1, 3116: 1, 3291: 2, 2164: 1, 7722: 1, 3556: 1, 7274: 1, 2239: 1, 604: 2, 3358: 1, 1727: 2, 2705: 1, 4954: 1, 6546: 1, 1048: 1, 2345: 1, 5593: 1, 5941: 1, 1514: 1, 3279: 2, 4452: 1, 6398: 1, 477: 1, 7925: 1, 6870: 1, 6171: 1, 4219: 1, 1581: 1, 5526: 1, 6547: 1, 723: 1, 4502: 2, 5922: 2, 2508: 1, 1411: 1, 60: 1, 387: 1, 6561: 1, 1612: 2, 6047: 2, 5498: 1, 2270: 2, 3997: 1, 2: 1, 6020: 1, 2636: 1, 6010: 1, 4154: 2, 773: 1, 7403: 1, 4683: 1, 5753: 1, 5595: 1, 3989: 1, 224: 1, 3727: 1, 4441: 1, 3807: 1, 4935: 1, 1628: 1, 3108: 1, 1204: 2, 2743: 2, 2357: 1, 5185: 1, 6509: 1, 1943: 2, 954: 1, 6122: 1, 3278: 1, 5744: 2, 1963: 1, 5339: 1, 8119: 1, 7948: 1, 4244: 1, 836: 2, 8104: 1, 6417: 1, 1738: 1, 5620: 1, 2437: 1, 3489: 1, 515: 2, 3522: 1, 4614: 1, 2573: 1, 4245: 1, 3609: 2, 48: 1, 5837: 1, 468: 2, 4595: 1, 7958: 1, 1824: 1, 2809: 1, 7501: 1, 1524: 1, 7273: 1, 1124: 1, 3778: 1, 3049: 1, 7088: 1, 883: 2, 7084: 1, 1115: 2, 6756: 1, 5019: 1, 2880: 3, 1173: 1, 2128: 1, 2754: 2, 4899: 2, 3616: 1, 6110: 1, 6827: 1, 232: 1, 6408: 1, 1885: 2, 1683: 1, 7086: 1, 3685: 1, 6189: 2, 1914: 2, 5072: 2, 4438: 1, 4943: 1, 7833: 1, 4958: 1, 7109: 1, 1002: 1, 2062: 1, 7290: 1, 7804: 1, 3940: 1, 5313: 1, 7360: 1, 2405: 2, 4208: 1, 7646: 2, 44: 2, 2589: 1, 4860: 1, 3316: 1, 3538: 1, 6995: 1, 80: 1, 6493: 1, 5070: 1, 3783: 1, 7324: 2, 3270: 1, 12: 1, 859: 1, 507: 2, 7130: 1, 4093: 1, 3398: 1, 2676: 1, 2013: 1, 637: 1, 7738: 1, 3340: 1, 730: 1, 2820: 1, 7391: 2, 2368: 1, 1759: 1, 3192: 1, 3186: 2, 845: 1, 607: 1, 4420: 1, 4833: 1, 2175: 1, 7509: 1, 1962: 1, 4242: 1, 7533: 1, 1328: 2, 5992: 1, 6768: 1, 3640: 2, 2898: 1, 101: 1, 5081: 1, 6525: 2, 3267: 2, 4848: 1, 5191: 2, 3304: 1, 2802: 2, 7683: 1, 2295: 1, 2205: 1, 4409: 1, 5164: 2, 2098: 2, 1429: 1, 72: 1, 5890: 1, 3439: 1, 4061: 2, 3027: 1, 1592: 1, 3843: 1, 5650: 1, 7141: 1, 2466: 1, 111: 1, 170: 1, 1448: 1, 5417: 1, 7383: 1, 3564: 1, 7558: 1, 3410: 1, 1374: 1, 65: 1, 259: 1, 720: 1, 3118: 1, 1789: 2, 3513: 1, 7653: 1, 7798: 1, 5322: 1, 7328: 1, 7576: 1, 5993: 1, 324: 2, 128: 1, 2041: 1, 3217: 2, 4092: 2, 3166: 1, 3241: 1, 3644: 1, 2551: 1, 2966: 1, 7003: 1, 8054: 1, 1782: 1, 4451: 1, 1703: 2, 807: 3, 5282: 1, 7300: 1, 2458: 1, 2912: 1, 1929: 1, 2969: 1, 4517: 1, 3099: 1, 7839: 1, 3328: 1, 3417: 1, 3717: 2, 3696: 1, 3496: 1, 2749: 1, 6535: 1, 5450: 2, 3541: 1, 1729: 1, 4799: 1, 6187: 1, 5202: 1, 1917: 1, 6539: 1, 8005: 1, 5808: 1, 5318: 1, 3246: 1, 2824: 1, 6384: 1, 7177: 1, 3306: 1, 1578: 2, 4272: 2, 2053: 1, 6069: 1, 3560: 1, 1297: 1, 2947: 1, 1004: 1, 1835: 1, 2065: 1, 3087: 1, 6982: 1, 2271: 1, 3015: 1, 6924: 1, 3442: 1, 3428: 1, 45: 1, 5529: 2, 5572: 1, 1494: 1, 2686: 1, 3315: 1, 2149: 1, 177: 1, 4480: 1, 4974: 1, 6645: 1, 3582: 1, 7788: 1, 5267: 1, 3506: 1, 442: 1, 1595: 1, 5597: 1, 980: 1, 1555: 2, 2201: 1, 4753: 2, 7574: 1, 7732: 1, 500: 2, 6934: 1, 678: 1, 785: 2, 4299: 1, 4501: 1, 1852: 1, 519: 1, 5425: 1, 724: 1, 3365: 1, 5500: 1, 3785: 1, 6056: 1, 1896: 1, 3504: 1, 357: 1, 6240: 1, 3483: 1, 6776: 2, 6847: 1, 657: 1, 506: 1, 4402: 1, 3808: 1, 4791: 1, 5095: 2, 1893: 1, 2516: 1, 6932: 1, 1686: 1, 2148: 1, 3138: 1, 3886: 1, 5230: 2, 2184: 1, 4986: 1, 7974: 2, 4654: 1, 975: 1, 7421: 1, 7718: 1, 3928: 1, 2108: 1, 2856: 1, 1299: 1, 1950: 1, 5048: 1, 5330: 1, 6433: 1, 2619: 2, 298: 1, 1907: 1, 4435: 1, 7596: 1, 3581: 1, 4494: 1, 1875: 1, 1161: 1, 4528: 1, 8026: 1, 531: 1, 4040: 1, 2693: 1, 5178: 2, 2490: 1, 3759: 1, 2842: 1, 3238: 1, 1945: 1, 1597: 1, 6186: 1, 3800: 1, 5366: 1, 590: 1, 1284: 1, 3908: 1, 1271: 1, 1308: 1, 4546: 2, 6278: 1, 5168: 1, 2942: 1, 2555: 2, 969: 2, 2930: 2, 141: 1, 1684: 1, 6929: 1, 3678: 1, 7964: 1, 916: 1, 7483: 1, 6742: 1, 5800: 1, 7378: 1, 1398: 1, 6442: 1, 4385: 1, 7295: 1, 4549: 1, 2166: 1, 6857: 1, 3851: 1, 8081: 1, 963: 1, 2396: 1, 6697: 1, 4002: 1, 5262: 1, 616: 1, 4941: 1, 2303: 1, 5833: 1, 3067: 1, 2482: 1, 5757: 1, 5646: 1, 5584: 1, 625: 1, 4031: 1, 1641: 1, 1414: 1, 73: 1, 415: 1, 3875: 1, 1169: 1, 3220: 1, 2542: 2, 7583: 1, 2784: 1, 2095: 1, 3373: 1, 1481: 1, 5245: 1, 6044: 1, 74: 1, 4087: 2, 7441: 1, 553: 1, 5142: 1, 5958: 1, 3214: 1, 4029: 1, 6812: 1, 4783: 1, 5013: 1, 3899: 1, 1567: 1, 2189: 1, 185: 1, 7983: 1, 1522: 1, 6021: 1, 1174: 2, 1296: 1, 7852: 1, 4729: 1, 3683: 1, 5956: 1, 6564: 1, 8111: 1, 5671: 1, 404: 1, 2248: 1, 1623: 1, 3053: 1, 652: 1, 3010: 1, 1466: 1, 3651: 1, 856: 1, 4959: 1, 4717: 1, 6823: 1, 1234: 2, 6026: 1, 6279: 1, 5138: 1, 3176: 1, 3553: 1, 835: 1, 487: 1, 2544: 1, 7701: 1, 6944: 1, 1094: 1, 5645: 1, 3282: 1, 3261: 2, 3501: 1, 775: 2, 5686: 1, 5096: 1, 5220: 1, 2585: 1, 6840: 1, 1449: 1, 7834: 1, 4739: 1, 2758: 1, 6515: 1, 508: 1, 5792: 1, 631: 2, 3510: 1, 7350: 1, 6099: 1, 368: 2, 1503: 1, 4500: 1, 7868: 1, 318: 1, 5176: 1, 42: 1, 5189: 1, 7026: 1, 4772: 1, 490: 1, 6355: 1, 4922: 1, 1939: 1, 3327: 1, 8055: 1, 3970: 1, 2992: 1, 7899: 1, 2015: 1, 7319: 1, 3337: 1, 7772: 1, 7096: 1, 2950: 1, 7802: 1, 3048: 1, 5815: 1, 6893: 1, 3578: 1, 7720: 1, 5711: 1, 505: 1, 719: 1, 1267: 1, 8052: 2, 7233: 1, 1084: 1, 151: 2, 7965: 1, 7107: 1, 15: 1, 1444: 1, 285: 1, 7638: 1, 1631: 1, 3763: 1, 2884: 1, 2958: 1, 4176: 1, 428: 1, 8045: 1, 34: 1, 3480: 1, 4175: 1, 7911: 1, 6833: 1, 1715: 2, 1260: 1, 7249: 1, 790: 1, 4418: 1, 6728: 1, 2358: 3, 6210: 1, 1125: 1, 3992: 1, 5648: 1, 1110: 1, 2006: 1, 1909: 1, 739: 2, 2349: 1, 1979: 1, 614: 1, 3440: 1, 838: 1, 3006: 1, 6773: 1, 6866: 1, 1975: 1, 5644: 1, 1959: 1, 7461: 2, 7775: 1, 216: 1, 754: 1, 4384: 1, 97: 1, 207: 2, 6205: 1, 3030: 1, 7557: 1, 5946: 1, 4599: 1, 7181: 1, 1754: 1, 2211: 1, 2155: 1, 4988: 1, 489: 1, 3663: 1, 3559: 1, 5612: 1, 4345: 1, 2803: 1, 3630: 1, 2418: 1, 6160: 1, 1385: 1, 493: 1, 1197: 1, 5279: 1, 1958: 2, 2835: 1, 2151: 1, 3184: 1, 877: 1, 7632: 1, 364: 1, 7522: 1, 7955: 1, 75: 1, 4523: 1, 7610: 1, 1957: 1, 6402: 1, 234: 1, 6722: 1, 5153: 1, 7587: 1, 7216: 1, 5192: 1, 3905: 1, 2738: 1, 4868: 1, 6157: 1, 3468: 1, 2811: 1, 4519: 1, 49: 1, 1635: 1, 6282: 1, 6459: 1, 2059: 1, 4797: 1, 3467: 1, 3106: 1, 4694: 1, 247: 1, 3110: 1, 7940: 1, 5075: 1, 6235: 1, 1924: 1, 6382: 1, 3633: 1, 2519: 1, 2422: 1, 7099: 1, 7394: 1, 2631: 1, 4240: 1, 1972: 1, 2129: 1, 2046: 1, 2382: 1}

In [56]:
dist = INaturalistSSL(root='../../datasets', filename='./data/iNaturalist18_train.txt').get_data_dist()
print(dist)

[(2820, 1000), (2757, 1000), (4241, 1000), (3383, 1000), (3804, 1000), (3087, 1000), (3209, 1000), (3612, 1000), (1566, 1000), (3250, 1000), (3191, 1000), (2969, 1000), (3586, 1000), (4436, 1000), (3553, 1000), (1447, 1000), (3609, 1000), (3452, 1000), (1652, 1000), (2844, 1000), (3836, 1000), (3357, 1000), (1493, 1000), (3807, 1000), (3986, 1000), (3749, 1000), (3582, 1000), (2944, 1000), (4529, 1000), (2809, 1000), (4134, 1000), (2062, 1000), (3890, 1000), (4069, 1000), (3119, 1000), (488, 1000), (3676, 1000), (3345, 1000), (4245, 1000), (3822, 1000), (2015, 1000), (3412, 989), (2070, 983), (2785, 966), (3817, 961), (4373, 961), (4068, 950), (248, 948), (3828, 935), (3167, 930), (3009, 928), (1403, 922), (5807, 920), (3779, 913), (3277, 910), (4078, 900), (4450, 876), (2918, 876), (3096, 869), (3242, 857), (6134, 854), (1595, 846), (2679, 835), (3366, 800), (3905, 793), (1068, 783), (2759, 776), (3287, 768), (2747, 763), (3356, 761), (2036, 760), (3877, 760), (7512, 758), (3861, 756)

In [63]:
# Loop through classids sorted by sample count
baseline = {'Many': 0, 'Medium': 0, 'Few': 0}
lotar = {'Many': 0, 'Medium': 0, 'Few': 0}
classes_per_cat = len(dist) / 3
print(classes_per_cat)
for i, (classid, _) in enumerate(dist):
    if i < classes_per_cat:
        cat_name = 'Many'
    elif i > classes_per_cat and i < 2*classes_per_cat:
        cat_name = 'Medium'
    else:
        cat_name = 'Few'
    if classid in baseline_accs:
        baseline[cat_name] += baseline_accs[classid]
    if classid in lotar_accs:
        lotar[cat_name] += lotar_accs[classid]
print(baseline)
print(lotar)

2714.0
{'Many': 3362, 'Medium': 3793, 'Few': 4096}
{'Many': 2896, 'Medium': 3396, 'Few': 3579}
